In [1]:
import os
import pandas as pd

# Specify the folder containing your Excel files
data_folder = "Data"

# Define the column names
col_names = ['Case ID', 'Suspect Product Active Ingredients', 'Reason for Use', 'Reactions', 'Serious', 'Outcomes', 'Sex', 'Patient Age', 'Patient Weight']

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame(columns=col_names)

# Iterate over all Excel files in the folder
for file in os.listdir(data_folder):
    if file.endswith(".xlsx"):  # Check if the file is an Excel file
        file_path = os.path.join(data_folder, file)
        # Read the Excel file and add it to the combined DataFrame
        data = pd.read_excel(file_path, usecols=col_names)  # Load only the specified columns
        combined_data = pd.concat([combined_data, data], ignore_index=True)

# Display the combined dataset
#print(combined_data)

In [2]:
selected_columns = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions',  'Serious', 'Sex', 'Patient Age', 'Patient Weight']

#df = data[selected_columns]
df = combined_data[selected_columns]

#df.head()
#print(df)

In [3]:
df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')
df_split_temp = df.explode('Suspect Product Active Ingredients', ignore_index=True)

df_split_temp['Reactions'] = df_split_temp['Reactions'].str.split(';')
df_split = df_split_temp.explode('Reactions', ignore_index = True)

#df_split.head()

C:\Users\Dara\AppData\Local\Temp\ipykernel_202420\1393876987.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Suspect Product Active Ingredients'] = df['Suspect Product Active Ingredients'].str.split(';')


In [4]:
df_multi = pd.get_dummies(df_split, columns = ['Suspect Product Active Ingredients', 'Reactions'], prefix=['Product', 'Reaction'], prefix_sep='_')
reactions = [col for col in df_multi.columns if col.startswith("Reaction_")]
df_reaction = df_multi[reactions]
df_multi = df_multi.groupby('Case ID').max().reset_index()

columns_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Reactions', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']

columns_to_convert = [col for col in df_multi.columns if col.startswith("Product_") or col.startswith("Reaction_")] #not in columns_to_exclude]

df_multi[columns_to_convert] = df_multi[columns_to_convert].astype(int)

df_final = df_multi
#print(df_multi.columns)
#print(df_final)

#df_final.head()

In [5]:
df_encoded = df_multi.copy()
#print(df_encoded.columns)
#print(df_split['Patient Age'])
df_encoded['Patient Age'] = df_encoded['Patient Age'].astype(str)
df_encoded['Patient Age'] = df_encoded['Patient Age'].str.replace(r'\D+', '', regex=True)
df_encoded['Patient Age'] = pd.to_numeric(df_encoded['Patient Age'], errors='coerce')  # Converts to numeric, sets invalid values to NaN

df_encoded['Patient Weight'] = df_encoded['Patient Weight'].replace('Not Specified', "0 KG")
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].astype(str)
df_encoded['Patient Weight'] = df_encoded['Patient Weight'].str.replace(r'[^\d.]', '', regex=True)
df_encoded['Patient Weight'] = pd.to_numeric(df_encoded['Patient Weight'], errors='coerce')

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

#df_encoded['Suspect Product Active Ingredients'] = label_encoder.fit_transform(df_encoded['Suspect Product Active Ingredients'])
df_encoded['Sex'] = label_encoder.fit_transform(df_encoded['Sex'])
df_encoded['Serious'] = label_encoder.fit_transform(df_encoded['Serious'])
#print(df_encoded.isnull().sum())
df_encoded.dropna(inplace=True)
#print(df_encoded.isnull().sum())
#df_encoded.head()

In [6]:
# Sum columns that start with "Reaction_"
reaction_columns = df_encoded.filter(like="Reaction_", axis=1)
column_sums = reaction_columns.sum(axis=0)

#print(len(reaction_columns.columns))
# Set the threshold
threshold = 50 #300

# Filter column names with sums greater than the threshold
columns_above_threshold = column_sums[column_sums > threshold].index.tolist()

#print(columns_above_threshold)
#print(len(columns_above_threshold))

filtered_df = df_encoded[df_encoded[columns_above_threshold].sum(axis=1) > 0]

# Print the number of rows before and after filtering
#print(f"Number of rows before filtering: {df_encoded.shape[0]}")
#print(f"Number of rows after removing all-zero rows: {filtered_df.shape[0]}")

In [8]:
cols_to_exclude = ['Case ID', 'Suspect Product Active Ingredients', 'Serious' ,'Reactions']

feature_cols = [col for col in df_encoded.columns if col not in cols_to_exclude and not col.startswith("Reaction_")]

print("Feature Columns of First Model:", feature_cols)
print(len(feature_cols))

X_serious = df_encoded[feature_cols]
Y_serious = df_encoded.Serious

# Reaction Predictor

feat_cols = ["Serious"]
feat_cols.extend(feature_cols)

print("Feature Columns of Second Model: ",feat_cols)
print(len(feat_cols))

predict_cols = []#["Serious"]
reaction_cols = columns_above_threshold #[col for col in df_encoded.columns if col.startswith("Reaction_")]
predict_cols.extend(reaction_cols)

print("Predicted columns of Second Model:", predict_cols)
print(len(predict_cols))

df_filtered = df_encoded[df_encoded[predict_cols].sum(axis=1) > 0]

X_reaction = df_filtered[feat_cols]
Y_reaction = df_filtered[predict_cols]

Feature Columns of First Model: ['Sex', 'Patient Age', 'Patient Weight', 'Product_.Alpha.-Acetyldigoxin', 'Product_.Alpha.-Glucose-1-Phosphate Disodium Tetrahydrate', 'Product_.Alpha.-Lipoic Acid', 'Product_.Alpha.-Tocopherol', 'Product_.Alpha.-Tocopherol Acetate', 'Product_.Alpha.-Tocopherol Acetate, D-', 'Product_.Alpha.-Tocopherol Acetate, Dl-', 'Product_.Alpha.-Tocopherol Acetate, Dl-\\.Beta.-Carotene\\Calcium Ascorbate\\Calcium Carbonate\\Calcium Pantothenate\\Cholecalciferol\\Cyanocobalamin\\Ferrous Fumarate\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Mononitrate\\Zinc Oxide', 'Product_.Alpha.-Tocopherol Succinate, D-', 'Product_.Alpha.-Tocopherol Succinate, Dl-', 'Product_.Alpha.-Tocopherol, D-', 'Product_.Alpha.-Tocopherol, Dl-', 'Product_.Alpha.-Tocopherol, Dl-\\Fish Oil', "Product_.Alpha.-Tocopherol\\.Alpha.-Tocopherol Acetate, Dl-\\Ascorbic Acid\\Biotin\\Cyanocobalamin\\Dexpanthenol\\Ergocalciferol\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochlori

In [10]:
first_model_features = ['Sex', 'Patient Age', 'Patient Weight', 'Product_.Alpha.-Acetyldigoxin', 'Product_.Alpha.-Glucose-1-Phosphate Disodium Tetrahydrate', 'Product_.Alpha.-Lipoic Acid', 'Product_.Alpha.-Tocopherol', 'Product_.Alpha.-Tocopherol Acetate', 'Product_.Alpha.-Tocopherol Acetate, D-', 'Product_.Alpha.-Tocopherol Acetate, Dl-', 'Product_.Alpha.-Tocopherol Acetate, Dl-\\.Beta.-Carotene\\Calcium Ascorbate\\Calcium Carbonate\\Calcium Pantothenate\\Cholecalciferol\\Cyanocobalamin\\Ferrous Fumarate\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Mononitrate\\Zinc Oxide', 'Product_.Alpha.-Tocopherol Succinate, D-', 'Product_.Alpha.-Tocopherol Succinate, Dl-', 'Product_.Alpha.-Tocopherol, D-', 'Product_.Alpha.-Tocopherol, Dl-', 'Product_.Alpha.-Tocopherol, Dl-\\Fish Oil', "Product_.Alpha.-Tocopherol\\.Alpha.-Tocopherol Acetate, Dl-\\Ascorbic Acid\\Biotin\\Cyanocobalamin\\Dexpanthenol\\Ergocalciferol\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Retinol\\Riboflavin 5'-Phosphate Sodium\\Thiamine", 'Product_.Alpha.-Tocopherol\\Allantoin', 'Product_.Beta.-Acetyldigoxin', 'Product_.Beta.-Carotene', 'Product_Abacavir Sulfate', 'Product_Abaloparatide', 'Product_Abatacept', 'Product_Abiraterone Acetate', 'Product_Abiraterone\\Abiraterone Acetate', 'Product_Abobotulinumtoxina', 'Product_Acarbose', 'Product_Acebutolol', 'Product_Acebutolol Hydrochloride', 'Product_Acenocoumarol', 'Product_Acetaminophen', 'Product_Acetaminophen\\Belladonna Leaf\\Caffeine\\Opium', 'Product_Acetaminophen\\Butabarbital\\Caffeine', 'Product_Acetaminophen\\Caffeine\\Codeine', 'Product_Acetaminophen\\Caffeine\\Codeine Phosphate', 'Product_Acetaminophen\\Caffeine\\Dihydrocodeine Bitartrate', 'Product_Acetaminophen\\Caffeine\\Opium', 'Product_Acetaminophen\\Caffeine\\Propyphenazone', 'Product_Acetaminophen\\Chlorpheniramine\\Dextromethorphan\\Diphenhydramine\\Doxylamine\\Pseudoephedrine', 'Product_Acetaminophen\\Chlorpheniramine\\Dextromethorphan\\Pseudoephedrine Hydrochloride', 'Product_Acetaminophen\\Codeine', 'Product_Acetaminophen\\Codeine Phosphate', 'Product_Acetaminophen\\Codeine Phosphate\\Meprobamate', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Doxylamine Succinate', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Pseudoephedrine', 'Product_Acetaminophen\\Dihydrocodeine', 'Product_Acetaminophen\\Diphenhydramine Hydrochloride', 'Product_Acetaminophen\\Hydrocodone', 'Product_Acetaminophen\\Hydrocodone Bitartrate', 'Product_Acetaminophen\\Opium', 'Product_Acetaminophen\\Oxycodone Hydrochloride', 'Product_Acetaminophen\\Propoxyphene', 'Product_Acetaminophen\\Propoxyphene Hydrochloride', 'Product_Acetaminophen\\Propoxyphene Napsylate', 'Product_Acetaminophen\\Tramadol', 'Product_Acetaminophen\\Tramadol Hydrochloride', 'Product_Acetazolamide', 'Product_Acetohexamide', 'Product_Acetylcysteine', 'Product_Acetylleucine', 'Product_Acitretin', 'Product_Aclidinium Bromide', 'Product_Acotiamide Hydrochloride', 'Product_Acyclovir', 'Product_Acyclovir\\Lidocaine\\Tetracaine Hydrochloride', 'Product_Adalimumab', 'Product_Adapalene', 'Product_Adenosine', 'Product_Adenosine Phosphate\\Cyanocobalamin\\Lidocaine Hydrochloride\\Pyridoxine Hydrochloride\\Thiamine Hydrochloride', 'Product_Adrafinil', 'Product_Afatinib', 'Product_Afatinib Dimaleate', 'Product_Aflibercept', 'Product_Alanine, Dl-\\Arginine\\Cysteine Hydrochloride\\Glycine\\Histidine\\Isoleucine\\Leucine\\Lysine Acetate\\Methionine\\Phenylalanine\\Phosphoric Acid\\Proline\\Serine\\Threonine\\Tryptophan\\Valine', 'Product_Alanine\\Arginine\\Aspartic Acid\\Glutamic Acid\\Glycine\\Histidine\\Isoleucine\\Leucine\\Lysine Acetate\\Methionine\\Phenylalanine\\Proline\\Serine\\Threonine\\Tryptophan\\Tyrosine\\Valine', 'Product_Alanyl Glutamine', 'Product_Albiglutide', 'Product_Albumin Human', 'Product_Albuterol', 'Product_Albuterol Sulfate', 'Product_Albuterol Sulfate\\Ipratropium Bromide', 'Product_Albuterol\\Ipratropium', 'Product_Albuterol\\Ipratropium Bromide', 'Product_Albuterol\\Theophylline', 'Product_Alcaftadine', 'Product_Alcohol', 'Product_Alectinib', 'Product_Alemtuzumab', 'Product_Alendronate Sodium', 'Product_Alendronic Acid', 'Product_Alfacalcidol', 'Product_Alfuzosin', 'Product_Alfuzosin Hydrochloride', 'Product_Alginic Acid\\Aluminum Hydroxide\\Magnesium Trisilicate\\Sodium Alginate\\Sodium Bicarbonate', 'Product_Alirocumab', 'Product_Aliskiren', 'Product_Aliskiren Hemifumarate', 'Product_Allantoin', 'Product_Allopurinol', 'Product_Almond Oil\\Benzalkonium Chloride\\Dexpanthenol', 'Product_Aloe Vera Leaf', 'Product_Alogliptin', 'Product_Alogliptin Benzoate', 'Product_Alogliptin Benzoate\\Metformin Hydrochloride', 'Product_Alogliptin Benzoate\\Pioglitazone Hydrochloride', 'Product_Alpelisib', 'Product_Alprazolam', 'Product_Alprostadil', 'Product_Alteplase', 'Product_Althiazide\\Spironolactone', 'Product_Aluminum', 'Product_Aluminum Hydroxide', 'Product_Aluminum Hydroxide \\Calcium Carbonate\\Sodium Alginate\\Sodium Bicarbonate', 'Product_Aluminum Hydroxide\\Dimethicone\\Magnesium Aluminum Silicate\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Dimethicone\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Magnesium Carbonate', 'Product_Aluminum Hydroxide\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Magnesium Trisilicate', 'Product_Aluminum Sulfate Tetradecahydrate\\Calcium Acetate Monohydrate', 'Product_Amantadine Hydrochloride', 'Product_Ambenonium Chloride', 'Product_Ambrisentan', 'Product_Ambroxol', 'Product_Ambroxol Hydrochloride', 'Product_Amikacin', 'Product_Amikacin Sulfate', 'Product_Amiloride', 'Product_Amiloride Hydrochloride', 'Product_Amiloride Hydrochloride\\Hydrochlorothiazide', 'Product_Amino Acids', 'Product_Amino Acids, Source Unspecified', 'Product_Amino Acids\\Calcium Chloride\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Amino Acids\\Carbohydrates\\Electrolytes Nos\\Vitamins', 'Product_Amino Acids\\Dextrose', 'Product_Amino Acids\\Dextrose\\Egg Phospholipids\\Electrolytes Nos\\Safflower Oil\\Soybean Oil', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos\\Soybean Oil', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos\\Soybean Oil\\Vitamins', 'Product_Amino Acids\\Dextrose\\Vitamins', 'Product_Amino Acids\\Electrolytes Nos\\Xylitol', 'Product_Amino Acids\\Vitamins', 'Product_Aminobenzoic Acid', 'Product_Aminophylline', 'Product_Aminopyrine\\Dipyrone', 'Product_Aminoxyacetic Acid\\Ferrous Sulfate', 'Product_Amiodarone', 'Product_Amiodarone Hydrochloride', 'Product_Amisulpride', 'Product_Amitriptyline', 'Product_Amitriptyline Hydrochloride', 'Product_Amlodipine Besylate', 'Product_Amlodipine Besylate\\Atorvastatin Calcium', 'Product_Amlodipine Besylate\\Benazepril Hydrochloride', 'Product_Amlodipine Besylate\\Indapamide\\Perindopril Arginine', 'Product_Amlodipine Besylate\\Irbesartan', 'Product_Amlodipine Besylate\\Olmesartan Medoxomil', 'Product_Amlodipine Besylate\\Telmisartan', 'Product_Amlodipine Besylate\\Valsartan', 'Product_Amlodipine Maleate', 'Product_Amlodipine\\Atorvastatin\\Perindopril', 'Product_Amlodipine\\Perindopril', 'Product_Amlodipine\\Telmisartan', 'Product_Amlodipine\\Valsartan', 'Product_Ammonium Bromide', 'Product_Ammonium Chloride', 'Product_Ammonium Chloride\\Chlorpheniramine Maleate', 'Product_Ammonium Chloride\\Chlorpheniramine\\Codeine\\Ephedrine', 'Product_Ammonium Chloride\\Chlorpheniramine\\Sodium Citrate', 'Product_Ammonium Chloride\\Diphenhydramine\\Etofylline\\Sodium Citrate\\Theophylline', 'Product_Ammonium Glycyrrhizate', 'Product_Ammonium Lactate', 'Product_Amoxicillin', 'Product_Amoxicillin Sodium', 'Product_Amoxicillin\\Clarithromycin\\Lansoprazole', 'Product_Amoxicillin\\Clarithromycin\\Pantoprazole', 'Product_Amoxicillin\\Clavulanate Potassium', 'Product_Amoxicillin\\Clavulanic Acid', 'Product_Amphetamine Aspartate\\Amphetamine Sulfate\\Dextroamphetamine Saccharate\\Dextroamphetamine Sulfate', 'Product_Amphotericin B', 'Product_Amphotericin B\\Tetracycline', 'Product_Ampicillin', 'Product_Ampicillin Sodium\\Sulbactam Sodium', 'Product_Anagliptin', 'Product_Anagrelide Hydrochloride', 'Product_Anastrozole', 'Product_Angiotensin Ii', 'Product_Anhydrous Citric Acid', 'Product_Anhydrous Citric Acid\\Magnesium Oxide\\Sodium Picosulfate', 'Product_Anidulafungin', 'Product_Antithrombin Iii Human', 'Product_Apixaban', 'Product_Apomorphine Hydrochloride', 'Product_Apremilast', 'Product_Arformoterol Tartrate', 'Product_Aripiprazole', 'Product_Ascorbic Acid', 'Product_Ascorbic Acid\\Biotin\\Calcium Pantothenate\\Cyanocobalamin\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Mononitrate', 'Product_Ascorbic Acid\\Ergocalciferol\\Folic Acid\\Niacinamide\\Panthenol\\Retinol\\Riboflavin\\Thiamine Hydrochloride', 'Product_Ascorbic Acid\\Hesperidin Methylchalcone\\Ruscus Aculeatus Root', 'Product_Ascorbic Acid\\Hesperidin Methylchalcone\\Troxerutin', 'Product_Ascorbic Acid\\Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Ascorbate\\Sodium Chloride\\Sodium Sulfate', 'Product_Ascorbic Acid\\Polyethylene Glycol 4000\\Potassium Cl\\Sod Ascorbate\\Sod Cl\\Sod Sulfate', 'Product_Asian Ginseng', 'Product_Asparagine', 'Product_Aspartame', 'Product_Aspirin', 'Product_Aspirin Dl-Lysine', 'Product_Aspirin Lysine', 'Product_Aspirin\\Clopidogrel', 'Product_Aspirin\\Clopidogrel Bisulfate', 'Product_Aspirin\\Dihydroxyaluminum Aminoacetate Anhydrous\\Magnesium Carbonate', 'Product_Aspirin\\Dipyridamole', 'Product_Aspirin\\Glycine', 'Product_Aspirin\\Lansoprazole', 'Product_Astemizole', 'Product_Atazanavir', 'Product_Atenolol', 'Product_Atenolol Hydrochloride', 'Product_Atezolizumab', 'Product_Atlantic Salmon Oil', 'Product_Atorvastatin', 'Product_Atorvastatin Calcium', 'Product_Atorvastatin Calcium Trihydrate\\Ezetimibe', 'Product_Atorvastatin Calcium\\Ezetimibe', 'Product_Atovaquone', 'Product_Atracurium', 'Product_Atracurium Besylate', 'Product_Atropine', 'Product_Atropine Sulfate', 'Product_Atropine Sulfate\\Diphenoxylate Hydrochloride', 'Product_Avapritinib', 'Product_Avelumab', 'Product_Avobenzone\\Octisalate\\Octocrylene', 'Product_Avocado Oil', 'Product_Axitinib', 'Product_Azacitidine', 'Product_Azathioprine', 'Product_Azathioprine Sodium', 'Product_Azd-1222', 'Product_Azelnidipine', 'Product_Azelnidipine\\Olmesartan Medoxomil', 'Product_Azilsartan Kamedoxomil', 'Product_Azithromycin', 'Product_Azithromycin Anhydrous', 'Product_Azithromycin Dihydrate', 'Product_Azosemide', 'Product_Bacillus Calmette-Guerin Substrain Rivm Live Antigen', 'Product_Bacillus Calmette-Guerin Substrain Tice Live Antigen', 'Product_Bacillus Subtilis', 'Product_Bacitracin\\Hydrocortisone\\Neomycin Sulfate', 'Product_Baclofen', 'Product_Bambuterol Hydrochloride', 'Product_Bamlanivimab', 'Product_Bamlanivimab\\Etesevimab', 'Product_Barnidipine Hydrochloride', 'Product_Bazedoxifene', 'Product_Bcg Vaccine', 'Product_Beclomethasone', 'Product_Beclomethasone Dipropionate', 'Product_Beclomethasone Dipropionate\\Clotrimazole', 'Product_Beclomethasone Dipropionate\\Formoterol', 'Product_Belatacept', 'Product_Bempedoic Acid\\Ezetimibe', 'Product_Benazepril Hydrochloride', 'Product_Bendamustine Hydrochloride', 'Product_Benfluorex', 'Product_Benfluorex Hydrochloride', 'Product_Benfotiamine', 'Product_Benfotiamine\\Cyanocobalamin', 'Product_Benidipine', 'Product_Benidipine Hydrochloride', 'Product_Benserazide', 'Product_Benserazide Hydrochloride\\Levodopa', 'Product_Benserazide\\Levodopa', 'Product_Benzalkonium Chloride\\Salicylic Acid\\Triamcinolone Acetonide', 'Product_Benzodiazepine', 'Product_Benzoic Acid\\Methenamine\\Sodium Salicylate', 'Product_Benzonatate', 'Product_Benzthiazide\\Triamterene', 'Product_Benzydamine Hydrochloride\\Chlorhexidine Gluconate', 'Product_Bepotastine Besylate', 'Product_Beraprost Sodium', 'Product_Beta Carotene', 'Product_Beta Glucan', 'Product_Betahistine Hydrochloride', 'Product_Betahistine Mesilate', 'Product_Betamethasone', 'Product_Betamethasone Acetate\\Betamethasone Sodium Phosphate', 'Product_Betamethasone Dipropionate', 'Product_Betamethasone Dipropionate\\Betamethasone Sodium Phosphate', 'Product_Betamethasone Dipropionate\\Clioquinol', 'Product_Betamethasone Dipropionate\\Clotrimazole', 'Product_Betamethasone Dipropionate\\Clotrimazole\\Gentamicin Sulfate', 'Product_Betamethasone Sodium Phosphate', 'Product_Betamethasone Valerate', 'Product_Betamethasone Valerate\\Gentamicin Sulfate', 'Product_Betamethasone\\Betamethasone Dipropionate\\Clotrimazole', 'Product_Betamethasone\\Clotrimazole', 'Product_Betamethasone\\Fusidic Acid', 'Product_Betaxolol', 'Product_Betaxolol Hydrochloride', 'Product_Bethanechol Chloride', 'Product_Bevacizumab', 'Product_Bezafibrate', 'Product_Bicalutamide', 'Product_Bifidobacterium Bifidum', 'Product_Bifidobacterium Longum Subsp. Infantis', 'Product_Bifidobacterium Spp.', 'Product_Bilastine', 'Product_Bilberry', 'Product_Bimatoprost', 'Product_Binimetinib', 'Product_Biotin', 'Product_Bisacodyl', 'Product_Bisacodyl\\Docusate Sodium\\Sennosides A And B', 'Product_Bisacodyl\\Sennosides A And B', 'Product_Bismuth', 'Product_Bismuth Hydroxide', 'Product_Bismuth Subcarbonate\\Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Bismuth Subcitrate Potassium\\Metronidazole\\Tetracycline Hydrochloride', 'Product_Bismuth Subgallate', 'Product_Bismuth Subgallate\\Bufexamac\\Lidocaine Hydrochloride', 'Product_Bismuth Subgallate\\Bufexamac\\Lidocaine Hydrochloride\\Titanium Dioxide', 'Product_Bismuth Subnitrate', 'Product_Bismuth Subnitrate\\Magnesium Oxide\\Magnesium Tris', 'Product_Bismuth Subnitrate\\Magnesium Oxide\\Magnesium Trisilicate\\Sodium Phosphate, Monobasic, Anhydrous', 'Product_Bismuth Subsalicylate', 'Product_Bismuth\\Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Bisoprolol', 'Product_Bisoprolol Fumarate', 'Product_Bisoprolol Fumarate\\Hydrochlorothiazide', 'Product_Bisoprolol\\Perindopril', 'Product_Blinatumomab', 'Product_Bococizumab', 'Product_Boldine\\Sodium Bicarbonate\\Sodium Phosphate, Monobasic, Anhydrous\\Sodium Sulfate', 'Product_Boric Acid\\Sodium Borate', 'Product_Boric Acid\\Zinc Oxide', 'Product_Bortezomib', 'Product_Bosentan', 'Product_Brexpiprazole', 'Product_Brimonidine Tartrate\\Timolol Maleate', 'Product_Brolucizumab', 'Product_Bromazepam', 'Product_Bromfenac Sodium', 'Product_Bromhexine', 'Product_Brotizolam', 'Product_Budesonide', 'Product_Budesonide\\Formoterol', 'Product_Budesonide\\Formoterol Fumarate', 'Product_Budesonide\\Formoterol Fumarate Dihydrate', 'Product_Budesonide\\Formoterol\\Glycopyrronium', 'Product_Bufexamac', 'Product_Bufexamac\\Lidocaine Hydrochloride', 'Product_Buflomedil', 'Product_Buflomedil Hydrochloride', 'Product_Buformin Hydrochloride', 'Product_Bumetanide', 'Product_Bupivacaine Hydrochloride', 'Product_Buprenorphine', 'Product_Buprenorphine Hydrochloride', 'Product_Bupropion', 'Product_Bupropion Hydrochloride', 'Product_Bupropion Hydrochloride\\Naltrexone Hydrochloride', 'Product_Buspirone Hydrochloride', 'Product_Cabergoline', 'Product_Cabozantinib', 'Product_Cabozantinib S-Malate', 'Product_Cafedrine\\Theodrenaline', 'Product_Caffeine', 'Product_Caffeine\\Diphenhydramine Hydrochloride\\Ethenzamide\\Phenacetin\\Thiamine Mononitrate', 'Product_Caffeine\\Phenobarbital', 'Product_Calcitonin Salmon', 'Product_Calcitriol', 'Product_Calcium', 'Product_Calcium Acetate', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Acetate', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Acetate\\Sodium Acetate\\Sodium Chlori', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Chloride', 'Product_Calcium Ascorbate', 'Product_Calcium Carbonate', 'Product_Calcium Carbonate\\Cholecalciferol', 'Product_Calcium Carbonate\\Ergocalciferol', 'Product_Calcium Carbonate\\Ergocalciferol\\Retinol', 'Product_Calcium Carbonate\\Ergocalciferol\\Vitamin A', 'Product_Calcium Carbonate\\Vitamin D', 'Product_Calcium Carbonate\\Vitamin D\\Zinc', 'Product_Calcium Chloride', 'Product_Calcium Chloride\\Dextrose Monohydrate\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Dextrose\\Magnesium Chloride\\Potassium Chloride', 'Product_Calcium Chloride\\Dextrose\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Chloride', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Hetastarch\\Magnesium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Icodextrin\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Magnesium Chloride\\Potassium Chloride\\Sodium Acetate Anhydrous\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride\\Trihydroxypalmitamidohydroxypropyl Myristyl Ether', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate\\Sorbitol', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Lactate', 'Product_Calcium Citrate', 'Product_Calcium Citrate\\Cholecalciferol', 'Product_Calcium Dobesilate', 'Product_Calcium Gluceptate', 'Product_Calcium Gluconate', 'Product_Calcium Gluconate Monohydrate', 'Product_Calcium Gluconate\\Dextrose\\Magnesium Sulfate\\Potassium Acetate\\Potassium Phosphate, Dibasic\\Zinc Sulfate', 'Product_Calcium Lactate', 'Product_Calcium Pantothenate', 'Product_Calcium Pantothenate\\Cyanocobalamin\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Hydrochloride', 'Product_Calcium Phosphate', 'Product_Calcium Phosphate, Dibasic, Anhydrous\\Magnesium Glycerophosphate\\Phosphoric Acid\\Sodium Phosphate, Dibasic', 'Product_Calcium Phosphate, Monobasic, Anhydrous\\Magnesium Glycerophosphate\\Phosphoric Acid\\Sodium Phosphate, Dibasic', 'Product_Calcium Phosphate\\Ergocalciferol', 'Product_Calcium Polycarbophil', 'Product_Calcium Polystyrene Sulfonate', 'Product_Calcium\\Cholecalciferol', 'Product_Calcium\\Cholecalciferol\\Ergocalciferol', 'Product_Calcium\\Cholecalciferol\\Magnesium', 'Product_Calcium\\Ergocalciferol', 'Product_Calcium\\Vitamin D', 'Product_Canagliflozin', 'Product_Canagliflozin\\Metformin Hydrochloride', 'Product_Canagliflozin\\Teneligliptin', 'Product_Canakinumab', 'Product_Candesartan', 'Product_Candesartan Cilexetil', 'Product_Candesartan Cilexetil\\Hydrochlorothiazide', 'Product_Cannabidiol', 'Product_Canrenoate Potassium', 'Product_Canrenone', 'Product_Capecitabine', 'Product_Caprylic Acid', 'Product_Capsaicin', 'Product_Captopril', 'Product_Captopril\\Hydrochlorothiazide', 'Product_Carbamazepine', 'Product_Carbazochrome Sodium Sulfonate', 'Product_Carbidopa\\Entacapone\\Levodopa', 'Product_Carbidopa\\Levodopa', 'Product_Carbimazole', 'Product_Carbocysteine', 'Product_Carbohydrates', 'Product_Carbohydrates\\Electrolytes Nos\\Minerals\\Protein\\Vitamins', 'Product_Carbohydrates\\Fatty Acids\\Minerals\\Protein\\Vitamins', 'Product_Carbohydrates\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Carbomer', 'Product_Carboplatin', 'Product_Carboxymethylcellulose Sodium', 'Product_Carisoprodol', 'Product_Carperitide', 'Product_Carvedilol', 'Product_Carvedilol Hydrochloride', 'Product_Carvedilol Phosphate', 'Product_Caspofungin', 'Product_Caspofungin Acetate', 'Product_Cefaclor', 'Product_Cefazolin', 'Product_Cefazolin Sodium', 'Product_Cefcapene Pivoxil Hydrochloride', 'Product_Cefdinir', 'Product_Cefditoren Pivoxil', 'Product_Cefepime Hydrochloride', 'Product_Cefixime', 'Product_Cefmetazole', 'Product_Cefmetazole Sodium', 'Product_Cefoperazone Sodium\\Sulbactam Sodium', 'Product_Cefoperazone\\Sulbactam', 'Product_Cefoperazone\\Sulbactam Sodium', 'Product_Cefotaxime Sodium', 'Product_Cefotiam\\Mepivacaine', 'Product_Cefoxitin Sodium', 'Product_Cefpodoxime Proxetil', 'Product_Ceftazidime', 'Product_Ceftazidime Sodium', 'Product_Ceftriaxone', 'Product_Ceftriaxone Sodium', 'Product_Ceftriaxone Sodium\\Tazobactam Sodium', 'Product_Cefuroxime', 'Product_Cefuroxime Axetil', 'Product_Cefuroxime Sodium', 'Product_Celecoxib', 'Product_Celiprolol', 'Product_Celiprolol Hydrochloride', 'Product_Cephalexin', 'Product_Cephalexin Hydrochloride', 'Product_Cephalexin Sodium', 'Product_Cerivastatin Sodium', 'Product_Certolizumab', 'Product_Certolizumab Pegol', 'Product_Certoparin Sodium', 'Product_Cetirizine Hydrochloride', 'Product_Cetirizine Hydrochloride\\Pseudoephedrine Hydrochloride', 'Product_Cetuximab', 'Product_Chloramphenicol', 'Product_Chloramphenicol Palmitate', 'Product_Chloramphenicol\\Hydrocortisone', 'Product_Chloramphenicol\\Hydrocortisone Acetate', 'Product_Chloramphenicol\\Hydrocortisone Acetate\\Metronidazole\\Natamycin', 'Product_Chloramphenicol\\Hydrocortisone Acetate\\Metronidazole\\Nystatin', 'Product_Chlorhexidine Gluconate', 'Product_Chlorpheniramine', 'Product_Chlorpheniramine Maleate', 'Product_Chlorpheniramine Maleate\\Dextromethorphan Hydrobromide\\Guaifenesin\\Phenylephrine Hydrochloride', 'Product_Chlorpromazine', 'Product_Chlorpromazine Hydrochloride', 'Product_Chlorpropamide', 'Product_Chlorthalidone', 'Product_Cholecalciferol', 'Product_Cholecalciferol\\Ergocalciferol', 'Product_Cholesterol', 'Product_Cholestyramine', 'Product_Choline Bitartrate\\Cyanocobalamin\\Cysteine\\Inositol\\Mammal Liver', 'Product_Chondroitin Sulfate (Bovine)', 'Product_Chondroitin Sulfate (Bovine)\\Glucosamine Hydrochloride', 'Product_Chondroitin Sulfate (Bovine)\\Glucosamine Sulfate', 'Product_Chromic Chloride\\Cuprous Chloride\\Ferric Chloride\\Manganese Chloride\\Potassium Iodide\\Sodium Fluoride\\Sodium Molybdate Dihydrate\\Sodium Selenite\\Zinc Chloride', 'Product_Ciclesonide', 'Product_Ciclopirox', 'Product_Cifenline', 'Product_Cifenline Succinate', 'Product_Cilastatin Sodium\\Imipenem', 'Product_Cilazapril Anhydrous', 'Product_Cilnidipine', 'Product_Cilostazol', 'Product_Cimetidine', 'Product_Cinacalcet Hydrochloride', 'Product_Cinnamon\\Herbals', 'Product_Cinnarizine', 'Product_Ciprofibrate', 'Product_Ciprofloxacin', 'Product_Ciprofloxacin Hydrochloride', 'Product_Ciprofloxacin Hydrochloride\\Hydrocortisone', 'Product_Ciprofloxacin Lactate', 'Product_Ciprofloxacin\\Ciprofloxacin Hydrochloride', 'Product_Cisapride', 'Product_Cisatracurium', 'Product_Cisatracurium Besylate', 'Product_Cisplatin', 'Product_Citalopram', 'Product_Citalopram Hydrobromide', 'Product_Citicoline', 'Product_Citicoline Sodium', 'Product_Citric Acid Monohydrate', 'Product_Citric Acid Monohydrate\\Dextrose\\Sodium Citrate', 'Product_Citric Acid Monohydrate\\Potassium Bicarbonate', 'Product_Citric Acid Monohydrate\\Potassium Bicarbonate\\Potassium Citrate', 'Product_Citric Acid Monohydrate\\Sodium Citrate', 'Product_Citrus Fiber', 'Product_Clarithromycin', 'Product_Clemastine', 'Product_Clemastine Fumarate', 'Product_Clevidipine', 'Product_Clikstar', 'Product_Clindamycin', 'Product_Clindamycin Hydrochloride', 'Product_Clindamycin Palmitate Hydrochloride', 'Product_Clindamycin Phosphate', 'Product_Clioquinol', 'Product_Clioquinol\\Hydrocortisone', 'Product_Clobazam', 'Product_Clobetasol', 'Product_Clobetasol Propionate', 'Product_Clobutinol', 'Product_Clomipramine Hydrochloride', 'Product_Clonazepam', 'Product_Clonidine', 'Product_Clonidine Hydrochloride', 'Product_Clonixin', 'Product_Clopidogrel Besilate', 'Product_Clopidogrel Bisulfate', 'Product_Clorazepate Dipotassium', 'Product_Clorazepic Acid', 'Product_Clotiazepam', 'Product_Clotrimazole', 'Product_Clotrimazole\\Hydrocortisone', 'Product_Cloxacillin', 'Product_Cloxacillin Sodium', 'Product_Clozapine', 'Product_Coagulation Factor Ix Human\\Coagulation Factor Vii Human\\Coagulation Factor X Human\\Protein C\\Protein S Human\\Prothrombin', 'Product_Cocarboxylase', 'Product_Cocoa Butter\\Phenylephrine Hydrochloride', 'Product_Coconut Oil', 'Product_Cod Liver Oil\\Urea\\Zinc Oxide', 'Product_Codeine', 'Product_Codeine Phosphate', 'Product_Codeine Phosphate\\Guaifenesin', 'Product_Codeine Sulfate', 'Product_Coenzyme A', 'Product_Colchicine', 'Product_Colchicine\\Dicyclomine Hydrochloride', 'Product_Colchicine\\Opium', 'Product_Colchicine\\Opium\\Tiemonium Methylsulfate', 'Product_Colesevelam Hydrochloride', 'Product_Colestipol', 'Product_Convallaria Majalis\\Crataegus Laevigata Leaf\\Proxyphylline', 'Product_Corticosteroid Nos', 'Product_Cortisone', 'Product_Cortisone Acetate', 'Product_Cosmetics', 'Product_Covid-19 Vaccine', 'Product_Creatine', 'Product_Creatine\\Dietary Supplement', 'Product_Cresol', 'Product_Croscarmellose Sodium', 'Product_Cupric Sulfate', 'Product_Cupric Sulfate Anhydrous', 'Product_Cyamemazine', 'Product_Cyanocobalamin', "Product_Cyanocobalamin\\Dexpanthenol\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin 5'-Phosphate Sodium\\Thiamine Hydrochloride\\Vitamin B Complex", 'Product_Cyanocobalamin\\Folic Acid', 'Product_Cyanocobalamin\\Folic Acid\\Pyridoxine', 'Product_Cyanocobalamin\\Folic Acid\\Pyridoxine Hydrochloride', 'Product_Cyanocobalamin\\Pyridoxine Hydrochloride\\Riboflavin\\Sodium Pantothenate\\Thiamine Hydrochloride\\Vitamin B Complex', 'Product_Cyanocobalamin\\Pyridoxine Hydrochloride\\Thiamine', 'Product_Cyanocobalamin\\Pyridoxine\\Thiamine', 'Product_Cyclobenzaprine', 'Product_Cyclobenzaprine Hydrochloride', 'Product_Cyclophosphamide', 'Product_Cyclophosphamide\\Cytarabine\\Dexamethasone\\Doxorubicin\\Methotrexate\\Vincristine', 'Product_Cyclosporine', 'Product_Cyproheptadine', 'Product_Cyproheptadine Hydrochloride', 'Product_Cytarabine', 'Product_Dabigatran', 'Product_Dabigatran Etexilate', 'Product_Dabigatran Etexilate Mesylate', 'Product_Dabrafenib', 'Product_Dabrafenib Mesylate', 'Product_Daclatasvir', 'Product_Daclizumab', 'Product_Dalteparin', 'Product_Dalteparin Sodium', 'Product_Danazol', 'Product_Dapagliflozin', 'Product_Dapagliflozin Propanediol', 'Product_Dapagliflozin Propanediol\\Metformin Hydrochloride', 'Product_Dapsone', 'Product_Dapsone\\Ferrous Oxalate', 'Product_Daptomycin', 'Product_Darbepoetin Alfa', 'Product_Dasabuvir\\Ombitasvir\\Paritaprevir\\Ritonavir', 'Product_Dasatinib', 'Product_Daunorubicin', 'Product_Deferasirox', 'Product_Deferiprone', 'Product_Deflazacort', 'Product_Degarelix Acetate', 'Product_Denosumab', 'Product_Desipramine Hydrochloride', 'Product_Desloratadine', 'Product_Desmopressin Acetate', 'Product_Desoximetasone', 'Product_Desvenlafaxine Succinate', 'Product_Device', 'Product_Device\\Insulin Nos', 'Product_Dexamethasone', 'Product_Dexamethasone Acetate', 'Product_Dexamethasone Sodium Phosphate', 'Product_Dexamethasone Sodium Phosphate\\Moxifloxacin Hydrochloride', 'Product_Dexamethasone Sodium Succinate', 'Product_Dexamethasone\\Tobramycin', 'Product_Dexchlorpheniramine Maleate', 'Product_Dexketoprofen', 'Product_Dexketoprofen Trometamol', 'Product_Dexlansoprazole', 'Product_Dexmedetomidine', 'Product_Dexpanthenol', "Product_Dexpanthenol\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin 5'-Phosphate Sodium\\Thiamine Hydrochloride", 'Product_Dextran', 'Product_Dextroamphetamine', 'Product_Dextroamphetamine Sulfate', 'Product_Dextromethorphan Hydrobromide\\Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Dextromethorphan Hydrobromide\\Quinidine Sulfate', 'Product_Dextrose Monohydrate', 'Product_Dextrose Monohydrate\\Sodium Chloride', 'Product_Dextrose\\Heparin', 'Product_Dextrose\\Heparin Sodium', 'Product_Dextrose\\Potassium Chloride\\Sodium Chloride', 'Product_Dextrose\\Potassium Chloride\\Sodium Chloride\\Sodium Citrate', 'Product_Dextrose\\Sodium Chloride', 'Product_Dextrose\\Water', 'Product_Diazepam', 'Product_Diclofenac', 'Product_Diclofenac Epolamine', 'Product_Diclofenac Sodium', 'Product_Diclofenac Sodium\\Misoprostol', 'Product_Dicyclomine Hydrochloride', 'Product_Dietary Supplement', 'Product_Dietary Supplement\\Herbals', 'Product_Dietary Supplement\\Herbals\\Minerals\\Vitamins', 'Product_Dietary Supplement\\Probiotics Nos', 'Product_Dietary Supplement\\Ubidecarenone', 'Product_Digitoxin', 'Product_Digoxin', 'Product_Dihydralazine Sulfate', 'Product_Dihydralazine Sulfate\\Hydrochlorothiazide\\Reserpine', 'Product_Dihydrocodeine', 'Product_Dihydrocodeine Bitartrate', 'Product_Dihydrocodeine Phosphate', 'Product_Dihydroergotamine Mesylate\\Heparin Sodium\\Lidocaine Hydrochloride', 'Product_Diltiazem', 'Product_Diltiazem Hydrochloride', 'Product_Diltiazem Malate', 'Product_Dimenhydrinate', 'Product_Dimethicone', 'Product_Dimethicone\\Polyethylene Glycol 4000\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Dimethindene', 'Product_Dimethindene Maleate', 'Product_Dimethyl Fumarate', 'Product_Dimethyl Sulfone', 'Product_Diosmin', 'Product_Diosmin\\Hesperidin', 'Product_Diphenhydramine', 'Product_Diphenhydramine Hydrochloride', 'Product_Diphtheria And Tetanus Toxoids And Acellular Pertussis Adsorbed And Inactivated Poliovirus Vaccine', 'Product_Dipivefrin Hydrochloride', 'Product_Dipyridamole', 'Product_Dipyrone', 'Product_Disodium Hydrogen Citrate', 'Product_Disopyramide Phosphate', 'Product_Distigmine Bromide', 'Product_Divalproex Sodium', 'Product_Dobutamine Hydrochloride', 'Product_Docetaxel', 'Product_Doconexent\\ Icosapent\\ Fish Oil\\ Omega-3 Fatty Acids', 'Product_Doconexent\\Minerals\\Vitamins', 'Product_Docusate', 'Product_Docusate Calcium', 'Product_Docusate Sodium', 'Product_Docusate Sodium\\Senna Leaf', 'Product_Docusate Sodium\\Sennoside B', 'Product_Docusate Sodium\\Sennosides A And B', 'Product_Dofetilide', 'Product_Domperidone', 'Product_Domperidone Maleate', 'Product_Donepezil', 'Product_Donepezil Hydrochloride', 'Product_Dopamine Hydrochloride', 'Product_Doripenem Monohydrate', 'Product_Dorzolamide Hydrochloride', 'Product_Dorzolamide Hydrochloride\\Timolol Maleate', 'Product_Doxazosin Mesylate', 'Product_Doxazosin\\Doxazosin Mesylate', 'Product_Doxepin', 'Product_Doxepin Hydrochloride', 'Product_Doxofylline', 'Product_Doxorubicin', 'Product_Doxorubicin Hydrochloride', 'Product_Doxycycline', 'Product_Doxycycline Calcium', 'Product_Doxycycline Hyclate', 'Product_Dronabinol', 'Product_Dronedarone', 'Product_Drotaverine', 'Product_Drotrecogin Alfa (Activated)', 'Product_Droxidopa', 'Product_Dulaglutide', 'Product_Duloxetine', 'Product_Duloxetine Hydrochloride', 'Product_Duloxetine Hydrochloride\\Repaglinide\\Ticagrelor', 'Product_Dupilumab', 'Product_Durvalumab', 'Product_Dutasteride', 'Product_Dutasteride\\Tamsulosin Hydrochloride', 'Product_Ebastine', 'Product_Ecabet', 'Product_Econazole Nitrate', 'Product_Edaravone', 'Product_Efinaconazole', 'Product_Efmoroctocog Alfa', 'Product_Efonidipine Hydrochloride', 'Product_Egg Phospholipids\\Glycerin\\Medium-Chain Triglycerides\\Soybean Oil', 'Product_Egg Phospholipids\\Glycerin\\Soybean Oil', 'Product_Egg Phospholipids\\Safflower Oil\\Soybean Oil', 'Product_Elasomeran', 'Product_Elbasvir', 'Product_Eldecalcitol', 'Product_Electrolytes Nos', 'Product_Electrolytes Nos\\Minerals', 'Product_Electrolytes Nos\\Minerals\\Sorbitol', 'Product_Electrolytes Nos\\Polyethylene Glycol 3350', 'Product_Electrolytes Nos\\Polyethylene Glycol 4000', 'Product_Electrolytes Nos\\Polyethylene Glycols', 'Product_Elexacaftor\\Ivacaftor\\Tezacaftor', 'Product_Empagliflozin', 'Product_Empagliflozin\\Linagliptin', 'Product_Empagliflozin\\Metformin Hydrochloride', 'Product_Emtricitabine\\Tenofovir Disoproxil Fumarate', 'Product_Enalapril', 'Product_Enalapril Maleate', 'Product_Enalapril Maleate\\Hydrochlorothiazide', 'Product_Enalapril\\Hydrochlorothiazide', 'Product_Encorafenib', 'Product_Enflurane', 'Product_Enfortumab Vedotin', 'Product_Enfortumab Vedotin\\Enfortumab Vedotin-Ejfv', 'Product_Enocitabine', 'Product_Enoxaparin', 'Product_Enoxaparin Sodium', 'Product_Enoximone', 'Product_Ensulizole\\Eucerin\\Octinoxate\\Octisalate\\Titanium Dioxide\\Zinc Oxide', 'Product_Entecavir', 'Product_Enzalutamide', 'Product_Epalrestat', 'Product_Eperisone Hydrochloride', 'Product_Ephedrine', 'Product_Epinastine Hydrochloride', 'Product_Epinephrine', 'Product_Epirubicin', 'Product_Epirubicin Hydrochloride', 'Product_Eplerenone', 'Product_Epoetin Delta', 'Product_Epoprostenol', 'Product_Epoprostenol Sodium', 'Product_Eprosartan Mesylate', 'Product_Eprosartan Mesylate\\Hydrochlorothiazide', 'Product_Erdafitinib', 'Product_Ergocalciferol', 'Product_Ergocalciferol\\Phytonadione\\Retinol\\Tocopherol', 'Product_Ergoloid Mesylates', 'Product_Erlotinib', 'Product_Erlotinib Hydrochloride', 'Product_Ertapenem', 'Product_Ertapenem Sodium', 'Product_Ertugliflozin', 'Product_Ertugliflozin Pidolate', 'Product_Erythromycin', 'Product_Erythromycin Ethylsuccinate', 'Product_Erythromycin Stearate', 'Product_Erythropoietin', 'Product_Escitalopram Oxalate', 'Product_Esomeprazole', 'Product_Esomeprazole Magnesium', 'Product_Esomeprazole Magnesium\\Naproxen', 'Product_Esomeprazole Sodium', 'Product_Estradiol', 'Product_Estramustine', 'Product_Estramustine Phosphate Sodium', 'Product_Estriol', 'Product_Estrogens', 'Product_Estrogens, Conjugated', 'Product_Estropipate', 'Product_Eszopiclone', 'Product_Etanercept', 'Product_Ethacrynic Acid', 'Product_Ethambutol Hydrochloride', 'Product_Ethyl Loflazepate', 'Product_Ethylenediamine Dihydrochloride', 'Product_Etilefrine', 'Product_Etizolam', 'Product_Etodolac', 'Product_Etofylline\\Theophylline', 'Product_Etomidate', 'Product_Etoposide', 'Product_Etoricoxib', 'Product_Everolimus', 'Product_Evolocumab', 'Product_Exemestane', 'Product_Exenatide', 'Product_Ezetimibe', 'Product_Ezetimibe\\Simvastatin', 'Product_Famotidine', 'Product_Favipiravir', 'Product_Febuxostat', 'Product_Felodipine', 'Product_Felodipine\\Metoprolol Succinate', 'Product_Felodipine\\Ramipril', 'Product_Fenofibrate', 'Product_Fenofibric Acid', 'Product_Fenoterol', 'Product_Fenoterol Hydrobromide', 'Product_Fenoterol Hydrobromide\\Ipratropium Bromide', 'Product_Fenoterol\\Ipratropium', 'Product_Fenoterol\\Ipratropium Bromide', 'Product_Fentanyl', 'Product_Fentanyl Citrate', 'Product_Ferric Carboxymaltose', 'Product_Ferric Derisomaltose', 'Product_Ferric Sodium Citrate', 'Product_Ferrous Ammonium Sulfate Anhydrous\\Folic Acid', 'Product_Ferrous Ascorbate', 'Product_Ferrous Fumarate', 'Product_Ferrous Glycine Sulfate', 'Product_Ferrous Succinate', 'Product_Ferrous Sulfate', 'Product_Ferrous Sulfate Anhydrous', 'Product_Ferrous Sulfate\\Folic Acid', 'Product_Ferrous Sulfate\\Folic Acid\\Gastric Mucin', 'Product_Ferrous Sulfate\\Mammal Liver', 'Product_Fesoterodine', 'Product_Fesoterodine Fumarate', 'Product_Fexofenadine Hydrochloride', 'Product_Fexofenadine Hydrochloride\\Pseudoephedrine Hydrochloride', 'Product_Filgotinib', 'Product_Filgrastim', 'Product_Finasteride', 'Product_Finasteride\\Tamsulosin', 'Product_Fish Oil', 'Product_Fish Oil\\Medium-Chain Triglycerides\\Olive Oil\\Soybean Oil', 'Product_Flavoxate Hydrochloride', 'Product_Flax Seed', 'Product_Flecainide', 'Product_Flecainide Acetate', 'Product_Floctafenine', 'Product_Flomoxef', 'Product_Flucloxacillin', 'Product_Fluconazole', 'Product_Flucytosine', 'Product_Fludarabine Phosphate', 'Product_Fludrocortisone', 'Product_Fludrocortisone Acetate', 'Product_Fluindione', 'Product_Flunisolide', 'Product_Flunitrazepam', 'Product_Fluorouracil', 'Product_Fluorouracil\\Irinotecan\\Leucovorin', 'Product_Fluorouracil\\Leucovorin Calcium\\Oxaliplatin', 'Product_Fluoxetine Hydrochloride', 'Product_Flurazepam Hydrochloride', 'Product_Flurbiprofen', 'Product_Flutamide', 'Product_Fluticasone', 'Product_Fluticasone Furoate', 'Product_Fluticasone Furoate Or Umeclidinium Bromide Or Vilanterol Trifenatate', 'Product_Fluticasone Furoate\\Umeclidinium Bromide\\Vilanterol Trifenatate', 'Product_Fluticasone Furoate\\Vilanterol Trifenatate', 'Product_Fluticasone Propionate', 'Product_Fluticasone Propionate\\Salmeterol Xinafoate', 'Product_Fluvastatin Sodium', 'Product_Folate Sodium', 'Product_Folic Acid', 'Product_Folic Acid\\Iron\\Minerals\\Vitamins', 'Product_Fondaparinux', 'Product_Fondaparinux Sodium', 'Product_Formoterol', 'Product_Formoterol Fumarate', 'Product_Formoterol Fumarate Dihydrate\\Mometasone Furoate', 'Product_Formoterol Fumarate\\Mometasone Furoate', 'Product_Formoterol\\Mometasone', 'Product_Fosfluconazole', 'Product_Fosfomycin', 'Product_Fosfomycin Sodium', 'Product_Fosfomycin Tromethamine', 'Product_Fosinopril', 'Product_Fosinopril Sodium', 'Product_Fosinopril Sodium\\Hydrochlorothiazide', 'Product_Fosinopril\\Hydrochlorothiazide', 'Product_Fosphenytoin', 'Product_Fructose\\Glycerin', 'Product_Fructose\\Insulin Beef', 'Product_Fulvestrant', 'Product_Furazolidone', 'Product_Furosemide', 'Product_Furosemide\\Potassium Chloride', 'Product_Furosemide\\Reserpine', 'Product_Furosemide\\Spironolactone', 'Product_Fusidate Sodium', 'Product_Gabapentin', 'Product_Gabapentin Enacarbil', 'Product_Galantamine', 'Product_Galantamine Hydrobromide', 'Product_Ganciclovir', 'Product_Ganciclovir Sodium', 'Product_Garlic', 'Product_Gatifloxacin', 'Product_Gatifloxacin\\Prednisolone Acetate', 'Product_Gelatin', 'Product_Gemcitabine', 'Product_Gemcitabine Hydrochloride', 'Product_Gemcitabine\\Gemcitabine Hydrochloride', 'Product_Gemfibrozil', 'Product_Gemigliptin\\Metformin', 'Product_Gentamicin', 'Product_Gentamicin Sulfate', 'Product_Gimeracil\\Oteracil Potassium\\Tegafur', 'Product_Ginkgo', 'Product_Ginseng Extract\\Herbs', 'Product_Glecaprevir\\Pibrentasvir', 'Product_Gliclazide', 'Product_Glimepiride', 'Product_Glimepiride\\Metformin', 'Product_Glimepiride\\Pioglitazone Hydrochloride', 'Product_Glimepiride\\Rosiglitazone Maleate', 'Product_Glipizide', 'Product_Gliquidone', 'Product_Glucagon', 'Product_Glucagon Hydrochloride', 'Product_Gluconic Acid', 'Product_Glucosamine', 'Product_Glucose Oxidase', 'Product_Glucose Oxidase Unspecified Source', 'Product_Glutamine\\Sodium Gualenate', 'Product_Glyburide', 'Product_Glyburide\\Metformin', 'Product_Glyburide\\Metformin Hydrochloride', 'Product_Glycerin\\Medium-Chain Triglycerides\\Phospholipid\\Soybean Oil', 'Product_Glycerin\\Mineral Oil\\Petrolatum', 'Product_Glyceryl Monostearate', 'Product_Glycine Max (Soybean) Seed Extract', 'Product_Glycine Max Whole', 'Product_Glycopyrrolate', 'Product_Glycopyrronium\\Indacaterol', 'Product_Glycosaminoglycans', 'Product_Golimumab', 'Product_Gonadorelin', 'Product_Goserelin', 'Product_Goserelin Acetate', 'Product_Granisetron Hydrochloride', 'Product_Granulocyte Colony-Stimulating Factor Nos', 'Product_Grazoprevir', 'Product_Guaifenesin', 'Product_Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Guanabenz Acetate', 'Product_Guanfacine Hydrochloride', 'Product_Guar Gum', 'Product_Haloperidol', 'Product_Haloperidol Decanoate', 'Product_Hamamelis Virginiana Top Water\\Phenylephrine Hydrochloride', 'Product_Heparin Calcium', 'Product_Heparin Sodium', 'Product_Heparin Sodium\\Sodium Chloride', 'Product_Heparin Sodium\\Zinc Sulfate', 'Product_Hepatitis B Virus Subtype Adw2 Hbsag Surface Protein Antigen', 'Product_Herbals', 'Product_Herbals\\Homeopathics', 'Product_Herbals\\Iron\\Vitamins', 'Product_Herbals\\Karaya Gum', 'Product_Herbals\\Paullinia Cupana Seed', 'Product_Herbals\\Plantago Indica Whole', 'Product_Herbals\\Plantago Ovata Seed Coat', 'Product_Herbals\\Saw Palmetto', 'Product_Herbals\\Sennosides', 'Product_Herbals\\Vitamins', 'Product_Herbals\\Zinc Oxide', 'Product_Herpes Simplex Vaccine', 'Product_Hetastarch\\Sodium Chloride', 'Product_Homeopathics', 'Product_Horse Chestnut', 'Product_Human C1-Esterase Inhibitor', 'Product_Human Immunoglobulin G', 'Product_Human Immunoglobulin G\\Hyaluronidase (Human Recombinant)', 'Product_Hyaluronate Sodium', 'Product_Hyaluronate Sodium\\Sodium Chloride', 'Product_Hydralazine Hydrochloride', 'Product_Hydralazine Hydrochloride\\Isosorbide Dinitrate', 'Product_Hydrochlorothiazide', 'Product_Hydrochlorothiazide\\Irbesartan', 'Product_Hydrochlorothiazide\\Lisinopril', 'Product_Hydrochlorothiazide\\Losartan', 'Product_Hydrochlorothiazide\\Losartan Potassium', 'Product_Hydrochlorothiazide\\Metoprolol Succinate', 'Product_Hydrochlorothiazide\\Olmesartan Medoxomil', 'Product_Hydrochlorothiazide\\Quinapril Hydrochloride', 'Product_Hydrochlorothiazide\\Ramipril', 'Product_Hydrochlorothiazide\\Spironolactone', 'Product_Hydrochlorothiazide\\Telmisartan', 'Product_Hydrochlorothiazide\\Triamterene', 'Product_Hydrochlorothiazide\\Valsartan', 'Product_Hydrocodone', 'Product_Hydrocodone Bitartrate', 'Product_Hydrocortisone', 'Product_Hydrocortisone Acetate', 'Product_Hydrocortisone Sodium Succinate', 'Product_Hydroflumethiazide\\Reserpine', 'Product_Hydromorphone', 'Product_Hydromorphone Hydrochloride', 'Product_Hydroquinone', 'Product_Hydroxocobalamin', 'Product_Hydroxychloroquine', 'Product_Hydroxychloroquine Diphosphate', 'Product_Hydroxychloroquine Sulfate', 'Product_Hydroxyethyl Starch 130/0.4', 'Product_Hydroxyurea', 'Product_Hydroxyzine Hydrochloride', 'Product_Hylan G-F 20', 'Product_Hymecromone', 'Product_Hypromellose 2910 (4000 Mpa.S)', 'Product_Ibandronate Sodium', 'Product_Ibrutinib', 'Product_Ibuprofen', 'Product_Icodextrin', 'Product_Icosapent', 'Product_Icosapent Ethyl', 'Product_Idarubicin Hydrochloride', 'Product_Ifenprodil Tartrate', 'Product_Iloprost', 'Product_Imatinib', 'Product_Imatinib Mesylate', 'Product_Imeglimin', 'Product_Imeglimin Hydrochloride', 'Product_Imidapril Hydrochloride', 'Product_Imipenem', 'Product_Imipramine', 'Product_Imipramine Pamoate', 'Product_Imrecoxib', 'Product_Inclisiran', 'Product_Inclisiran Sodium', 'Product_Indacaterol', 'Product_Indacaterol Maleate', 'Product_Indapamide', 'Product_Indapamide\\Perindopril', 'Product_Indapamide\\Perindopril Arginine', 'Product_Indapamide\\Perindopril Erbumine', 'Product_Indomethacin', 'Product_Infliximab', 'Product_Influenza A Virus A/California/7/2009 X-179a (H1n1) Antigen (Formaldehyde Inactivated)\\Influenza A Virus A/Victoria/210/2009 X-187 (H3n2) Antigen (Formaldehyde Inactivated)\\Influenza B Virus B/Brisbane/60/2008 Antigen (Formaldehyde Inactivated)', 'Product_Influenza A Virus A/California/7/2009(H1n1)-Like Antigen (Formaldehyde Inactivated)\\Influenza A Virus A/California/7/2009(H1n1)-Like Hemagglutinin Antigen (Propiolactone Inactivated)', 'Product_Influenza Virus Vaccine', 'Product_Influenza Virus Vaccine Quadrivalent Type A&B', 'Product_Influenza Virus Vaccine Trivalent Type A&B', 'Product_Influenza Virus Vaccine, Swine', 'Product_Ingenol Mebutate', 'Product_Insulin Aspart', 'Product_Insulin Aspart\\Insulin Degludec', 'Product_Insulin Beef', 'Product_Insulin Beef/Pork', 'Product_Insulin Defalan (Bovine)\\Insulin Defalan (Porcine)', 'Product_Insulin Degludec', 'Product_Insulin Degludec\\Liraglutide', 'Product_Insulin Detemir', 'Product_Insulin Glargine', 'Product_Insulin Glargine-Aglr', 'Product_Insulin Glargine-Yfgn', 'Product_Insulin Glargine\\Lixisenatide', 'Product_Insulin Glulisine', 'Product_Insulin Human', 'Product_Insulin Lispro', 'Product_Insulin Lispro-Aabc', 'Product_Insulin Nos', 'Product_Insulin Pork\\Insulin Purified Pork', 'Product_Interferon Beta-1a', 'Product_Interferon Beta-1b', 'Product_Interferon Gamma-1b', 'Product_Investigational Product', 'Product_Iodine', 'Product_Iodixanol', 'Product_Iodoquinol\\Metronidazole', 'Product_Iohexol', 'Product_Iomeprol', 'Product_Iopamidol', 'Product_Ioversol', 'Product_Ipilimumab', 'Product_Ipragliflozin', 'Product_Ipragliflozin L-Proline', 'Product_Ipragliflozin L-Proline\\Sitagliptin Phosphate', 'Product_Ipratropium', 'Product_Ipratropium Bromide', 'Product_Ipratropium Bromide\\Xylometazoline Hydrochloride', 'Product_Irbesartan', 'Product_Irinotecan Hydrochloride', 'Product_Iron', 'Product_Iron Polymaltose', 'Product_Iron Sucrose', 'Product_Iron\\Vitamins', 'Product_Isavuconazonium', 'Product_Isoflavones Soy', 'Product_Isoniazid', 'Product_Isoniazid\\Pyrazinamide\\Rifampin', 'Product_Isoniazid\\Rifampin', 'Product_Isophane Insulin Human Suspension', 'Product_Isopropanolamine', 'Product_Isopropyl Myristate', 'Product_Isoproterenol Hydrochloride', 'Product_Isosorbide', 'Product_Isosorbide Dinitrate', 'Product_Isosorbide Mononitrate', 'Product_Itraconazole', 'Product_Ivabradine', 'Product_Ivabradine Hydrochloride', 'Product_Ivabradine Oxalate', 'Product_Ixekizumab', 'Product_Jojoba Oil', 'Product_Ketamine', 'Product_Ketamine Hydrochloride', 'Product_Ketoprofen', 'Product_Ketorolac Tromethamine', 'Product_Labetalol', 'Product_Labetalol Hydrochloride', 'Product_Lacidipine', 'Product_Lacosamide', 'Product_Lactitol', 'Product_Lactulose', 'Product_Lamivudine', 'Product_Lamotrigine', 'Product_Lanolin\\Petrolatum', 'Product_Lanreotide', 'Product_Lanreotide Acetate', 'Product_Lansoprazole', 'Product_Lanthanum', 'Product_Lanthanum Carbonate', 'Product_Latanoprost', 'Product_Ledipasvir\\Sofosbuvir', 'Product_Leflunomide', 'Product_Lemborexant', 'Product_Lenalidomide', 'Product_Lenvatinib', 'Product_Lercanidipine', 'Product_Lercanidipine Hydrochloride', 'Product_Lercanidipine\\Valsartan', 'Product_Letrozole', 'Product_Leucogen', 'Product_Leucovorin Calcium', 'Product_Leucovorin\\Leucovorin Calcium', 'Product_Leuprolide Acetate', 'Product_Levacetylleucine', 'Product_Levalbuterol', 'Product_Levalbuterol Hydrochloride', 'Product_Levetiracetam', 'Product_Levocarnitine', 'Product_Levocetirizine Dihydrochloride', 'Product_Levodopa', 'Product_Levodropropizine', 'Product_Levofloxacin', 'Product_Levomepromazine', 'Product_Levomepromazine Embonate', 'Product_Levothyroxine', 'Product_Levothyroxine Sodium', 'Product_Lidocaine', 'Product_Lidocaine Hydrochloride', 'Product_Lidocaine\\Prilocaine', 'Product_Limaprost', 'Product_Linagliptin', 'Product_Linagliptin\\Metformin Hydrochloride', 'Product_Linezolid', 'Product_Liothyronine Sodium', 'Product_Liraglutide', 'Product_Lisinopril', 'Product_Lithium', 'Product_Lithium Carbonate', 'Product_Lixisenatide', 'Product_Loperamide', 'Product_Loperamide Hydrochloride', 'Product_Loperamide Oxide', 'Product_Lopinavir', 'Product_Lopinavir\\Ritonavir', 'Product_Loprazolam', 'Product_Loratadine', 'Product_Loratadine\\Pseudoephedrine Sulfate', 'Product_Lorazepam', 'Product_Lorcaserin Hydrochloride', 'Product_Lormetazepam', 'Product_Losartan', 'Product_Losartan Potassium', 'Product_Lovastatin', 'Product_Loxapine', 'Product_Loxapine Succinate', 'Product_Loxoprofen Sodium', 'Product_Lubiprostone', 'Product_Lurasidone Hydrochloride', 'Product_Luseogliflozin', 'Product_Lutein', 'Product_Lypressin', 'Product_Macitentan', 'Product_Magnesium', 'Product_Magnesium Aspartate', 'Product_Magnesium Aspartate Hydrochloride', 'Product_Magnesium Aspartate\\Potassium Aspartate', 'Product_Magnesium Carbonate', 'Product_Magnesium Carbonate Hydroxide', 'Product_Magnesium Carbonate\\Magnesium Trisilicate\\Sodium Bicarbonate', 'Product_Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Magnesium Chloride', 'Product_Magnesium Chloride\\Potassium Chloride\\Sodium Acetate Anhydrous\\Sodium Chloride\\Sodium Gluconate', 'Product_Magnesium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride\\Sodium Gluconate', 'Product_Magnesium Gluconate', 'Product_Magnesium Hydroxide', 'Product_Magnesium Oxide', 'Product_Magnesium Sulfate', 'Product_Magnesium Sulfate Heptahydrate', 'Product_Magnesium Trisilicate', 'Product_Maltodextrin', 'Product_Maltose', 'Product_Manidipine', 'Product_Manidipine Hydrochloride', 'Product_Mannitol', 'Product_Maprotiline Hydrochloride', 'Product_Mebeverine', 'Product_Meclizine Hydrochloride', 'Product_Medical Food', 'Product_Medium-Chain Triglycerides', 'Product_Medroxyprogesterone Acetate', 'Product_Megestrol', 'Product_Megestrol Acetate', 'Product_Melatonin', 'Product_Meloxicam', 'Product_Melperone', 'Product_Melperone Hydrochloride', 'Product_Melphalan', 'Product_Memantine', 'Product_Memantine Hydrochloride', 'Product_Meperidine', 'Product_Meperidine Hydrochloride', 'Product_Mepifylline', 'Product_Mepolizumab', 'Product_Mercaptopurine', 'Product_Meropenem', 'Product_Mesalamine', 'Product_Mesna', 'Product_Metacresol', 'Product_Metamizole', 'Product_Metamizole Magnesium', 'Product_Metamizole Sodium', 'Product_Metaraminol', 'Product_Metaraminol Bitartrate', 'Product_Metaxalone', 'Product_Metformin Hydrochloride', 'Product_Metformin Hydrochloride\\Pioglitazone Hydrochloride', 'Product_Metformin Hydrochloride\\Rosiglitazone Maleate', 'Product_Metformin Hydrochloride\\Saxagliptin', 'Product_Metformin Hydrochloride\\Saxagliptin Hydrochloride', 'Product_Metformin Hydrochloride\\Sitagliptin', 'Product_Metformin Hydrochloride\\Sitagliptin Phosphate', 'Product_Metformin Hydrochloride\\Vildagliptin', 'Product_Metformin Pamoate', 'Product_Metformin\\Sitagliptin', 'Product_Metformin\\Sitagliptin Phosphate', 'Product_Metformin\\Vildagliptin', 'Product_Methadone Hydrochloride', 'Product_Methimazole', 'Product_Methocarbamol', 'Product_Methotrexate', 'Product_Methotrexate Sodium', 'Product_Methoxy Polyethylene Glycol-Epoetin Beta', 'Product_Methylcellulose (4000 Cps)', 'Product_Methylcobalamin', 'Product_Methylcobalamin\\Niacinamide\\Pyridoxine', 'Product_Methyldopa', 'Product_Methylene Blue', 'Product_Methylephedrine Hydrochloride', 'Product_Methylprednisolone', 'Product_Methylprednisolone Aceponate', 'Product_Methylprednisolone Sodium Succinate', 'Product_Metildigoxin', 'Product_Metoclopramide', 'Product_Metoclopramide Hydrochloride', 'Product_Metolazone', 'Product_Metopimazine', 'Product_Metoprolol', 'Product_Metoprolol Succinate', 'Product_Metoprolol Tartrate', 'Product_Metronidazole', 'Product_Metronidazole Benzoate', 'Product_Metronidazole\\Metronidazole Hydrochloride', 'Product_Metyrapone', 'Product_Metyrapone Tartrate', 'Product_Mexiletine Hydrochloride', 'Product_Mezlocillin Sodium', 'Product_Mianserin', 'Product_Mianserin Hydrochloride', 'Product_Micafungin Sodium', 'Product_Miconazole', 'Product_Miconazole Nitrate', 'Product_Midazolam', 'Product_Midazolam Hydrochloride', 'Product_Midazolam Maleate', 'Product_Midodrine', 'Product_Midodrine Hydrochloride', 'Product_Mifepristone', 'Product_Miglitol', 'Product_Milk Thistle', 'Product_Milrinone', 'Product_Mineral Oil', 'Product_Minerals', 'Product_Minerals\\Omega-3 Fatty Acids\\Vitamins', 'Product_Minerals\\Vitamins', 'Product_Minocycline Hydrochloride', 'Product_Minodronic Acid', 'Product_Minoxidil', 'Product_Mirabegron', 'Product_Mirtazapine', 'Product_Misoprostol', 'Product_Mitiglinide', 'Product_Mitiglinide Calcium Anhydrous', 'Product_Mitomycin', 'Product_Moclobemide', 'Product_Modafinil', 'Product_Moexipril Hydrochloride', 'Product_Molnupiravir', 'Product_Molsidomine', 'Product_Mometasone', 'Product_Mometasone Furoate', 'Product_Montelukast Sodium', 'Product_Montelukast\\Montelukast Sodium', 'Product_Montmorillonite', 'Product_Morniflumate', 'Product_Morniflumate\\Niflumic Acid', 'Product_Morphine', 'Product_Morphine Hydrochloride', 'Product_Morphine Sulfate', 'Product_Mosapride', 'Product_Moxifloxacin', 'Product_Moxifloxacin Hydrochloride', 'Product_Moxonidine', 'Product_Mycophenolate Mofetil', 'Product_Mycophenolate Sodium', 'Product_Mycophenolic Acid', 'Product_Nabumetone', 'Product_Nadolol', 'Product_Nadroparin', 'Product_Nadroparin Calcium', 'Product_Nafamostat Mesylate', 'Product_Nafronyl', 'Product_Nafronyl Oxalate', 'Product_Naftopidil', 'Product_Nalmefene', 'Product_Naloxegol', 'Product_Naloxegol Oxalate', 'Product_Naloxone', 'Product_Naloxone Hydrochloride', 'Product_Naloxone Hydrochloride\\Oxycodone Hydrochloride', 'Product_Naloxone Hydrochloride\\Tilidine Hydrochloride', 'Product_Naloxone Hydrochloride\\Tilidine Phosphate', 'Product_Naproxen', 'Product_Naproxen Sodium', 'Product_Naratriptan Hydrochloride', 'Product_Natamycin', 'Product_Nateglinide', 'Product_Nebivolol', 'Product_Nebivolol Hydrochloride', 'Product_Nefopam', 'Product_Nefopam Hydrochloride', 'Product_Neostigmine Methylsulfate', 'Product_Nepafenac', 'Product_Nesiritide', 'Product_Nevirapine', 'Product_Niacin', 'Product_Niacinamide', 'Product_Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Hydrochloride', 'Product_Nicardipine', 'Product_Nicardipine Hydrochloride', 'Product_Nicorandil', 'Product_Nifedipine', 'Product_Nifuroxazide', 'Product_Nilotinib', 'Product_Nimesulide', 'Product_Nimodipine', 'Product_Nintedanib', 'Product_Niraparib', 'Product_Nirmatrelvir\\Ritonavir', 'Product_Nitrazepam', 'Product_Nitrendipine', 'Product_Nitric Oxide', 'Product_Nitrofurantoin', 'Product_Nitrofurantoin\\Nitrofurantoin Monohydrate', 'Product_Nitrofurazone', 'Product_Nitroglycerin', 'Product_Nivolumab', 'Product_Nizatidine', 'Product_Norepinephrine Bitartrate', 'Product_Norepinephrine Hydrochloride, (+/-)-', 'Product_Norepinephrine\\Norepinephrine Bitartrate', 'Product_Norfenefrine Hydrochloride', 'Product_Norfloxacin', 'Product_Nortriptyline', 'Product_Nortriptyline Hydrochloride', 'Product_Novobiocin', 'Product_Nystatin', 'Product_Obinutuzumab', 'Product_Ocrelizumab', 'Product_Octreotide', 'Product_Octreotide Acetate', 'Product_Ofloxacin', 'Product_Olanzapine', 'Product_Olmesartan', 'Product_Olmesartan Medoxomil', 'Product_Olodaterol Hydrochloride\\Tiotropium Bromide Monohydrate', 'Product_Olopatadine Hydrochloride', 'Product_Omalizumab', 'Product_Omapatrilat', 'Product_Omarigliptin', 'Product_Omega-3 Fatty Acids', 'Product_Omega-3-Acid Ethyl Esters', 'Product_Omeprazole', 'Product_Omeprazole Magnesium', 'Product_Omeprazole Sodium', 'Product_Ondansetron', 'Product_Ondansetron Hydrochloride', 'Product_Opipramol', 'Product_Opipramol Hydrochloride', 'Product_Ornithine Aspartate', 'Product_Ornithine Oxoglurate', 'Product_Orphenadrine Citrate', 'Product_Oseltamivir', 'Product_Oseltamivir Phosphate', 'Product_Otilimab', 'Product_Oxacillin Sodium', 'Product_Oxaliplatin', 'Product_Oxazepam', 'Product_Oxcarbazepine', 'Product_Oxybutynin', 'Product_Oxybutynin Chloride', 'Product_Oxycodone', 'Product_Oxycodone Hydrochloride', 'Product_Oxygen', 'Product_Oxymetazoline Hydrochloride', 'Product_Ozagrel', 'Product_Paclitaxel', 'Product_Palbociclib', 'Product_Paliperidone', 'Product_Paliperidone Palmitate', 'Product_Pamidronate Disodium', 'Product_Panax Ginseng Whole', 'Product_Pancrelipase', 'Product_Pancrelipase Amylase\\Pancrelipase Lipase\\Pancrelipase Protease', 'Product_Panitumumab', 'Product_Panobinostat', 'Product_Panthenol', 'Product_Pantoprazole', 'Product_Pantoprazole Magnesium', 'Product_Pantoprazole Sodium', 'Product_Pantoprazole Sodium Anhydrous', 'Product_Papaverine Hydrochloride', 'Product_Paraffin', 'Product_Parethoxycaine Hydrochloride', 'Product_Paricalcitol', 'Product_Paroxetine', 'Product_Paroxetine Hydrochloride', 'Product_Paroxetine Mesylate', 'Product_Pasireotide', 'Product_Pasireotide Diaspartate', 'Product_Patiromer', 'Product_Pazopanib Hydrochloride', 'Product_Pazufloxacin', 'Product_Pegfilgrastim', 'Product_Peginterferon Alfa-2a', 'Product_Pegvisomant', 'Product_Pembrolizumab', 'Product_Pemetrexed Disodium', 'Product_Penicillin', 'Product_Penicillin G', 'Product_Penicillin G Sodium', 'Product_Penicillin V', 'Product_Penicillin V Potassium', 'Product_Pentaerythritol Tetranitrate', 'Product_Pentamidine Isethionate', 'Product_Pentastarch', 'Product_Pentazocine', 'Product_Pentazocine Lactate', 'Product_Pentosan Polysulfate Sodium', 'Product_Pentoxifylline', 'Product_Pergolide', 'Product_Perindopril', 'Product_Perindopril Arginine', 'Product_Perindopril Erbumine', 'Product_Permethrin', 'Product_Petrolatum', 'Product_Phenacetin', 'Product_Phenazopyridine', 'Product_Phenethicillin Potassium', 'Product_Phenformin Hydrochloride', 'Product_Pheniramine', 'Product_Phenobarbital', 'Product_Phenobarbital Sodium', 'Product_Phenoxyethanol', 'Product_Phenprocoumon', 'Product_Phentermine\\Phentermine Hydrochloride', 'Product_Phenylbutazone', 'Product_Phenylephrine', 'Product_Phenylephrine Hydrochloride', 'Product_Phenylephrine Hydrochloride\\Witch Hazel', 'Product_Phenylephrine\\Shark Liver Oil', 'Product_Phenytoin', 'Product_Phenytoin Calcium', 'Product_Phenytoin Sodium', 'Product_Phloroglucinol', 'Product_Phloroglucinol\\1,3,5-Trimethoxybenzene', 'Product_Phospholipid', 'Product_Phytonadione', 'Product_Picloxydine Dihydrochloride', 'Product_Pimavanserin Tartrate', 'Product_Pindolol', 'Product_Pioglitazone', 'Product_Pioglitazone Hydrochloride', 'Product_Pipamperone', 'Product_Pipamperone Hydrochloride', 'Product_Piperacillin', 'Product_Piperacillin Sodium', 'Product_Piperacillin Sodium\\Tazobactam Sodium', 'Product_Piperazine Bisferulate', 'Product_Piracetam', 'Product_Piretanide', 'Product_Piretanide\\Ramipril', 'Product_Piribedil', 'Product_Piritramide', 'Product_Piroxicam', 'Product_Piroxicam Betadex', 'Product_Pitavastatin Calcium', 'Product_Pivmecillinam Hydrochloride', 'Product_Placebo', 'Product_Pneumococcal 7-Valent Conjugate Vaccine (Diphtheria Crm197 Protein)', 'Product_Pneumococcal Vaccine, Polyvalent', 'Product_Polycarbophil', 'Product_Polyethylene Glycol', 'Product_Polyethylene Glycol 3350', 'Product_Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol 3350\\Potassium\\Sodium Bicarbonate\\Sodium Sulfate', 'Product_Polyethylene Glycol 3350\\Potassium\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol 400', 'Product_Polyethylene Glycol 4000', 'Product_Polyethylene Glycol 4000\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol, Unspecified', 'Product_Polyethylene Glycol\\Potassium Chloride\\Sodium Bicarbonate', 'Product_Polyethylene Glycol\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Polyethylene Glycol\\Propylene Glycol', 'Product_Polyethylene Glycols', 'Product_Polyethylene Glycols\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycols\\Sodium Citrate\\Sodium Lauryl Sulfate\\Sorbic Acid', 'Product_Polystyrene Sulfonic Acid', 'Product_Polyvinyl Alcohol\\Povidone', 'Product_Pomalidomide', 'Product_Ponatinib', 'Product_Ponatinib Hydrochloride', 'Product_Posaconazole', 'Product_Potassium', 'Product_Potassium Aspartate', 'Product_Potassium Bicarbonate', 'Product_Potassium Bicarbonate\\Potassium Citrate', 'Product_Potassium Bromide', 'Product_Potassium Carbonate\\Potassium Citrate', 'Product_Potassium Chlorate', 'Product_Potassium Chloride', "Product_Potassium Chloride\\Riboflavin 5'-Phosphate Sodium", 'Product_Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Potassium Chloride\\Sodium Chloride', 'Product_Potassium Citrate', 'Product_Potassium Gluconate', 'Product_Potassium Iodide', 'Product_Potassium Perchlorate', 'Product_Potassium Phosphate', 'Product_Potassium Phosphate, Monobasic', 'Product_Poziotinib', 'Product_Pramipexole Dihydrochloride', 'Product_Pramipexole\\Pramipexole Dihydrochloride', 'Product_Pramlintide', 'Product_Pramlintide Acetate', 'Product_Prasugrel', 'Product_Prasugrel Hydrochloride', 'Product_Pravastatin', 'Product_Pravastatin Sodium', 'Product_Prazepam', 'Product_Prazosin', 'Product_Prazosin Hydrochloride', 'Product_Prednisolone', 'Product_Prednisolone Acetate', 'Product_Prednisolone Hemisuccinate', 'Product_Prednisolone Sodium Metazoate', 'Product_Prednisolone Sodium Succinate', 'Product_Prednisone', 'Product_Pregabalin', 'Product_Prilocaine', 'Product_Prilocaine Hydrochloride', 'Product_Primidone', 'Product_Pristinamycin', 'Product_Probiotics Nos', 'Product_Procaine Hydrochloride', 'Product_Procarbazine', 'Product_Procarbazine Hydrochloride', 'Product_Procaterol Hydrochloride', 'Product_Prochlorperazine', 'Product_Prochlorperazine Maleate', 'Product_Promethazine Hydrochloride', 'Product_Propacetamol Hydrochloride', 'Product_Propiverine', 'Product_Propofol', 'Product_Propoxyphene', 'Product_Propoxyphene Hydrochloride', 'Product_Propoxyphene Napsylate', 'Product_Propranolol Hydrochloride', 'Product_Propylene Glycol', 'Product_Propylene Glycol\\Sodium Benzoate\\Sodium Lauryl Sulfate', 'Product_Protamine', 'Product_Protamine Sulfate', 'Product_Prothipendyl', 'Product_Pseudoephedrine', 'Product_Pseudoephedrine Hydrochloride', 'Product_Psyllium Husk', 'Product_Psyllium Husk\\Senna Leaf', 'Product_Pygeum', 'Product_Pyrazinamide', 'Product_Pyridostigmine', 'Product_Pyridostigmine Bromide', 'Product_Pyridoxine', 'Product_Pyridoxine Hydrochloride', 'Product_Quazepam', 'Product_Quetiapine', 'Product_Quetiapine Fumarate', 'Product_Quinapril', 'Product_Quinapril Hydrochloride', 'Product_Quinidine', 'Product_Rabeprazole', 'Product_Rabeprazole Sodium', 'Product_Racecadotril', 'Product_Radiation Therapy', 'Product_Raloxifene Hydrochloride', 'Product_Ramelteon', 'Product_Ramipril', 'Product_Ramucirumab', 'Product_Ranibizumab', 'Product_Ranitidine', 'Product_Ranitidine Hydrochloride', 'Product_Ranolazine', 'Product_Rasagiline', 'Product_Rasagiline Mesylate', 'Product_Rasburicase', 'Product_Raxtozinameran', 'Product_Rebamipide', 'Product_Recombinant Varicella Zoster Virus (Vzv) Glycoprotein E', 'Product_Regorafenib Monohydrate', 'Product_Relugolix', 'Product_Remdesivir', 'Product_Remifentanil', 'Product_Remifentanil Hydrochloride', 'Product_Repaglinide', 'Product_Reproterol Hydrochloride', 'Product_Retinol', 'Product_Revefenacin', 'Product_Ribavirin', 'Product_Ribociclib', 'Product_Riboflavin', "Product_Riboflavin 5'-Phosphate Sodium", "Product_Riboflavin 5'-Phosphate Sodium Anhydrous", 'Product_Rifabutin', 'Product_Rifampin', 'Product_Rifaximin', 'Product_Rilmenidine', 'Product_Rilmenidine Phosphate', 'Product_Riltozinameran\\Tozinameran', 'Product_Riluzole', 'Product_Riociguat', 'Product_Risedronate Sodium', 'Product_Risperidone', 'Product_Ritonavir', 'Product_Rituximab', 'Product_Rivaroxaban', 'Product_Rivastigmine', 'Product_Rivastigmine Tartrate', 'Product_Rocuronium Bromide', 'Product_Rofecoxib', 'Product_Roflumilast', 'Product_Romosozumab', 'Product_Romosozumab-Aqqg', 'Product_Ropinirole', 'Product_Ropinirole Hydrochloride', 'Product_Ropivacaine Hydrochloride', 'Product_Rosiglitazone', 'Product_Rosiglitazone Maleate', 'Product_Rosuvastatin', 'Product_Rosuvastatin Calcium', 'Product_Rotigotine', 'Product_Roxadustat', 'Product_Rupatadine', 'Product_Rupatadine Fumarate', 'Product_Ruxolitinib', 'Product_Saccharomyces Cerevisiae', 'Product_Sacubitril', 'Product_Sacubitril\\Valsartan', 'Product_Safflower', 'Product_Salmeterol', 'Product_Salmeterol Xinafoate', 'Product_Salsalate', 'Product_Sargramostim', 'Product_Sarilumab', 'Product_Sasanlimab', 'Product_Saxagliptin', 'Product_Saxagliptin Hydrochloride', 'Product_Secukinumab', 'Product_Selexipag', 'Product_Semaglutide', 'Product_Senna Alata Whole', 'Product_Senna Leaf', 'Product_Senna Leaf\\Sennosides\\Sennosides A And B', 'Product_Sennoside A', 'Product_Sennosides', 'Product_Sennosides A And B', 'Product_Serelaxin', 'Product_Serenoa Repens Whole', 'Product_Sertraline Hydrochloride', 'Product_Sevelamer', 'Product_Sevelamer Carbonate', 'Product_Sevelamer Hydrochloride', 'Product_Sevoflurane', 'Product_Sibutramine Hydrochloride', 'Product_Sildenafil', 'Product_Sildenafil Citrate', 'Product_Silodosin', 'Product_Simeprevir', 'Product_Simvastatin', 'Product_Simvastatin\\Valsartan', 'Product_Sintilimab', 'Product_Sitagliptin', 'Product_Sitagliptin Phosphate', 'Product_Sodium', 'Product_Sodium Alginate\\Sodium Bicarbonate', 'Product_Sodium Ascorbate', 'Product_Sodium Bicarbonate', 'Product_Sodium Bromide', 'Product_Sodium Chloride', 'Product_Sodium Chloride\\Sodium Citrate', 'Product_Sodium Citrate', 'Product_Sodium Citrate\\Sodium Lauryl Sulfoacetate', 'Product_Sodium Citrate\\Sodium Lauryl Sulfoacetate\\Sorbitol', 'Product_Sodium Ferric Gluconate Complex', 'Product_Sodium Ferrous Citrate', 'Product_Sodium Fluoride', 'Product_Sodium Lactate', 'Product_Sodium Nitroprusside', 'Product_Sodium Perchlorate', 'Product_Sodium Perchlorate Monohydrate', 'Product_Sodium Phosphate', 'Product_Sodium Phosphate, Dibasic, Anhydrous\\Sodium Phosphate, Monobasic, Monohydrate', 'Product_Sodium Phosphate, Monobasic', 'Product_Sodium Phosphate, Monobasic, Anhydrous', 'Product_Sodium Phosphate, Monobasic, Unspecified Form', 'Product_Sodium Picosulfate', 'Product_Sodium Polystyrene Sulfonate', 'Product_Sodium Sulfate', 'Product_Sodium Zirconium Cyclosilicate', 'Product_Sofosbuvir', 'Product_Solifenacin', 'Product_Solifenacin Succinate', 'Product_Solifenacin\\Tamsulosin', 'Product_Somatostatin', 'Product_Somatropin', 'Product_Sorafenib', 'Product_Sorbitol', 'Product_Sotagliflozin', 'Product_Sotalol', 'Product_Sotalol Hydrochloride', 'Product_Sotrovimab', 'Product_Soy Isoflavones', 'Product_Soybean Oil', 'Product_Spiramycin', 'Product_Spiramycin Adipate', 'Product_Spironolactone', 'Product_Spironolactone\\Torsemide', 'Product_Starch, Corn', 'Product_Stavudine', 'Product_Stearic Acid', 'Product_Stearyl Alcohol', 'Product_Sucralfate', 'Product_Sucroferric Oxyhydroxide', 'Product_Sucrose', 'Product_Sufentanil', 'Product_Sufentanil Citrate', 'Product_Sulbactam', 'Product_Sulbactam Sodium', 'Product_Sulfadiazine', 'Product_Sulfadiazine\\Tetroxoprim', 'Product_Sulfadiazine\\Trimethoprim', 'Product_Sulfamethoxazole', 'Product_Sulfamethoxazole\\Trimethoprim', 'Product_Sulfasalazine', 'Product_Sulindac', 'Product_Sulpiride', 'Product_Sultamicillin Tosylate', 'Product_Sunitinib', 'Product_Sunitinib Malate', 'Product_Suprofen', 'Product_Sus Scrofa Corpus Luteum', 'Product_Suvorexant', 'Product_Tacrolimus', 'Product_Tadalafil', 'Product_Tafamidis', 'Product_Taliglucerase Alfa', 'Product_Tamoxifen', 'Product_Tamoxifen Citrate', 'Product_Tamsulosin', 'Product_Tamsulosin Hydrochloride', 'Product_Tapentadol', 'Product_Tapentadol Hydrochloride', 'Product_Taurine', 'Product_Tazobactam', 'Product_Technetium Tc-99m Medronate', 'Product_Technetium Tc-99m Sestamibi', 'Product_Technetium Tc-99m Tetrofosmin', 'Product_Tegaserod Maleate', 'Product_Teicoplanin', 'Product_Telithromycin', 'Product_Telmisartan', 'Product_Telotristat Ethyl', 'Product_Temazepam', 'Product_Temocapril Hydrochloride', 'Product_Temozolomide', 'Product_Temsirolimus', 'Product_Teneligliptin', 'Product_Tenofovir Disoproxil Fumarate', 'Product_Teprenone', 'Product_Terazosin Hydrochloride', 'Product_Terbinafine Hydrochloride', 'Product_Terbinafine\\Terbinafine Hydrochloride', 'Product_Terbutaline', 'Product_Terbutaline Sulfate', 'Product_Teriflunomide', 'Product_Teriparatide', 'Product_Terodiline Hydrochloride', 'Product_Testosterone', 'Product_Testosterone Cypionate', 'Product_Testosterone Enanthate', 'Product_Tetracycline', 'Product_Tetracycline Hydrochloride', 'Product_Tetrahydrozoline Hydrochloride', 'Product_Thalidomide', 'Product_Theophylline', 'Product_Theophylline Anhydrous', 'Product_Thiabendazole', 'Product_Thiamine', 'Product_Thiamine Hydrochloride', 'Product_Thioctic Acid', 'Product_Thiopental Sodium', 'Product_Thioridazine', 'Product_Thymopentin', 'Product_Tianeptine', 'Product_Tiapride', 'Product_Tiapride Hydrochloride', 'Product_Ticagrelor', 'Product_Ticlopidine Hydrochloride', 'Product_Tigecycline', 'Product_Tilidine', 'Product_Timolol Maleate', 'Product_Timolol Maleate\\Travoprost', 'Product_Timolol\\Timolol Maleate', 'Product_Tinzaparin', 'Product_Tinzaparin Sodium', 'Product_Tiotropium', 'Product_Tiotropium Bromide', 'Product_Tiotropium Bromide Monohydrate', 'Product_Tipiracil Hydrochloride\\Trifluridine', 'Product_Tiquizium Bromide', 'Product_Tirzepatide', 'Product_Titanium', 'Product_Titanium Dioxide', 'Product_Tobramycin', 'Product_Tobramycin\\Tobramycin Sulfate', 'Product_Tocilizumab', 'Product_Tocopherol', 'Product_Tofacitinib Citrate', 'Product_Tofogliflozin', 'Product_Tolrestat', 'Product_Tolterodine', 'Product_Tolterodine Tartrate', 'Product_Tolvaptan', 'Product_Topiramate', 'Product_Topotecan Hydrochloride', 'Product_Toremifene', 'Product_Torsemide', 'Product_Tozinameran', 'Product_Tramadol', 'Product_Tramadol Hydrochloride', 'Product_Trametinib', 'Product_Trametinib Dimethyl Sulfoxide', 'Product_Trandolapril', 'Product_Trandolapril Hydrochloride', 'Product_Trandolapril\\Verapamil', 'Product_Trandolapril\\Verapamil Hydrochloride', 'Product_Tranexamic Acid', 'Product_Trastuzumab', 'Product_Travoprost', 'Product_Trazodone Hydrochloride', 'Product_Trelagliptin Succinate', 'Product_Tremelimumab', 'Product_Treprostinil', 'Product_Triamcinolone', 'Product_Triamcinolone Acetonide', 'Product_Triamcinolone Acetonide 21-Acetate', 'Product_Triazolam', 'Product_Trichlormethiazide', 'Product_Tricholine Citrate', 'Product_Triclosan', 'Product_Trihexyphenidyl Hydrochloride', 'Product_Trimebutine', 'Product_Trimebutine Maleate', 'Product_Trimeprazine', 'Product_Trimetazidine', 'Product_Trimetazidine Dihydrochloride', 'Product_Trimethoprim', 'Product_Triptorelin', 'Product_Triptorelin Acetate', 'Product_Triptorelin Pamoate', 'Product_Trisodium Citrate Dihydrate', 'Product_Troglitazone', 'Product_Tropicamide', 'Product_Trospium', 'Product_Trospium Chloride', 'Product_Tuberculin Purified Protein Derivative', 'Product_Umeclidinium', 'Product_Umeclidinium Bromide', 'Product_Umeclidinium Bromide\\Vilanterol Trifenatate', 'Product_Unspecified Ingredient', 'Product_Upadacitinib', 'Product_Urapidil', 'Product_Ursocholic Acid', 'Product_Ursodiol', 'Product_Ursolic Acid', 'Product_Ustekinumab', 'Product_Valacyclovir Hydrochloride', 'Product_Valdecoxib', 'Product_Valganciclovir', 'Product_Valganciclovir Hydrochloride', 'Product_Valproate Sodium', 'Product_Valproic Acid', 'Product_Valsartan', 'Product_Vancomycin', 'Product_Vancomycin Hydrochloride', 'Product_Vandetanib', 'Product_Varenicline Tartrate', 'Product_Vasopressin', 'Product_Vecuronium Bromide', 'Product_Venetoclax', 'Product_Venlafaxine Hydrochloride', 'Product_Verapamil Hydrochloride', 'Product_Vilanterol', 'Product_Vilanterol Trifenatate', 'Product_Vildagliptin', 'Product_Viloxazine Hydrochloride', 'Product_Vinblastine Sulfate', 'Product_Vincristine', 'Product_Vincristine Sulfate', 'Product_Virginiamycin', 'Product_Vitamin A Palmitate', 'Product_Vitamin B', 'Product_Vitamin D Nos', 'Product_Vitamins', 'Product_Voglibose', 'Product_Voriconazole', 'Product_Vortioxetine Hydrobromide', 'Product_Warfarin', 'Product_Warfarin Potassium', 'Product_Warfarin Sodium', 'Product_Water', 'Product_Xanthinol Niacinate', 'Product_Xantofyl Palmitate', 'Product_Xipamide', 'Product_Xylitol', 'Product_Xylometazoline', 'Product_Xylometazoline Hydrochloride', 'Product_Yeast', 'Product_Zeaxanthin', 'Product_Zinc', 'Product_Zinc Sulfate', 'Product_Ziprasidone', 'Product_Zirconium Silicate', 'Product_Zoledronic Acid', 'Product_Zolpidem Tartrate', 'Product_Zopiclone', 'Product_Zuclopenthixol']
print(len(first_model_features))
second_model_features = ['Serious', 'Sex', 'Patient Age', 'Patient Weight', 'Product_.Alpha.-Acetyldigoxin', 'Product_.Alpha.-Glucose-1-Phosphate Disodium Tetrahydrate', 'Product_.Alpha.-Lipoic Acid', 'Product_.Alpha.-Tocopherol', 'Product_.Alpha.-Tocopherol Acetate', 'Product_.Alpha.-Tocopherol Acetate, D-', 'Product_.Alpha.-Tocopherol Acetate, Dl-', 'Product_.Alpha.-Tocopherol Acetate, Dl-\\.Beta.-Carotene\\Calcium Ascorbate\\Calcium Carbonate\\Calcium Pantothenate\\Cholecalciferol\\Cyanocobalamin\\Ferrous Fumarate\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Mononitrate\\Zinc Oxide', 'Product_.Alpha.-Tocopherol Succinate, D-', 'Product_.Alpha.-Tocopherol Succinate, Dl-', 'Product_.Alpha.-Tocopherol, D-', 'Product_.Alpha.-Tocopherol, Dl-', 'Product_.Alpha.-Tocopherol, Dl-\\Fish Oil', "Product_.Alpha.-Tocopherol\\.Alpha.-Tocopherol Acetate, Dl-\\Ascorbic Acid\\Biotin\\Cyanocobalamin\\Dexpanthenol\\Ergocalciferol\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Retinol\\Riboflavin 5'-Phosphate Sodium\\Thiamine", 'Product_.Alpha.-Tocopherol\\Allantoin', 'Product_.Beta.-Acetyldigoxin', 'Product_.Beta.-Carotene', 'Product_Abacavir Sulfate', 'Product_Abaloparatide', 'Product_Abatacept', 'Product_Abiraterone Acetate', 'Product_Abiraterone\\Abiraterone Acetate', 'Product_Abobotulinumtoxina', 'Product_Acarbose', 'Product_Acebutolol', 'Product_Acebutolol Hydrochloride', 'Product_Acenocoumarol', 'Product_Acetaminophen', 'Product_Acetaminophen\\Belladonna Leaf\\Caffeine\\Opium', 'Product_Acetaminophen\\Butabarbital\\Caffeine', 'Product_Acetaminophen\\Caffeine\\Codeine', 'Product_Acetaminophen\\Caffeine\\Codeine Phosphate', 'Product_Acetaminophen\\Caffeine\\Dihydrocodeine Bitartrate', 'Product_Acetaminophen\\Caffeine\\Opium', 'Product_Acetaminophen\\Caffeine\\Propyphenazone', 'Product_Acetaminophen\\Chlorpheniramine\\Dextromethorphan\\Diphenhydramine\\Doxylamine\\Pseudoephedrine', 'Product_Acetaminophen\\Chlorpheniramine\\Dextromethorphan\\Pseudoephedrine Hydrochloride', 'Product_Acetaminophen\\Codeine', 'Product_Acetaminophen\\Codeine Phosphate', 'Product_Acetaminophen\\Codeine Phosphate\\Meprobamate', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Doxylamine Succinate', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Acetaminophen\\Dextromethorphan Hydrobromide\\Pseudoephedrine', 'Product_Acetaminophen\\Dihydrocodeine', 'Product_Acetaminophen\\Diphenhydramine Hydrochloride', 'Product_Acetaminophen\\Hydrocodone', 'Product_Acetaminophen\\Hydrocodone Bitartrate', 'Product_Acetaminophen\\Opium', 'Product_Acetaminophen\\Oxycodone Hydrochloride', 'Product_Acetaminophen\\Propoxyphene', 'Product_Acetaminophen\\Propoxyphene Hydrochloride', 'Product_Acetaminophen\\Propoxyphene Napsylate', 'Product_Acetaminophen\\Tramadol', 'Product_Acetaminophen\\Tramadol Hydrochloride', 'Product_Acetazolamide', 'Product_Acetohexamide', 'Product_Acetylcysteine', 'Product_Acetylleucine', 'Product_Acitretin', 'Product_Aclidinium Bromide', 'Product_Acotiamide Hydrochloride', 'Product_Acyclovir', 'Product_Acyclovir\\Lidocaine\\Tetracaine Hydrochloride', 'Product_Adalimumab', 'Product_Adapalene', 'Product_Adenosine', 'Product_Adenosine Phosphate\\Cyanocobalamin\\Lidocaine Hydrochloride\\Pyridoxine Hydrochloride\\Thiamine Hydrochloride', 'Product_Adrafinil', 'Product_Afatinib', 'Product_Afatinib Dimaleate', 'Product_Aflibercept', 'Product_Alanine, Dl-\\Arginine\\Cysteine Hydrochloride\\Glycine\\Histidine\\Isoleucine\\Leucine\\Lysine Acetate\\Methionine\\Phenylalanine\\Phosphoric Acid\\Proline\\Serine\\Threonine\\Tryptophan\\Valine', 'Product_Alanine\\Arginine\\Aspartic Acid\\Glutamic Acid\\Glycine\\Histidine\\Isoleucine\\Leucine\\Lysine Acetate\\Methionine\\Phenylalanine\\Proline\\Serine\\Threonine\\Tryptophan\\Tyrosine\\Valine', 'Product_Alanyl Glutamine', 'Product_Albiglutide', 'Product_Albumin Human', 'Product_Albuterol', 'Product_Albuterol Sulfate', 'Product_Albuterol Sulfate\\Ipratropium Bromide', 'Product_Albuterol\\Ipratropium', 'Product_Albuterol\\Ipratropium Bromide', 'Product_Albuterol\\Theophylline', 'Product_Alcaftadine', 'Product_Alcohol', 'Product_Alectinib', 'Product_Alemtuzumab', 'Product_Alendronate Sodium', 'Product_Alendronic Acid', 'Product_Alfacalcidol', 'Product_Alfuzosin', 'Product_Alfuzosin Hydrochloride', 'Product_Alginic Acid\\Aluminum Hydroxide\\Magnesium Trisilicate\\Sodium Alginate\\Sodium Bicarbonate', 'Product_Alirocumab', 'Product_Aliskiren', 'Product_Aliskiren Hemifumarate', 'Product_Allantoin', 'Product_Allopurinol', 'Product_Almond Oil\\Benzalkonium Chloride\\Dexpanthenol', 'Product_Aloe Vera Leaf', 'Product_Alogliptin', 'Product_Alogliptin Benzoate', 'Product_Alogliptin Benzoate\\Metformin Hydrochloride', 'Product_Alogliptin Benzoate\\Pioglitazone Hydrochloride', 'Product_Alpelisib', 'Product_Alprazolam', 'Product_Alprostadil', 'Product_Alteplase', 'Product_Althiazide\\Spironolactone', 'Product_Aluminum', 'Product_Aluminum Hydroxide', 'Product_Aluminum Hydroxide \\Calcium Carbonate\\Sodium Alginate\\Sodium Bicarbonate', 'Product_Aluminum Hydroxide\\Dimethicone\\Magnesium Aluminum Silicate\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Dimethicone\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Magnesium Carbonate', 'Product_Aluminum Hydroxide\\Magnesium Hydroxide', 'Product_Aluminum Hydroxide\\Magnesium Trisilicate', 'Product_Aluminum Sulfate Tetradecahydrate\\Calcium Acetate Monohydrate', 'Product_Amantadine Hydrochloride', 'Product_Ambenonium Chloride', 'Product_Ambrisentan', 'Product_Ambroxol', 'Product_Ambroxol Hydrochloride', 'Product_Amikacin', 'Product_Amikacin Sulfate', 'Product_Amiloride', 'Product_Amiloride Hydrochloride', 'Product_Amiloride Hydrochloride\\Hydrochlorothiazide', 'Product_Amino Acids', 'Product_Amino Acids, Source Unspecified', 'Product_Amino Acids\\Calcium Chloride\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Amino Acids\\Carbohydrates\\Electrolytes Nos\\Vitamins', 'Product_Amino Acids\\Dextrose', 'Product_Amino Acids\\Dextrose\\Egg Phospholipids\\Electrolytes Nos\\Safflower Oil\\Soybean Oil', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos\\Soybean Oil', 'Product_Amino Acids\\Dextrose\\Electrolytes Nos\\Soybean Oil\\Vitamins', 'Product_Amino Acids\\Dextrose\\Vitamins', 'Product_Amino Acids\\Electrolytes Nos\\Xylitol', 'Product_Amino Acids\\Vitamins', 'Product_Aminobenzoic Acid', 'Product_Aminophylline', 'Product_Aminopyrine\\Dipyrone', 'Product_Aminoxyacetic Acid\\Ferrous Sulfate', 'Product_Amiodarone', 'Product_Amiodarone Hydrochloride', 'Product_Amisulpride', 'Product_Amitriptyline', 'Product_Amitriptyline Hydrochloride', 'Product_Amlodipine Besylate', 'Product_Amlodipine Besylate\\Atorvastatin Calcium', 'Product_Amlodipine Besylate\\Benazepril Hydrochloride', 'Product_Amlodipine Besylate\\Indapamide\\Perindopril Arginine', 'Product_Amlodipine Besylate\\Irbesartan', 'Product_Amlodipine Besylate\\Olmesartan Medoxomil', 'Product_Amlodipine Besylate\\Telmisartan', 'Product_Amlodipine Besylate\\Valsartan', 'Product_Amlodipine Maleate', 'Product_Amlodipine\\Atorvastatin\\Perindopril', 'Product_Amlodipine\\Perindopril', 'Product_Amlodipine\\Telmisartan', 'Product_Amlodipine\\Valsartan', 'Product_Ammonium Bromide', 'Product_Ammonium Chloride', 'Product_Ammonium Chloride\\Chlorpheniramine Maleate', 'Product_Ammonium Chloride\\Chlorpheniramine\\Codeine\\Ephedrine', 'Product_Ammonium Chloride\\Chlorpheniramine\\Sodium Citrate', 'Product_Ammonium Chloride\\Diphenhydramine\\Etofylline\\Sodium Citrate\\Theophylline', 'Product_Ammonium Glycyrrhizate', 'Product_Ammonium Lactate', 'Product_Amoxicillin', 'Product_Amoxicillin Sodium', 'Product_Amoxicillin\\Clarithromycin\\Lansoprazole', 'Product_Amoxicillin\\Clarithromycin\\Pantoprazole', 'Product_Amoxicillin\\Clavulanate Potassium', 'Product_Amoxicillin\\Clavulanic Acid', 'Product_Amphetamine Aspartate\\Amphetamine Sulfate\\Dextroamphetamine Saccharate\\Dextroamphetamine Sulfate', 'Product_Amphotericin B', 'Product_Amphotericin B\\Tetracycline', 'Product_Ampicillin', 'Product_Ampicillin Sodium\\Sulbactam Sodium', 'Product_Anagliptin', 'Product_Anagrelide Hydrochloride', 'Product_Anastrozole', 'Product_Angiotensin Ii', 'Product_Anhydrous Citric Acid', 'Product_Anhydrous Citric Acid\\Magnesium Oxide\\Sodium Picosulfate', 'Product_Anidulafungin', 'Product_Antithrombin Iii Human', 'Product_Apixaban', 'Product_Apomorphine Hydrochloride', 'Product_Apremilast', 'Product_Arformoterol Tartrate', 'Product_Aripiprazole', 'Product_Ascorbic Acid', 'Product_Ascorbic Acid\\Biotin\\Calcium Pantothenate\\Cyanocobalamin\\Folic Acid\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Mononitrate', 'Product_Ascorbic Acid\\Ergocalciferol\\Folic Acid\\Niacinamide\\Panthenol\\Retinol\\Riboflavin\\Thiamine Hydrochloride', 'Product_Ascorbic Acid\\Hesperidin Methylchalcone\\Ruscus Aculeatus Root', 'Product_Ascorbic Acid\\Hesperidin Methylchalcone\\Troxerutin', 'Product_Ascorbic Acid\\Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Ascorbate\\Sodium Chloride\\Sodium Sulfate', 'Product_Ascorbic Acid\\Polyethylene Glycol 4000\\Potassium Cl\\Sod Ascorbate\\Sod Cl\\Sod Sulfate', 'Product_Asian Ginseng', 'Product_Asparagine', 'Product_Aspartame', 'Product_Aspirin', 'Product_Aspirin Dl-Lysine', 'Product_Aspirin Lysine', 'Product_Aspirin\\Clopidogrel', 'Product_Aspirin\\Clopidogrel Bisulfate', 'Product_Aspirin\\Dihydroxyaluminum Aminoacetate Anhydrous\\Magnesium Carbonate', 'Product_Aspirin\\Dipyridamole', 'Product_Aspirin\\Glycine', 'Product_Aspirin\\Lansoprazole', 'Product_Astemizole', 'Product_Atazanavir', 'Product_Atenolol', 'Product_Atenolol Hydrochloride', 'Product_Atezolizumab', 'Product_Atlantic Salmon Oil', 'Product_Atorvastatin', 'Product_Atorvastatin Calcium', 'Product_Atorvastatin Calcium Trihydrate\\Ezetimibe', 'Product_Atorvastatin Calcium\\Ezetimibe', 'Product_Atovaquone', 'Product_Atracurium', 'Product_Atracurium Besylate', 'Product_Atropine', 'Product_Atropine Sulfate', 'Product_Atropine Sulfate\\Diphenoxylate Hydrochloride', 'Product_Avapritinib', 'Product_Avelumab', 'Product_Avobenzone\\Octisalate\\Octocrylene', 'Product_Avocado Oil', 'Product_Axitinib', 'Product_Azacitidine', 'Product_Azathioprine', 'Product_Azathioprine Sodium', 'Product_Azd-1222', 'Product_Azelnidipine', 'Product_Azelnidipine\\Olmesartan Medoxomil', 'Product_Azilsartan Kamedoxomil', 'Product_Azithromycin', 'Product_Azithromycin Anhydrous', 'Product_Azithromycin Dihydrate', 'Product_Azosemide', 'Product_Bacillus Calmette-Guerin Substrain Rivm Live Antigen', 'Product_Bacillus Calmette-Guerin Substrain Tice Live Antigen', 'Product_Bacillus Subtilis', 'Product_Bacitracin\\Hydrocortisone\\Neomycin Sulfate', 'Product_Baclofen', 'Product_Bambuterol Hydrochloride', 'Product_Bamlanivimab', 'Product_Bamlanivimab\\Etesevimab', 'Product_Barnidipine Hydrochloride', 'Product_Bazedoxifene', 'Product_Bcg Vaccine', 'Product_Beclomethasone', 'Product_Beclomethasone Dipropionate', 'Product_Beclomethasone Dipropionate\\Clotrimazole', 'Product_Beclomethasone Dipropionate\\Formoterol', 'Product_Belatacept', 'Product_Bempedoic Acid\\Ezetimibe', 'Product_Benazepril Hydrochloride', 'Product_Bendamustine Hydrochloride', 'Product_Benfluorex', 'Product_Benfluorex Hydrochloride', 'Product_Benfotiamine', 'Product_Benfotiamine\\Cyanocobalamin', 'Product_Benidipine', 'Product_Benidipine Hydrochloride', 'Product_Benserazide', 'Product_Benserazide Hydrochloride\\Levodopa', 'Product_Benserazide\\Levodopa', 'Product_Benzalkonium Chloride\\Salicylic Acid\\Triamcinolone Acetonide', 'Product_Benzodiazepine', 'Product_Benzoic Acid\\Methenamine\\Sodium Salicylate', 'Product_Benzonatate', 'Product_Benzthiazide\\Triamterene', 'Product_Benzydamine Hydrochloride\\Chlorhexidine Gluconate', 'Product_Bepotastine Besylate', 'Product_Beraprost Sodium', 'Product_Beta Carotene', 'Product_Beta Glucan', 'Product_Betahistine Hydrochloride', 'Product_Betahistine Mesilate', 'Product_Betamethasone', 'Product_Betamethasone Acetate\\Betamethasone Sodium Phosphate', 'Product_Betamethasone Dipropionate', 'Product_Betamethasone Dipropionate\\Betamethasone Sodium Phosphate', 'Product_Betamethasone Dipropionate\\Clioquinol', 'Product_Betamethasone Dipropionate\\Clotrimazole', 'Product_Betamethasone Dipropionate\\Clotrimazole\\Gentamicin Sulfate', 'Product_Betamethasone Sodium Phosphate', 'Product_Betamethasone Valerate', 'Product_Betamethasone Valerate\\Gentamicin Sulfate', 'Product_Betamethasone\\Betamethasone Dipropionate\\Clotrimazole', 'Product_Betamethasone\\Clotrimazole', 'Product_Betamethasone\\Fusidic Acid', 'Product_Betaxolol', 'Product_Betaxolol Hydrochloride', 'Product_Bethanechol Chloride', 'Product_Bevacizumab', 'Product_Bezafibrate', 'Product_Bicalutamide', 'Product_Bifidobacterium Bifidum', 'Product_Bifidobacterium Longum Subsp. Infantis', 'Product_Bifidobacterium Spp.', 'Product_Bilastine', 'Product_Bilberry', 'Product_Bimatoprost', 'Product_Binimetinib', 'Product_Biotin', 'Product_Bisacodyl', 'Product_Bisacodyl\\Docusate Sodium\\Sennosides A And B', 'Product_Bisacodyl\\Sennosides A And B', 'Product_Bismuth', 'Product_Bismuth Hydroxide', 'Product_Bismuth Subcarbonate\\Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Bismuth Subcitrate Potassium\\Metronidazole\\Tetracycline Hydrochloride', 'Product_Bismuth Subgallate', 'Product_Bismuth Subgallate\\Bufexamac\\Lidocaine Hydrochloride', 'Product_Bismuth Subgallate\\Bufexamac\\Lidocaine Hydrochloride\\Titanium Dioxide', 'Product_Bismuth Subnitrate', 'Product_Bismuth Subnitrate\\Magnesium Oxide\\Magnesium Tris', 'Product_Bismuth Subnitrate\\Magnesium Oxide\\Magnesium Trisilicate\\Sodium Phosphate, Monobasic, Anhydrous', 'Product_Bismuth Subsalicylate', 'Product_Bismuth\\Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Bisoprolol', 'Product_Bisoprolol Fumarate', 'Product_Bisoprolol Fumarate\\Hydrochlorothiazide', 'Product_Bisoprolol\\Perindopril', 'Product_Blinatumomab', 'Product_Bococizumab', 'Product_Boldine\\Sodium Bicarbonate\\Sodium Phosphate, Monobasic, Anhydrous\\Sodium Sulfate', 'Product_Boric Acid\\Sodium Borate', 'Product_Boric Acid\\Zinc Oxide', 'Product_Bortezomib', 'Product_Bosentan', 'Product_Brexpiprazole', 'Product_Brimonidine Tartrate\\Timolol Maleate', 'Product_Brolucizumab', 'Product_Bromazepam', 'Product_Bromfenac Sodium', 'Product_Bromhexine', 'Product_Brotizolam', 'Product_Budesonide', 'Product_Budesonide\\Formoterol', 'Product_Budesonide\\Formoterol Fumarate', 'Product_Budesonide\\Formoterol Fumarate Dihydrate', 'Product_Budesonide\\Formoterol\\Glycopyrronium', 'Product_Bufexamac', 'Product_Bufexamac\\Lidocaine Hydrochloride', 'Product_Buflomedil', 'Product_Buflomedil Hydrochloride', 'Product_Buformin Hydrochloride', 'Product_Bumetanide', 'Product_Bupivacaine Hydrochloride', 'Product_Buprenorphine', 'Product_Buprenorphine Hydrochloride', 'Product_Bupropion', 'Product_Bupropion Hydrochloride', 'Product_Bupropion Hydrochloride\\Naltrexone Hydrochloride', 'Product_Buspirone Hydrochloride', 'Product_Cabergoline', 'Product_Cabozantinib', 'Product_Cabozantinib S-Malate', 'Product_Cafedrine\\Theodrenaline', 'Product_Caffeine', 'Product_Caffeine\\Diphenhydramine Hydrochloride\\Ethenzamide\\Phenacetin\\Thiamine Mononitrate', 'Product_Caffeine\\Phenobarbital', 'Product_Calcitonin Salmon', 'Product_Calcitriol', 'Product_Calcium', 'Product_Calcium Acetate', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Acetate', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Acetate\\Sodium Acetate\\Sodium Chlori', 'Product_Calcium Acetate\\Magnesium Acetate\\Potassium Chloride', 'Product_Calcium Ascorbate', 'Product_Calcium Carbonate', 'Product_Calcium Carbonate\\Cholecalciferol', 'Product_Calcium Carbonate\\Ergocalciferol', 'Product_Calcium Carbonate\\Ergocalciferol\\Retinol', 'Product_Calcium Carbonate\\Ergocalciferol\\Vitamin A', 'Product_Calcium Carbonate\\Vitamin D', 'Product_Calcium Carbonate\\Vitamin D\\Zinc', 'Product_Calcium Chloride', 'Product_Calcium Chloride\\Dextrose Monohydrate\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Dextrose\\Magnesium Chloride\\Potassium Chloride', 'Product_Calcium Chloride\\Dextrose\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Chloride', 'Product_Calcium Chloride\\Dextrose\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Hetastarch\\Magnesium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Icodextrin\\Magnesium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Magnesium Chloride\\Potassium Chloride\\Sodium Acetate Anhydrous\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride\\Trihydroxypalmitamidohydroxypropyl Myristyl Ether', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate\\Sorbitol', 'Product_Calcium Chloride\\Potassium Chloride\\Sodium Lactate', 'Product_Calcium Citrate', 'Product_Calcium Citrate\\Cholecalciferol', 'Product_Calcium Dobesilate', 'Product_Calcium Gluceptate', 'Product_Calcium Gluconate', 'Product_Calcium Gluconate Monohydrate', 'Product_Calcium Gluconate\\Dextrose\\Magnesium Sulfate\\Potassium Acetate\\Potassium Phosphate, Dibasic\\Zinc Sulfate', 'Product_Calcium Lactate', 'Product_Calcium Pantothenate', 'Product_Calcium Pantothenate\\Cyanocobalamin\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Hydrochloride', 'Product_Calcium Phosphate', 'Product_Calcium Phosphate, Dibasic, Anhydrous\\Magnesium Glycerophosphate\\Phosphoric Acid\\Sodium Phosphate, Dibasic', 'Product_Calcium Phosphate, Monobasic, Anhydrous\\Magnesium Glycerophosphate\\Phosphoric Acid\\Sodium Phosphate, Dibasic', 'Product_Calcium Phosphate\\Ergocalciferol', 'Product_Calcium Polycarbophil', 'Product_Calcium Polystyrene Sulfonate', 'Product_Calcium\\Cholecalciferol', 'Product_Calcium\\Cholecalciferol\\Ergocalciferol', 'Product_Calcium\\Cholecalciferol\\Magnesium', 'Product_Calcium\\Ergocalciferol', 'Product_Calcium\\Vitamin D', 'Product_Canagliflozin', 'Product_Canagliflozin\\Metformin Hydrochloride', 'Product_Canagliflozin\\Teneligliptin', 'Product_Canakinumab', 'Product_Candesartan', 'Product_Candesartan Cilexetil', 'Product_Candesartan Cilexetil\\Hydrochlorothiazide', 'Product_Cannabidiol', 'Product_Canrenoate Potassium', 'Product_Canrenone', 'Product_Capecitabine', 'Product_Caprylic Acid', 'Product_Capsaicin', 'Product_Captopril', 'Product_Captopril\\Hydrochlorothiazide', 'Product_Carbamazepine', 'Product_Carbazochrome Sodium Sulfonate', 'Product_Carbidopa\\Entacapone\\Levodopa', 'Product_Carbidopa\\Levodopa', 'Product_Carbimazole', 'Product_Carbocysteine', 'Product_Carbohydrates', 'Product_Carbohydrates\\Electrolytes Nos\\Minerals\\Protein\\Vitamins', 'Product_Carbohydrates\\Fatty Acids\\Minerals\\Protein\\Vitamins', 'Product_Carbohydrates\\Potassium Chloride\\Sodium Chloride\\Sodium Lactate', 'Product_Carbomer', 'Product_Carboplatin', 'Product_Carboxymethylcellulose Sodium', 'Product_Carisoprodol', 'Product_Carperitide', 'Product_Carvedilol', 'Product_Carvedilol Hydrochloride', 'Product_Carvedilol Phosphate', 'Product_Caspofungin', 'Product_Caspofungin Acetate', 'Product_Cefaclor', 'Product_Cefazolin', 'Product_Cefazolin Sodium', 'Product_Cefcapene Pivoxil Hydrochloride', 'Product_Cefdinir', 'Product_Cefditoren Pivoxil', 'Product_Cefepime Hydrochloride', 'Product_Cefixime', 'Product_Cefmetazole', 'Product_Cefmetazole Sodium', 'Product_Cefoperazone Sodium\\Sulbactam Sodium', 'Product_Cefoperazone\\Sulbactam', 'Product_Cefoperazone\\Sulbactam Sodium', 'Product_Cefotaxime Sodium', 'Product_Cefotiam\\Mepivacaine', 'Product_Cefoxitin Sodium', 'Product_Cefpodoxime Proxetil', 'Product_Ceftazidime', 'Product_Ceftazidime Sodium', 'Product_Ceftriaxone', 'Product_Ceftriaxone Sodium', 'Product_Ceftriaxone Sodium\\Tazobactam Sodium', 'Product_Cefuroxime', 'Product_Cefuroxime Axetil', 'Product_Cefuroxime Sodium', 'Product_Celecoxib', 'Product_Celiprolol', 'Product_Celiprolol Hydrochloride', 'Product_Cephalexin', 'Product_Cephalexin Hydrochloride', 'Product_Cephalexin Sodium', 'Product_Cerivastatin Sodium', 'Product_Certolizumab', 'Product_Certolizumab Pegol', 'Product_Certoparin Sodium', 'Product_Cetirizine Hydrochloride', 'Product_Cetirizine Hydrochloride\\Pseudoephedrine Hydrochloride', 'Product_Cetuximab', 'Product_Chloramphenicol', 'Product_Chloramphenicol Palmitate', 'Product_Chloramphenicol\\Hydrocortisone', 'Product_Chloramphenicol\\Hydrocortisone Acetate', 'Product_Chloramphenicol\\Hydrocortisone Acetate\\Metronidazole\\Natamycin', 'Product_Chloramphenicol\\Hydrocortisone Acetate\\Metronidazole\\Nystatin', 'Product_Chlorhexidine Gluconate', 'Product_Chlorpheniramine', 'Product_Chlorpheniramine Maleate', 'Product_Chlorpheniramine Maleate\\Dextromethorphan Hydrobromide\\Guaifenesin\\Phenylephrine Hydrochloride', 'Product_Chlorpromazine', 'Product_Chlorpromazine Hydrochloride', 'Product_Chlorpropamide', 'Product_Chlorthalidone', 'Product_Cholecalciferol', 'Product_Cholecalciferol\\Ergocalciferol', 'Product_Cholesterol', 'Product_Cholestyramine', 'Product_Choline Bitartrate\\Cyanocobalamin\\Cysteine\\Inositol\\Mammal Liver', 'Product_Chondroitin Sulfate (Bovine)', 'Product_Chondroitin Sulfate (Bovine)\\Glucosamine Hydrochloride', 'Product_Chondroitin Sulfate (Bovine)\\Glucosamine Sulfate', 'Product_Chromic Chloride\\Cuprous Chloride\\Ferric Chloride\\Manganese Chloride\\Potassium Iodide\\Sodium Fluoride\\Sodium Molybdate Dihydrate\\Sodium Selenite\\Zinc Chloride', 'Product_Ciclesonide', 'Product_Ciclopirox', 'Product_Cifenline', 'Product_Cifenline Succinate', 'Product_Cilastatin Sodium\\Imipenem', 'Product_Cilazapril Anhydrous', 'Product_Cilnidipine', 'Product_Cilostazol', 'Product_Cimetidine', 'Product_Cinacalcet Hydrochloride', 'Product_Cinnamon\\Herbals', 'Product_Cinnarizine', 'Product_Ciprofibrate', 'Product_Ciprofloxacin', 'Product_Ciprofloxacin Hydrochloride', 'Product_Ciprofloxacin Hydrochloride\\Hydrocortisone', 'Product_Ciprofloxacin Lactate', 'Product_Ciprofloxacin\\Ciprofloxacin Hydrochloride', 'Product_Cisapride', 'Product_Cisatracurium', 'Product_Cisatracurium Besylate', 'Product_Cisplatin', 'Product_Citalopram', 'Product_Citalopram Hydrobromide', 'Product_Citicoline', 'Product_Citicoline Sodium', 'Product_Citric Acid Monohydrate', 'Product_Citric Acid Monohydrate\\Dextrose\\Sodium Citrate', 'Product_Citric Acid Monohydrate\\Potassium Bicarbonate', 'Product_Citric Acid Monohydrate\\Potassium Bicarbonate\\Potassium Citrate', 'Product_Citric Acid Monohydrate\\Sodium Citrate', 'Product_Citrus Fiber', 'Product_Clarithromycin', 'Product_Clemastine', 'Product_Clemastine Fumarate', 'Product_Clevidipine', 'Product_Clikstar', 'Product_Clindamycin', 'Product_Clindamycin Hydrochloride', 'Product_Clindamycin Palmitate Hydrochloride', 'Product_Clindamycin Phosphate', 'Product_Clioquinol', 'Product_Clioquinol\\Hydrocortisone', 'Product_Clobazam', 'Product_Clobetasol', 'Product_Clobetasol Propionate', 'Product_Clobutinol', 'Product_Clomipramine Hydrochloride', 'Product_Clonazepam', 'Product_Clonidine', 'Product_Clonidine Hydrochloride', 'Product_Clonixin', 'Product_Clopidogrel Besilate', 'Product_Clopidogrel Bisulfate', 'Product_Clorazepate Dipotassium', 'Product_Clorazepic Acid', 'Product_Clotiazepam', 'Product_Clotrimazole', 'Product_Clotrimazole\\Hydrocortisone', 'Product_Cloxacillin', 'Product_Cloxacillin Sodium', 'Product_Clozapine', 'Product_Coagulation Factor Ix Human\\Coagulation Factor Vii Human\\Coagulation Factor X Human\\Protein C\\Protein S Human\\Prothrombin', 'Product_Cocarboxylase', 'Product_Cocoa Butter\\Phenylephrine Hydrochloride', 'Product_Coconut Oil', 'Product_Cod Liver Oil\\Urea\\Zinc Oxide', 'Product_Codeine', 'Product_Codeine Phosphate', 'Product_Codeine Phosphate\\Guaifenesin', 'Product_Codeine Sulfate', 'Product_Coenzyme A', 'Product_Colchicine', 'Product_Colchicine\\Dicyclomine Hydrochloride', 'Product_Colchicine\\Opium', 'Product_Colchicine\\Opium\\Tiemonium Methylsulfate', 'Product_Colesevelam Hydrochloride', 'Product_Colestipol', 'Product_Convallaria Majalis\\Crataegus Laevigata Leaf\\Proxyphylline', 'Product_Corticosteroid Nos', 'Product_Cortisone', 'Product_Cortisone Acetate', 'Product_Cosmetics', 'Product_Covid-19 Vaccine', 'Product_Creatine', 'Product_Creatine\\Dietary Supplement', 'Product_Cresol', 'Product_Croscarmellose Sodium', 'Product_Cupric Sulfate', 'Product_Cupric Sulfate Anhydrous', 'Product_Cyamemazine', 'Product_Cyanocobalamin', "Product_Cyanocobalamin\\Dexpanthenol\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin 5'-Phosphate Sodium\\Thiamine Hydrochloride\\Vitamin B Complex", 'Product_Cyanocobalamin\\Folic Acid', 'Product_Cyanocobalamin\\Folic Acid\\Pyridoxine', 'Product_Cyanocobalamin\\Folic Acid\\Pyridoxine Hydrochloride', 'Product_Cyanocobalamin\\Pyridoxine Hydrochloride\\Riboflavin\\Sodium Pantothenate\\Thiamine Hydrochloride\\Vitamin B Complex', 'Product_Cyanocobalamin\\Pyridoxine Hydrochloride\\Thiamine', 'Product_Cyanocobalamin\\Pyridoxine\\Thiamine', 'Product_Cyclobenzaprine', 'Product_Cyclobenzaprine Hydrochloride', 'Product_Cyclophosphamide', 'Product_Cyclophosphamide\\Cytarabine\\Dexamethasone\\Doxorubicin\\Methotrexate\\Vincristine', 'Product_Cyclosporine', 'Product_Cyproheptadine', 'Product_Cyproheptadine Hydrochloride', 'Product_Cytarabine', 'Product_Dabigatran', 'Product_Dabigatran Etexilate', 'Product_Dabigatran Etexilate Mesylate', 'Product_Dabrafenib', 'Product_Dabrafenib Mesylate', 'Product_Daclatasvir', 'Product_Daclizumab', 'Product_Dalteparin', 'Product_Dalteparin Sodium', 'Product_Danazol', 'Product_Dapagliflozin', 'Product_Dapagliflozin Propanediol', 'Product_Dapagliflozin Propanediol\\Metformin Hydrochloride', 'Product_Dapsone', 'Product_Dapsone\\Ferrous Oxalate', 'Product_Daptomycin', 'Product_Darbepoetin Alfa', 'Product_Dasabuvir\\Ombitasvir\\Paritaprevir\\Ritonavir', 'Product_Dasatinib', 'Product_Daunorubicin', 'Product_Deferasirox', 'Product_Deferiprone', 'Product_Deflazacort', 'Product_Degarelix Acetate', 'Product_Denosumab', 'Product_Desipramine Hydrochloride', 'Product_Desloratadine', 'Product_Desmopressin Acetate', 'Product_Desoximetasone', 'Product_Desvenlafaxine Succinate', 'Product_Device', 'Product_Device\\Insulin Nos', 'Product_Dexamethasone', 'Product_Dexamethasone Acetate', 'Product_Dexamethasone Sodium Phosphate', 'Product_Dexamethasone Sodium Phosphate\\Moxifloxacin Hydrochloride', 'Product_Dexamethasone Sodium Succinate', 'Product_Dexamethasone\\Tobramycin', 'Product_Dexchlorpheniramine Maleate', 'Product_Dexketoprofen', 'Product_Dexketoprofen Trometamol', 'Product_Dexlansoprazole', 'Product_Dexmedetomidine', 'Product_Dexpanthenol', "Product_Dexpanthenol\\Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin 5'-Phosphate Sodium\\Thiamine Hydrochloride", 'Product_Dextran', 'Product_Dextroamphetamine', 'Product_Dextroamphetamine Sulfate', 'Product_Dextromethorphan Hydrobromide\\Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Dextromethorphan Hydrobromide\\Quinidine Sulfate', 'Product_Dextrose Monohydrate', 'Product_Dextrose Monohydrate\\Sodium Chloride', 'Product_Dextrose\\Heparin', 'Product_Dextrose\\Heparin Sodium', 'Product_Dextrose\\Potassium Chloride\\Sodium Chloride', 'Product_Dextrose\\Potassium Chloride\\Sodium Chloride\\Sodium Citrate', 'Product_Dextrose\\Sodium Chloride', 'Product_Dextrose\\Water', 'Product_Diazepam', 'Product_Diclofenac', 'Product_Diclofenac Epolamine', 'Product_Diclofenac Sodium', 'Product_Diclofenac Sodium\\Misoprostol', 'Product_Dicyclomine Hydrochloride', 'Product_Dietary Supplement', 'Product_Dietary Supplement\\Herbals', 'Product_Dietary Supplement\\Herbals\\Minerals\\Vitamins', 'Product_Dietary Supplement\\Probiotics Nos', 'Product_Dietary Supplement\\Ubidecarenone', 'Product_Digitoxin', 'Product_Digoxin', 'Product_Dihydralazine Sulfate', 'Product_Dihydralazine Sulfate\\Hydrochlorothiazide\\Reserpine', 'Product_Dihydrocodeine', 'Product_Dihydrocodeine Bitartrate', 'Product_Dihydrocodeine Phosphate', 'Product_Dihydroergotamine Mesylate\\Heparin Sodium\\Lidocaine Hydrochloride', 'Product_Diltiazem', 'Product_Diltiazem Hydrochloride', 'Product_Diltiazem Malate', 'Product_Dimenhydrinate', 'Product_Dimethicone', 'Product_Dimethicone\\Polyethylene Glycol 4000\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Dimethindene', 'Product_Dimethindene Maleate', 'Product_Dimethyl Fumarate', 'Product_Dimethyl Sulfone', 'Product_Diosmin', 'Product_Diosmin\\Hesperidin', 'Product_Diphenhydramine', 'Product_Diphenhydramine Hydrochloride', 'Product_Diphtheria And Tetanus Toxoids And Acellular Pertussis Adsorbed And Inactivated Poliovirus Vaccine', 'Product_Dipivefrin Hydrochloride', 'Product_Dipyridamole', 'Product_Dipyrone', 'Product_Disodium Hydrogen Citrate', 'Product_Disopyramide Phosphate', 'Product_Distigmine Bromide', 'Product_Divalproex Sodium', 'Product_Dobutamine Hydrochloride', 'Product_Docetaxel', 'Product_Doconexent\\ Icosapent\\ Fish Oil\\ Omega-3 Fatty Acids', 'Product_Doconexent\\Minerals\\Vitamins', 'Product_Docusate', 'Product_Docusate Calcium', 'Product_Docusate Sodium', 'Product_Docusate Sodium\\Senna Leaf', 'Product_Docusate Sodium\\Sennoside B', 'Product_Docusate Sodium\\Sennosides A And B', 'Product_Dofetilide', 'Product_Domperidone', 'Product_Domperidone Maleate', 'Product_Donepezil', 'Product_Donepezil Hydrochloride', 'Product_Dopamine Hydrochloride', 'Product_Doripenem Monohydrate', 'Product_Dorzolamide Hydrochloride', 'Product_Dorzolamide Hydrochloride\\Timolol Maleate', 'Product_Doxazosin Mesylate', 'Product_Doxazosin\\Doxazosin Mesylate', 'Product_Doxepin', 'Product_Doxepin Hydrochloride', 'Product_Doxofylline', 'Product_Doxorubicin', 'Product_Doxorubicin Hydrochloride', 'Product_Doxycycline', 'Product_Doxycycline Calcium', 'Product_Doxycycline Hyclate', 'Product_Dronabinol', 'Product_Dronedarone', 'Product_Drotaverine', 'Product_Drotrecogin Alfa (Activated)', 'Product_Droxidopa', 'Product_Dulaglutide', 'Product_Duloxetine', 'Product_Duloxetine Hydrochloride', 'Product_Duloxetine Hydrochloride\\Repaglinide\\Ticagrelor', 'Product_Dupilumab', 'Product_Durvalumab', 'Product_Dutasteride', 'Product_Dutasteride\\Tamsulosin Hydrochloride', 'Product_Ebastine', 'Product_Ecabet', 'Product_Econazole Nitrate', 'Product_Edaravone', 'Product_Efinaconazole', 'Product_Efmoroctocog Alfa', 'Product_Efonidipine Hydrochloride', 'Product_Egg Phospholipids\\Glycerin\\Medium-Chain Triglycerides\\Soybean Oil', 'Product_Egg Phospholipids\\Glycerin\\Soybean Oil', 'Product_Egg Phospholipids\\Safflower Oil\\Soybean Oil', 'Product_Elasomeran', 'Product_Elbasvir', 'Product_Eldecalcitol', 'Product_Electrolytes Nos', 'Product_Electrolytes Nos\\Minerals', 'Product_Electrolytes Nos\\Minerals\\Sorbitol', 'Product_Electrolytes Nos\\Polyethylene Glycol 3350', 'Product_Electrolytes Nos\\Polyethylene Glycol 4000', 'Product_Electrolytes Nos\\Polyethylene Glycols', 'Product_Elexacaftor\\Ivacaftor\\Tezacaftor', 'Product_Empagliflozin', 'Product_Empagliflozin\\Linagliptin', 'Product_Empagliflozin\\Metformin Hydrochloride', 'Product_Emtricitabine\\Tenofovir Disoproxil Fumarate', 'Product_Enalapril', 'Product_Enalapril Maleate', 'Product_Enalapril Maleate\\Hydrochlorothiazide', 'Product_Enalapril\\Hydrochlorothiazide', 'Product_Encorafenib', 'Product_Enflurane', 'Product_Enfortumab Vedotin', 'Product_Enfortumab Vedotin\\Enfortumab Vedotin-Ejfv', 'Product_Enocitabine', 'Product_Enoxaparin', 'Product_Enoxaparin Sodium', 'Product_Enoximone', 'Product_Ensulizole\\Eucerin\\Octinoxate\\Octisalate\\Titanium Dioxide\\Zinc Oxide', 'Product_Entecavir', 'Product_Enzalutamide', 'Product_Epalrestat', 'Product_Eperisone Hydrochloride', 'Product_Ephedrine', 'Product_Epinastine Hydrochloride', 'Product_Epinephrine', 'Product_Epirubicin', 'Product_Epirubicin Hydrochloride', 'Product_Eplerenone', 'Product_Epoetin Delta', 'Product_Epoprostenol', 'Product_Epoprostenol Sodium', 'Product_Eprosartan Mesylate', 'Product_Eprosartan Mesylate\\Hydrochlorothiazide', 'Product_Erdafitinib', 'Product_Ergocalciferol', 'Product_Ergocalciferol\\Phytonadione\\Retinol\\Tocopherol', 'Product_Ergoloid Mesylates', 'Product_Erlotinib', 'Product_Erlotinib Hydrochloride', 'Product_Ertapenem', 'Product_Ertapenem Sodium', 'Product_Ertugliflozin', 'Product_Ertugliflozin Pidolate', 'Product_Erythromycin', 'Product_Erythromycin Ethylsuccinate', 'Product_Erythromycin Stearate', 'Product_Erythropoietin', 'Product_Escitalopram Oxalate', 'Product_Esomeprazole', 'Product_Esomeprazole Magnesium', 'Product_Esomeprazole Magnesium\\Naproxen', 'Product_Esomeprazole Sodium', 'Product_Estradiol', 'Product_Estramustine', 'Product_Estramustine Phosphate Sodium', 'Product_Estriol', 'Product_Estrogens', 'Product_Estrogens, Conjugated', 'Product_Estropipate', 'Product_Eszopiclone', 'Product_Etanercept', 'Product_Ethacrynic Acid', 'Product_Ethambutol Hydrochloride', 'Product_Ethyl Loflazepate', 'Product_Ethylenediamine Dihydrochloride', 'Product_Etilefrine', 'Product_Etizolam', 'Product_Etodolac', 'Product_Etofylline\\Theophylline', 'Product_Etomidate', 'Product_Etoposide', 'Product_Etoricoxib', 'Product_Everolimus', 'Product_Evolocumab', 'Product_Exemestane', 'Product_Exenatide', 'Product_Ezetimibe', 'Product_Ezetimibe\\Simvastatin', 'Product_Famotidine', 'Product_Favipiravir', 'Product_Febuxostat', 'Product_Felodipine', 'Product_Felodipine\\Metoprolol Succinate', 'Product_Felodipine\\Ramipril', 'Product_Fenofibrate', 'Product_Fenofibric Acid', 'Product_Fenoterol', 'Product_Fenoterol Hydrobromide', 'Product_Fenoterol Hydrobromide\\Ipratropium Bromide', 'Product_Fenoterol\\Ipratropium', 'Product_Fenoterol\\Ipratropium Bromide', 'Product_Fentanyl', 'Product_Fentanyl Citrate', 'Product_Ferric Carboxymaltose', 'Product_Ferric Derisomaltose', 'Product_Ferric Sodium Citrate', 'Product_Ferrous Ammonium Sulfate Anhydrous\\Folic Acid', 'Product_Ferrous Ascorbate', 'Product_Ferrous Fumarate', 'Product_Ferrous Glycine Sulfate', 'Product_Ferrous Succinate', 'Product_Ferrous Sulfate', 'Product_Ferrous Sulfate Anhydrous', 'Product_Ferrous Sulfate\\Folic Acid', 'Product_Ferrous Sulfate\\Folic Acid\\Gastric Mucin', 'Product_Ferrous Sulfate\\Mammal Liver', 'Product_Fesoterodine', 'Product_Fesoterodine Fumarate', 'Product_Fexofenadine Hydrochloride', 'Product_Fexofenadine Hydrochloride\\Pseudoephedrine Hydrochloride', 'Product_Filgotinib', 'Product_Filgrastim', 'Product_Finasteride', 'Product_Finasteride\\Tamsulosin', 'Product_Fish Oil', 'Product_Fish Oil\\Medium-Chain Triglycerides\\Olive Oil\\Soybean Oil', 'Product_Flavoxate Hydrochloride', 'Product_Flax Seed', 'Product_Flecainide', 'Product_Flecainide Acetate', 'Product_Floctafenine', 'Product_Flomoxef', 'Product_Flucloxacillin', 'Product_Fluconazole', 'Product_Flucytosine', 'Product_Fludarabine Phosphate', 'Product_Fludrocortisone', 'Product_Fludrocortisone Acetate', 'Product_Fluindione', 'Product_Flunisolide', 'Product_Flunitrazepam', 'Product_Fluorouracil', 'Product_Fluorouracil\\Irinotecan\\Leucovorin', 'Product_Fluorouracil\\Leucovorin Calcium\\Oxaliplatin', 'Product_Fluoxetine Hydrochloride', 'Product_Flurazepam Hydrochloride', 'Product_Flurbiprofen', 'Product_Flutamide', 'Product_Fluticasone', 'Product_Fluticasone Furoate', 'Product_Fluticasone Furoate Or Umeclidinium Bromide Or Vilanterol Trifenatate', 'Product_Fluticasone Furoate\\Umeclidinium Bromide\\Vilanterol Trifenatate', 'Product_Fluticasone Furoate\\Vilanterol Trifenatate', 'Product_Fluticasone Propionate', 'Product_Fluticasone Propionate\\Salmeterol Xinafoate', 'Product_Fluvastatin Sodium', 'Product_Folate Sodium', 'Product_Folic Acid', 'Product_Folic Acid\\Iron\\Minerals\\Vitamins', 'Product_Fondaparinux', 'Product_Fondaparinux Sodium', 'Product_Formoterol', 'Product_Formoterol Fumarate', 'Product_Formoterol Fumarate Dihydrate\\Mometasone Furoate', 'Product_Formoterol Fumarate\\Mometasone Furoate', 'Product_Formoterol\\Mometasone', 'Product_Fosfluconazole', 'Product_Fosfomycin', 'Product_Fosfomycin Sodium', 'Product_Fosfomycin Tromethamine', 'Product_Fosinopril', 'Product_Fosinopril Sodium', 'Product_Fosinopril Sodium\\Hydrochlorothiazide', 'Product_Fosinopril\\Hydrochlorothiazide', 'Product_Fosphenytoin', 'Product_Fructose\\Glycerin', 'Product_Fructose\\Insulin Beef', 'Product_Fulvestrant', 'Product_Furazolidone', 'Product_Furosemide', 'Product_Furosemide\\Potassium Chloride', 'Product_Furosemide\\Reserpine', 'Product_Furosemide\\Spironolactone', 'Product_Fusidate Sodium', 'Product_Gabapentin', 'Product_Gabapentin Enacarbil', 'Product_Galantamine', 'Product_Galantamine Hydrobromide', 'Product_Ganciclovir', 'Product_Ganciclovir Sodium', 'Product_Garlic', 'Product_Gatifloxacin', 'Product_Gatifloxacin\\Prednisolone Acetate', 'Product_Gelatin', 'Product_Gemcitabine', 'Product_Gemcitabine Hydrochloride', 'Product_Gemcitabine\\Gemcitabine Hydrochloride', 'Product_Gemfibrozil', 'Product_Gemigliptin\\Metformin', 'Product_Gentamicin', 'Product_Gentamicin Sulfate', 'Product_Gimeracil\\Oteracil Potassium\\Tegafur', 'Product_Ginkgo', 'Product_Ginseng Extract\\Herbs', 'Product_Glecaprevir\\Pibrentasvir', 'Product_Gliclazide', 'Product_Glimepiride', 'Product_Glimepiride\\Metformin', 'Product_Glimepiride\\Pioglitazone Hydrochloride', 'Product_Glimepiride\\Rosiglitazone Maleate', 'Product_Glipizide', 'Product_Gliquidone', 'Product_Glucagon', 'Product_Glucagon Hydrochloride', 'Product_Gluconic Acid', 'Product_Glucosamine', 'Product_Glucose Oxidase', 'Product_Glucose Oxidase Unspecified Source', 'Product_Glutamine\\Sodium Gualenate', 'Product_Glyburide', 'Product_Glyburide\\Metformin', 'Product_Glyburide\\Metformin Hydrochloride', 'Product_Glycerin\\Medium-Chain Triglycerides\\Phospholipid\\Soybean Oil', 'Product_Glycerin\\Mineral Oil\\Petrolatum', 'Product_Glyceryl Monostearate', 'Product_Glycine Max (Soybean) Seed Extract', 'Product_Glycine Max Whole', 'Product_Glycopyrrolate', 'Product_Glycopyrronium\\Indacaterol', 'Product_Glycosaminoglycans', 'Product_Golimumab', 'Product_Gonadorelin', 'Product_Goserelin', 'Product_Goserelin Acetate', 'Product_Granisetron Hydrochloride', 'Product_Granulocyte Colony-Stimulating Factor Nos', 'Product_Grazoprevir', 'Product_Guaifenesin', 'Product_Guaifenesin\\Pseudoephedrine Hydrochloride', 'Product_Guanabenz Acetate', 'Product_Guanfacine Hydrochloride', 'Product_Guar Gum', 'Product_Haloperidol', 'Product_Haloperidol Decanoate', 'Product_Hamamelis Virginiana Top Water\\Phenylephrine Hydrochloride', 'Product_Heparin Calcium', 'Product_Heparin Sodium', 'Product_Heparin Sodium\\Sodium Chloride', 'Product_Heparin Sodium\\Zinc Sulfate', 'Product_Hepatitis B Virus Subtype Adw2 Hbsag Surface Protein Antigen', 'Product_Herbals', 'Product_Herbals\\Homeopathics', 'Product_Herbals\\Iron\\Vitamins', 'Product_Herbals\\Karaya Gum', 'Product_Herbals\\Paullinia Cupana Seed', 'Product_Herbals\\Plantago Indica Whole', 'Product_Herbals\\Plantago Ovata Seed Coat', 'Product_Herbals\\Saw Palmetto', 'Product_Herbals\\Sennosides', 'Product_Herbals\\Vitamins', 'Product_Herbals\\Zinc Oxide', 'Product_Herpes Simplex Vaccine', 'Product_Hetastarch\\Sodium Chloride', 'Product_Homeopathics', 'Product_Horse Chestnut', 'Product_Human C1-Esterase Inhibitor', 'Product_Human Immunoglobulin G', 'Product_Human Immunoglobulin G\\Hyaluronidase (Human Recombinant)', 'Product_Hyaluronate Sodium', 'Product_Hyaluronate Sodium\\Sodium Chloride', 'Product_Hydralazine Hydrochloride', 'Product_Hydralazine Hydrochloride\\Isosorbide Dinitrate', 'Product_Hydrochlorothiazide', 'Product_Hydrochlorothiazide\\Irbesartan', 'Product_Hydrochlorothiazide\\Lisinopril', 'Product_Hydrochlorothiazide\\Losartan', 'Product_Hydrochlorothiazide\\Losartan Potassium', 'Product_Hydrochlorothiazide\\Metoprolol Succinate', 'Product_Hydrochlorothiazide\\Olmesartan Medoxomil', 'Product_Hydrochlorothiazide\\Quinapril Hydrochloride', 'Product_Hydrochlorothiazide\\Ramipril', 'Product_Hydrochlorothiazide\\Spironolactone', 'Product_Hydrochlorothiazide\\Telmisartan', 'Product_Hydrochlorothiazide\\Triamterene', 'Product_Hydrochlorothiazide\\Valsartan', 'Product_Hydrocodone', 'Product_Hydrocodone Bitartrate', 'Product_Hydrocortisone', 'Product_Hydrocortisone Acetate', 'Product_Hydrocortisone Sodium Succinate', 'Product_Hydroflumethiazide\\Reserpine', 'Product_Hydromorphone', 'Product_Hydromorphone Hydrochloride', 'Product_Hydroquinone', 'Product_Hydroxocobalamin', 'Product_Hydroxychloroquine', 'Product_Hydroxychloroquine Diphosphate', 'Product_Hydroxychloroquine Sulfate', 'Product_Hydroxyethyl Starch 130/0.4', 'Product_Hydroxyurea', 'Product_Hydroxyzine Hydrochloride', 'Product_Hylan G-F 20', 'Product_Hymecromone', 'Product_Hypromellose 2910 (4000 Mpa.S)', 'Product_Ibandronate Sodium', 'Product_Ibrutinib', 'Product_Ibuprofen', 'Product_Icodextrin', 'Product_Icosapent', 'Product_Icosapent Ethyl', 'Product_Idarubicin Hydrochloride', 'Product_Ifenprodil Tartrate', 'Product_Iloprost', 'Product_Imatinib', 'Product_Imatinib Mesylate', 'Product_Imeglimin', 'Product_Imeglimin Hydrochloride', 'Product_Imidapril Hydrochloride', 'Product_Imipenem', 'Product_Imipramine', 'Product_Imipramine Pamoate', 'Product_Imrecoxib', 'Product_Inclisiran', 'Product_Inclisiran Sodium', 'Product_Indacaterol', 'Product_Indacaterol Maleate', 'Product_Indapamide', 'Product_Indapamide\\Perindopril', 'Product_Indapamide\\Perindopril Arginine', 'Product_Indapamide\\Perindopril Erbumine', 'Product_Indomethacin', 'Product_Infliximab', 'Product_Influenza A Virus A/California/7/2009 X-179a (H1n1) Antigen (Formaldehyde Inactivated)\\Influenza A Virus A/Victoria/210/2009 X-187 (H3n2) Antigen (Formaldehyde Inactivated)\\Influenza B Virus B/Brisbane/60/2008 Antigen (Formaldehyde Inactivated)', 'Product_Influenza A Virus A/California/7/2009(H1n1)-Like Antigen (Formaldehyde Inactivated)\\Influenza A Virus A/California/7/2009(H1n1)-Like Hemagglutinin Antigen (Propiolactone Inactivated)', 'Product_Influenza Virus Vaccine', 'Product_Influenza Virus Vaccine Quadrivalent Type A&B', 'Product_Influenza Virus Vaccine Trivalent Type A&B', 'Product_Influenza Virus Vaccine, Swine', 'Product_Ingenol Mebutate', 'Product_Insulin Aspart', 'Product_Insulin Aspart\\Insulin Degludec', 'Product_Insulin Beef', 'Product_Insulin Beef/Pork', 'Product_Insulin Defalan (Bovine)\\Insulin Defalan (Porcine)', 'Product_Insulin Degludec', 'Product_Insulin Degludec\\Liraglutide', 'Product_Insulin Detemir', 'Product_Insulin Glargine', 'Product_Insulin Glargine-Aglr', 'Product_Insulin Glargine-Yfgn', 'Product_Insulin Glargine\\Lixisenatide', 'Product_Insulin Glulisine', 'Product_Insulin Human', 'Product_Insulin Lispro', 'Product_Insulin Lispro-Aabc', 'Product_Insulin Nos', 'Product_Insulin Pork\\Insulin Purified Pork', 'Product_Interferon Beta-1a', 'Product_Interferon Beta-1b', 'Product_Interferon Gamma-1b', 'Product_Investigational Product', 'Product_Iodine', 'Product_Iodixanol', 'Product_Iodoquinol\\Metronidazole', 'Product_Iohexol', 'Product_Iomeprol', 'Product_Iopamidol', 'Product_Ioversol', 'Product_Ipilimumab', 'Product_Ipragliflozin', 'Product_Ipragliflozin L-Proline', 'Product_Ipragliflozin L-Proline\\Sitagliptin Phosphate', 'Product_Ipratropium', 'Product_Ipratropium Bromide', 'Product_Ipratropium Bromide\\Xylometazoline Hydrochloride', 'Product_Irbesartan', 'Product_Irinotecan Hydrochloride', 'Product_Iron', 'Product_Iron Polymaltose', 'Product_Iron Sucrose', 'Product_Iron\\Vitamins', 'Product_Isavuconazonium', 'Product_Isoflavones Soy', 'Product_Isoniazid', 'Product_Isoniazid\\Pyrazinamide\\Rifampin', 'Product_Isoniazid\\Rifampin', 'Product_Isophane Insulin Human Suspension', 'Product_Isopropanolamine', 'Product_Isopropyl Myristate', 'Product_Isoproterenol Hydrochloride', 'Product_Isosorbide', 'Product_Isosorbide Dinitrate', 'Product_Isosorbide Mononitrate', 'Product_Itraconazole', 'Product_Ivabradine', 'Product_Ivabradine Hydrochloride', 'Product_Ivabradine Oxalate', 'Product_Ixekizumab', 'Product_Jojoba Oil', 'Product_Ketamine', 'Product_Ketamine Hydrochloride', 'Product_Ketoprofen', 'Product_Ketorolac Tromethamine', 'Product_Labetalol', 'Product_Labetalol Hydrochloride', 'Product_Lacidipine', 'Product_Lacosamide', 'Product_Lactitol', 'Product_Lactulose', 'Product_Lamivudine', 'Product_Lamotrigine', 'Product_Lanolin\\Petrolatum', 'Product_Lanreotide', 'Product_Lanreotide Acetate', 'Product_Lansoprazole', 'Product_Lanthanum', 'Product_Lanthanum Carbonate', 'Product_Latanoprost', 'Product_Ledipasvir\\Sofosbuvir', 'Product_Leflunomide', 'Product_Lemborexant', 'Product_Lenalidomide', 'Product_Lenvatinib', 'Product_Lercanidipine', 'Product_Lercanidipine Hydrochloride', 'Product_Lercanidipine\\Valsartan', 'Product_Letrozole', 'Product_Leucogen', 'Product_Leucovorin Calcium', 'Product_Leucovorin\\Leucovorin Calcium', 'Product_Leuprolide Acetate', 'Product_Levacetylleucine', 'Product_Levalbuterol', 'Product_Levalbuterol Hydrochloride', 'Product_Levetiracetam', 'Product_Levocarnitine', 'Product_Levocetirizine Dihydrochloride', 'Product_Levodopa', 'Product_Levodropropizine', 'Product_Levofloxacin', 'Product_Levomepromazine', 'Product_Levomepromazine Embonate', 'Product_Levothyroxine', 'Product_Levothyroxine Sodium', 'Product_Lidocaine', 'Product_Lidocaine Hydrochloride', 'Product_Lidocaine\\Prilocaine', 'Product_Limaprost', 'Product_Linagliptin', 'Product_Linagliptin\\Metformin Hydrochloride', 'Product_Linezolid', 'Product_Liothyronine Sodium', 'Product_Liraglutide', 'Product_Lisinopril', 'Product_Lithium', 'Product_Lithium Carbonate', 'Product_Lixisenatide', 'Product_Loperamide', 'Product_Loperamide Hydrochloride', 'Product_Loperamide Oxide', 'Product_Lopinavir', 'Product_Lopinavir\\Ritonavir', 'Product_Loprazolam', 'Product_Loratadine', 'Product_Loratadine\\Pseudoephedrine Sulfate', 'Product_Lorazepam', 'Product_Lorcaserin Hydrochloride', 'Product_Lormetazepam', 'Product_Losartan', 'Product_Losartan Potassium', 'Product_Lovastatin', 'Product_Loxapine', 'Product_Loxapine Succinate', 'Product_Loxoprofen Sodium', 'Product_Lubiprostone', 'Product_Lurasidone Hydrochloride', 'Product_Luseogliflozin', 'Product_Lutein', 'Product_Lypressin', 'Product_Macitentan', 'Product_Magnesium', 'Product_Magnesium Aspartate', 'Product_Magnesium Aspartate Hydrochloride', 'Product_Magnesium Aspartate\\Potassium Aspartate', 'Product_Magnesium Carbonate', 'Product_Magnesium Carbonate Hydroxide', 'Product_Magnesium Carbonate\\Magnesium Trisilicate\\Sodium Bicarbonate', 'Product_Magnesium Carbonate\\Sodium Bicarbonate', 'Product_Magnesium Chloride', 'Product_Magnesium Chloride\\Potassium Chloride\\Sodium Acetate Anhydrous\\Sodium Chloride\\Sodium Gluconate', 'Product_Magnesium Chloride\\Potassium Chloride\\Sodium Acetate\\Sodium Chloride\\Sodium Gluconate', 'Product_Magnesium Gluconate', 'Product_Magnesium Hydroxide', 'Product_Magnesium Oxide', 'Product_Magnesium Sulfate', 'Product_Magnesium Sulfate Heptahydrate', 'Product_Magnesium Trisilicate', 'Product_Maltodextrin', 'Product_Maltose', 'Product_Manidipine', 'Product_Manidipine Hydrochloride', 'Product_Mannitol', 'Product_Maprotiline Hydrochloride', 'Product_Mebeverine', 'Product_Meclizine Hydrochloride', 'Product_Medical Food', 'Product_Medium-Chain Triglycerides', 'Product_Medroxyprogesterone Acetate', 'Product_Megestrol', 'Product_Megestrol Acetate', 'Product_Melatonin', 'Product_Meloxicam', 'Product_Melperone', 'Product_Melperone Hydrochloride', 'Product_Melphalan', 'Product_Memantine', 'Product_Memantine Hydrochloride', 'Product_Meperidine', 'Product_Meperidine Hydrochloride', 'Product_Mepifylline', 'Product_Mepolizumab', 'Product_Mercaptopurine', 'Product_Meropenem', 'Product_Mesalamine', 'Product_Mesna', 'Product_Metacresol', 'Product_Metamizole', 'Product_Metamizole Magnesium', 'Product_Metamizole Sodium', 'Product_Metaraminol', 'Product_Metaraminol Bitartrate', 'Product_Metaxalone', 'Product_Metformin Hydrochloride', 'Product_Metformin Hydrochloride\\Pioglitazone Hydrochloride', 'Product_Metformin Hydrochloride\\Rosiglitazone Maleate', 'Product_Metformin Hydrochloride\\Saxagliptin', 'Product_Metformin Hydrochloride\\Saxagliptin Hydrochloride', 'Product_Metformin Hydrochloride\\Sitagliptin', 'Product_Metformin Hydrochloride\\Sitagliptin Phosphate', 'Product_Metformin Hydrochloride\\Vildagliptin', 'Product_Metformin Pamoate', 'Product_Metformin\\Sitagliptin', 'Product_Metformin\\Sitagliptin Phosphate', 'Product_Metformin\\Vildagliptin', 'Product_Methadone Hydrochloride', 'Product_Methimazole', 'Product_Methocarbamol', 'Product_Methotrexate', 'Product_Methotrexate Sodium', 'Product_Methoxy Polyethylene Glycol-Epoetin Beta', 'Product_Methylcellulose (4000 Cps)', 'Product_Methylcobalamin', 'Product_Methylcobalamin\\Niacinamide\\Pyridoxine', 'Product_Methyldopa', 'Product_Methylene Blue', 'Product_Methylephedrine Hydrochloride', 'Product_Methylprednisolone', 'Product_Methylprednisolone Aceponate', 'Product_Methylprednisolone Sodium Succinate', 'Product_Metildigoxin', 'Product_Metoclopramide', 'Product_Metoclopramide Hydrochloride', 'Product_Metolazone', 'Product_Metopimazine', 'Product_Metoprolol', 'Product_Metoprolol Succinate', 'Product_Metoprolol Tartrate', 'Product_Metronidazole', 'Product_Metronidazole Benzoate', 'Product_Metronidazole\\Metronidazole Hydrochloride', 'Product_Metyrapone', 'Product_Metyrapone Tartrate', 'Product_Mexiletine Hydrochloride', 'Product_Mezlocillin Sodium', 'Product_Mianserin', 'Product_Mianserin Hydrochloride', 'Product_Micafungin Sodium', 'Product_Miconazole', 'Product_Miconazole Nitrate', 'Product_Midazolam', 'Product_Midazolam Hydrochloride', 'Product_Midazolam Maleate', 'Product_Midodrine', 'Product_Midodrine Hydrochloride', 'Product_Mifepristone', 'Product_Miglitol', 'Product_Milk Thistle', 'Product_Milrinone', 'Product_Mineral Oil', 'Product_Minerals', 'Product_Minerals\\Omega-3 Fatty Acids\\Vitamins', 'Product_Minerals\\Vitamins', 'Product_Minocycline Hydrochloride', 'Product_Minodronic Acid', 'Product_Minoxidil', 'Product_Mirabegron', 'Product_Mirtazapine', 'Product_Misoprostol', 'Product_Mitiglinide', 'Product_Mitiglinide Calcium Anhydrous', 'Product_Mitomycin', 'Product_Moclobemide', 'Product_Modafinil', 'Product_Moexipril Hydrochloride', 'Product_Molnupiravir', 'Product_Molsidomine', 'Product_Mometasone', 'Product_Mometasone Furoate', 'Product_Montelukast Sodium', 'Product_Montelukast\\Montelukast Sodium', 'Product_Montmorillonite', 'Product_Morniflumate', 'Product_Morniflumate\\Niflumic Acid', 'Product_Morphine', 'Product_Morphine Hydrochloride', 'Product_Morphine Sulfate', 'Product_Mosapride', 'Product_Moxifloxacin', 'Product_Moxifloxacin Hydrochloride', 'Product_Moxonidine', 'Product_Mycophenolate Mofetil', 'Product_Mycophenolate Sodium', 'Product_Mycophenolic Acid', 'Product_Nabumetone', 'Product_Nadolol', 'Product_Nadroparin', 'Product_Nadroparin Calcium', 'Product_Nafamostat Mesylate', 'Product_Nafronyl', 'Product_Nafronyl Oxalate', 'Product_Naftopidil', 'Product_Nalmefene', 'Product_Naloxegol', 'Product_Naloxegol Oxalate', 'Product_Naloxone', 'Product_Naloxone Hydrochloride', 'Product_Naloxone Hydrochloride\\Oxycodone Hydrochloride', 'Product_Naloxone Hydrochloride\\Tilidine Hydrochloride', 'Product_Naloxone Hydrochloride\\Tilidine Phosphate', 'Product_Naproxen', 'Product_Naproxen Sodium', 'Product_Naratriptan Hydrochloride', 'Product_Natamycin', 'Product_Nateglinide', 'Product_Nebivolol', 'Product_Nebivolol Hydrochloride', 'Product_Nefopam', 'Product_Nefopam Hydrochloride', 'Product_Neostigmine Methylsulfate', 'Product_Nepafenac', 'Product_Nesiritide', 'Product_Nevirapine', 'Product_Niacin', 'Product_Niacinamide', 'Product_Niacinamide\\Pyridoxine Hydrochloride\\Riboflavin\\Thiamine Hydrochloride', 'Product_Nicardipine', 'Product_Nicardipine Hydrochloride', 'Product_Nicorandil', 'Product_Nifedipine', 'Product_Nifuroxazide', 'Product_Nilotinib', 'Product_Nimesulide', 'Product_Nimodipine', 'Product_Nintedanib', 'Product_Niraparib', 'Product_Nirmatrelvir\\Ritonavir', 'Product_Nitrazepam', 'Product_Nitrendipine', 'Product_Nitric Oxide', 'Product_Nitrofurantoin', 'Product_Nitrofurantoin\\Nitrofurantoin Monohydrate', 'Product_Nitrofurazone', 'Product_Nitroglycerin', 'Product_Nivolumab', 'Product_Nizatidine', 'Product_Norepinephrine Bitartrate', 'Product_Norepinephrine Hydrochloride, (+/-)-', 'Product_Norepinephrine\\Norepinephrine Bitartrate', 'Product_Norfenefrine Hydrochloride', 'Product_Norfloxacin', 'Product_Nortriptyline', 'Product_Nortriptyline Hydrochloride', 'Product_Novobiocin', 'Product_Nystatin', 'Product_Obinutuzumab', 'Product_Ocrelizumab', 'Product_Octreotide', 'Product_Octreotide Acetate', 'Product_Ofloxacin', 'Product_Olanzapine', 'Product_Olmesartan', 'Product_Olmesartan Medoxomil', 'Product_Olodaterol Hydrochloride\\Tiotropium Bromide Monohydrate', 'Product_Olopatadine Hydrochloride', 'Product_Omalizumab', 'Product_Omapatrilat', 'Product_Omarigliptin', 'Product_Omega-3 Fatty Acids', 'Product_Omega-3-Acid Ethyl Esters', 'Product_Omeprazole', 'Product_Omeprazole Magnesium', 'Product_Omeprazole Sodium', 'Product_Ondansetron', 'Product_Ondansetron Hydrochloride', 'Product_Opipramol', 'Product_Opipramol Hydrochloride', 'Product_Ornithine Aspartate', 'Product_Ornithine Oxoglurate', 'Product_Orphenadrine Citrate', 'Product_Oseltamivir', 'Product_Oseltamivir Phosphate', 'Product_Otilimab', 'Product_Oxacillin Sodium', 'Product_Oxaliplatin', 'Product_Oxazepam', 'Product_Oxcarbazepine', 'Product_Oxybutynin', 'Product_Oxybutynin Chloride', 'Product_Oxycodone', 'Product_Oxycodone Hydrochloride', 'Product_Oxygen', 'Product_Oxymetazoline Hydrochloride', 'Product_Ozagrel', 'Product_Paclitaxel', 'Product_Palbociclib', 'Product_Paliperidone', 'Product_Paliperidone Palmitate', 'Product_Pamidronate Disodium', 'Product_Panax Ginseng Whole', 'Product_Pancrelipase', 'Product_Pancrelipase Amylase\\Pancrelipase Lipase\\Pancrelipase Protease', 'Product_Panitumumab', 'Product_Panobinostat', 'Product_Panthenol', 'Product_Pantoprazole', 'Product_Pantoprazole Magnesium', 'Product_Pantoprazole Sodium', 'Product_Pantoprazole Sodium Anhydrous', 'Product_Papaverine Hydrochloride', 'Product_Paraffin', 'Product_Parethoxycaine Hydrochloride', 'Product_Paricalcitol', 'Product_Paroxetine', 'Product_Paroxetine Hydrochloride', 'Product_Paroxetine Mesylate', 'Product_Pasireotide', 'Product_Pasireotide Diaspartate', 'Product_Patiromer', 'Product_Pazopanib Hydrochloride', 'Product_Pazufloxacin', 'Product_Pegfilgrastim', 'Product_Peginterferon Alfa-2a', 'Product_Pegvisomant', 'Product_Pembrolizumab', 'Product_Pemetrexed Disodium', 'Product_Penicillin', 'Product_Penicillin G', 'Product_Penicillin G Sodium', 'Product_Penicillin V', 'Product_Penicillin V Potassium', 'Product_Pentaerythritol Tetranitrate', 'Product_Pentamidine Isethionate', 'Product_Pentastarch', 'Product_Pentazocine', 'Product_Pentazocine Lactate', 'Product_Pentosan Polysulfate Sodium', 'Product_Pentoxifylline', 'Product_Pergolide', 'Product_Perindopril', 'Product_Perindopril Arginine', 'Product_Perindopril Erbumine', 'Product_Permethrin', 'Product_Petrolatum', 'Product_Phenacetin', 'Product_Phenazopyridine', 'Product_Phenethicillin Potassium', 'Product_Phenformin Hydrochloride', 'Product_Pheniramine', 'Product_Phenobarbital', 'Product_Phenobarbital Sodium', 'Product_Phenoxyethanol', 'Product_Phenprocoumon', 'Product_Phentermine\\Phentermine Hydrochloride', 'Product_Phenylbutazone', 'Product_Phenylephrine', 'Product_Phenylephrine Hydrochloride', 'Product_Phenylephrine Hydrochloride\\Witch Hazel', 'Product_Phenylephrine\\Shark Liver Oil', 'Product_Phenytoin', 'Product_Phenytoin Calcium', 'Product_Phenytoin Sodium', 'Product_Phloroglucinol', 'Product_Phloroglucinol\\1,3,5-Trimethoxybenzene', 'Product_Phospholipid', 'Product_Phytonadione', 'Product_Picloxydine Dihydrochloride', 'Product_Pimavanserin Tartrate', 'Product_Pindolol', 'Product_Pioglitazone', 'Product_Pioglitazone Hydrochloride', 'Product_Pipamperone', 'Product_Pipamperone Hydrochloride', 'Product_Piperacillin', 'Product_Piperacillin Sodium', 'Product_Piperacillin Sodium\\Tazobactam Sodium', 'Product_Piperazine Bisferulate', 'Product_Piracetam', 'Product_Piretanide', 'Product_Piretanide\\Ramipril', 'Product_Piribedil', 'Product_Piritramide', 'Product_Piroxicam', 'Product_Piroxicam Betadex', 'Product_Pitavastatin Calcium', 'Product_Pivmecillinam Hydrochloride', 'Product_Placebo', 'Product_Pneumococcal 7-Valent Conjugate Vaccine (Diphtheria Crm197 Protein)', 'Product_Pneumococcal Vaccine, Polyvalent', 'Product_Polycarbophil', 'Product_Polyethylene Glycol', 'Product_Polyethylene Glycol 3350', 'Product_Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Polyethylene Glycol 3350\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol 3350\\Potassium\\Sodium Bicarbonate\\Sodium Sulfate', 'Product_Polyethylene Glycol 3350\\Potassium\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol 400', 'Product_Polyethylene Glycol 4000', 'Product_Polyethylene Glycol 4000\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycol, Unspecified', 'Product_Polyethylene Glycol\\Potassium Chloride\\Sodium Bicarbonate', 'Product_Polyethylene Glycol\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Polyethylene Glycol\\Propylene Glycol', 'Product_Polyethylene Glycols', 'Product_Polyethylene Glycols\\Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride\\Sodium Sulfate', 'Product_Polyethylene Glycols\\Sodium Citrate\\Sodium Lauryl Sulfate\\Sorbic Acid', 'Product_Polystyrene Sulfonic Acid', 'Product_Polyvinyl Alcohol\\Povidone', 'Product_Pomalidomide', 'Product_Ponatinib', 'Product_Ponatinib Hydrochloride', 'Product_Posaconazole', 'Product_Potassium', 'Product_Potassium Aspartate', 'Product_Potassium Bicarbonate', 'Product_Potassium Bicarbonate\\Potassium Citrate', 'Product_Potassium Bromide', 'Product_Potassium Carbonate\\Potassium Citrate', 'Product_Potassium Chlorate', 'Product_Potassium Chloride', "Product_Potassium Chloride\\Riboflavin 5'-Phosphate Sodium", 'Product_Potassium Chloride\\Sodium Bicarbonate\\Sodium Chloride', 'Product_Potassium Chloride\\Sodium Chloride', 'Product_Potassium Citrate', 'Product_Potassium Gluconate', 'Product_Potassium Iodide', 'Product_Potassium Perchlorate', 'Product_Potassium Phosphate', 'Product_Potassium Phosphate, Monobasic', 'Product_Poziotinib', 'Product_Pramipexole Dihydrochloride', 'Product_Pramipexole\\Pramipexole Dihydrochloride', 'Product_Pramlintide', 'Product_Pramlintide Acetate', 'Product_Prasugrel', 'Product_Prasugrel Hydrochloride', 'Product_Pravastatin', 'Product_Pravastatin Sodium', 'Product_Prazepam', 'Product_Prazosin', 'Product_Prazosin Hydrochloride', 'Product_Prednisolone', 'Product_Prednisolone Acetate', 'Product_Prednisolone Hemisuccinate', 'Product_Prednisolone Sodium Metazoate', 'Product_Prednisolone Sodium Succinate', 'Product_Prednisone', 'Product_Pregabalin', 'Product_Prilocaine', 'Product_Prilocaine Hydrochloride', 'Product_Primidone', 'Product_Pristinamycin', 'Product_Probiotics Nos', 'Product_Procaine Hydrochloride', 'Product_Procarbazine', 'Product_Procarbazine Hydrochloride', 'Product_Procaterol Hydrochloride', 'Product_Prochlorperazine', 'Product_Prochlorperazine Maleate', 'Product_Promethazine Hydrochloride', 'Product_Propacetamol Hydrochloride', 'Product_Propiverine', 'Product_Propofol', 'Product_Propoxyphene', 'Product_Propoxyphene Hydrochloride', 'Product_Propoxyphene Napsylate', 'Product_Propranolol Hydrochloride', 'Product_Propylene Glycol', 'Product_Propylene Glycol\\Sodium Benzoate\\Sodium Lauryl Sulfate', 'Product_Protamine', 'Product_Protamine Sulfate', 'Product_Prothipendyl', 'Product_Pseudoephedrine', 'Product_Pseudoephedrine Hydrochloride', 'Product_Psyllium Husk', 'Product_Psyllium Husk\\Senna Leaf', 'Product_Pygeum', 'Product_Pyrazinamide', 'Product_Pyridostigmine', 'Product_Pyridostigmine Bromide', 'Product_Pyridoxine', 'Product_Pyridoxine Hydrochloride', 'Product_Quazepam', 'Product_Quetiapine', 'Product_Quetiapine Fumarate', 'Product_Quinapril', 'Product_Quinapril Hydrochloride', 'Product_Quinidine', 'Product_Rabeprazole', 'Product_Rabeprazole Sodium', 'Product_Racecadotril', 'Product_Radiation Therapy', 'Product_Raloxifene Hydrochloride', 'Product_Ramelteon', 'Product_Ramipril', 'Product_Ramucirumab', 'Product_Ranibizumab', 'Product_Ranitidine', 'Product_Ranitidine Hydrochloride', 'Product_Ranolazine', 'Product_Rasagiline', 'Product_Rasagiline Mesylate', 'Product_Rasburicase', 'Product_Raxtozinameran', 'Product_Rebamipide', 'Product_Recombinant Varicella Zoster Virus (Vzv) Glycoprotein E', 'Product_Regorafenib Monohydrate', 'Product_Relugolix', 'Product_Remdesivir', 'Product_Remifentanil', 'Product_Remifentanil Hydrochloride', 'Product_Repaglinide', 'Product_Reproterol Hydrochloride', 'Product_Retinol', 'Product_Revefenacin', 'Product_Ribavirin', 'Product_Ribociclib', 'Product_Riboflavin', "Product_Riboflavin 5'-Phosphate Sodium", "Product_Riboflavin 5'-Phosphate Sodium Anhydrous", 'Product_Rifabutin', 'Product_Rifampin', 'Product_Rifaximin', 'Product_Rilmenidine', 'Product_Rilmenidine Phosphate', 'Product_Riltozinameran\\Tozinameran', 'Product_Riluzole', 'Product_Riociguat', 'Product_Risedronate Sodium', 'Product_Risperidone', 'Product_Ritonavir', 'Product_Rituximab', 'Product_Rivaroxaban', 'Product_Rivastigmine', 'Product_Rivastigmine Tartrate', 'Product_Rocuronium Bromide', 'Product_Rofecoxib', 'Product_Roflumilast', 'Product_Romosozumab', 'Product_Romosozumab-Aqqg', 'Product_Ropinirole', 'Product_Ropinirole Hydrochloride', 'Product_Ropivacaine Hydrochloride', 'Product_Rosiglitazone', 'Product_Rosiglitazone Maleate', 'Product_Rosuvastatin', 'Product_Rosuvastatin Calcium', 'Product_Rotigotine', 'Product_Roxadustat', 'Product_Rupatadine', 'Product_Rupatadine Fumarate', 'Product_Ruxolitinib', 'Product_Saccharomyces Cerevisiae', 'Product_Sacubitril', 'Product_Sacubitril\\Valsartan', 'Product_Safflower', 'Product_Salmeterol', 'Product_Salmeterol Xinafoate', 'Product_Salsalate', 'Product_Sargramostim', 'Product_Sarilumab', 'Product_Sasanlimab', 'Product_Saxagliptin', 'Product_Saxagliptin Hydrochloride', 'Product_Secukinumab', 'Product_Selexipag', 'Product_Semaglutide', 'Product_Senna Alata Whole', 'Product_Senna Leaf', 'Product_Senna Leaf\\Sennosides\\Sennosides A And B', 'Product_Sennoside A', 'Product_Sennosides', 'Product_Sennosides A And B', 'Product_Serelaxin', 'Product_Serenoa Repens Whole', 'Product_Sertraline Hydrochloride', 'Product_Sevelamer', 'Product_Sevelamer Carbonate', 'Product_Sevelamer Hydrochloride', 'Product_Sevoflurane', 'Product_Sibutramine Hydrochloride', 'Product_Sildenafil', 'Product_Sildenafil Citrate', 'Product_Silodosin', 'Product_Simeprevir', 'Product_Simvastatin', 'Product_Simvastatin\\Valsartan', 'Product_Sintilimab', 'Product_Sitagliptin', 'Product_Sitagliptin Phosphate', 'Product_Sodium', 'Product_Sodium Alginate\\Sodium Bicarbonate', 'Product_Sodium Ascorbate', 'Product_Sodium Bicarbonate', 'Product_Sodium Bromide', 'Product_Sodium Chloride', 'Product_Sodium Chloride\\Sodium Citrate', 'Product_Sodium Citrate', 'Product_Sodium Citrate\\Sodium Lauryl Sulfoacetate', 'Product_Sodium Citrate\\Sodium Lauryl Sulfoacetate\\Sorbitol', 'Product_Sodium Ferric Gluconate Complex', 'Product_Sodium Ferrous Citrate', 'Product_Sodium Fluoride', 'Product_Sodium Lactate', 'Product_Sodium Nitroprusside', 'Product_Sodium Perchlorate', 'Product_Sodium Perchlorate Monohydrate', 'Product_Sodium Phosphate', 'Product_Sodium Phosphate, Dibasic, Anhydrous\\Sodium Phosphate, Monobasic, Monohydrate', 'Product_Sodium Phosphate, Monobasic', 'Product_Sodium Phosphate, Monobasic, Anhydrous', 'Product_Sodium Phosphate, Monobasic, Unspecified Form', 'Product_Sodium Picosulfate', 'Product_Sodium Polystyrene Sulfonate', 'Product_Sodium Sulfate', 'Product_Sodium Zirconium Cyclosilicate', 'Product_Sofosbuvir', 'Product_Solifenacin', 'Product_Solifenacin Succinate', 'Product_Solifenacin\\Tamsulosin', 'Product_Somatostatin', 'Product_Somatropin', 'Product_Sorafenib', 'Product_Sorbitol', 'Product_Sotagliflozin', 'Product_Sotalol', 'Product_Sotalol Hydrochloride', 'Product_Sotrovimab', 'Product_Soy Isoflavones', 'Product_Soybean Oil', 'Product_Spiramycin', 'Product_Spiramycin Adipate', 'Product_Spironolactone', 'Product_Spironolactone\\Torsemide', 'Product_Starch, Corn', 'Product_Stavudine', 'Product_Stearic Acid', 'Product_Stearyl Alcohol', 'Product_Sucralfate', 'Product_Sucroferric Oxyhydroxide', 'Product_Sucrose', 'Product_Sufentanil', 'Product_Sufentanil Citrate', 'Product_Sulbactam', 'Product_Sulbactam Sodium', 'Product_Sulfadiazine', 'Product_Sulfadiazine\\Tetroxoprim', 'Product_Sulfadiazine\\Trimethoprim', 'Product_Sulfamethoxazole', 'Product_Sulfamethoxazole\\Trimethoprim', 'Product_Sulfasalazine', 'Product_Sulindac', 'Product_Sulpiride', 'Product_Sultamicillin Tosylate', 'Product_Sunitinib', 'Product_Sunitinib Malate', 'Product_Suprofen', 'Product_Sus Scrofa Corpus Luteum', 'Product_Suvorexant', 'Product_Tacrolimus', 'Product_Tadalafil', 'Product_Tafamidis', 'Product_Taliglucerase Alfa', 'Product_Tamoxifen', 'Product_Tamoxifen Citrate', 'Product_Tamsulosin', 'Product_Tamsulosin Hydrochloride', 'Product_Tapentadol', 'Product_Tapentadol Hydrochloride', 'Product_Taurine', 'Product_Tazobactam', 'Product_Technetium Tc-99m Medronate', 'Product_Technetium Tc-99m Sestamibi', 'Product_Technetium Tc-99m Tetrofosmin', 'Product_Tegaserod Maleate', 'Product_Teicoplanin', 'Product_Telithromycin', 'Product_Telmisartan', 'Product_Telotristat Ethyl', 'Product_Temazepam', 'Product_Temocapril Hydrochloride', 'Product_Temozolomide', 'Product_Temsirolimus', 'Product_Teneligliptin', 'Product_Tenofovir Disoproxil Fumarate', 'Product_Teprenone', 'Product_Terazosin Hydrochloride', 'Product_Terbinafine Hydrochloride', 'Product_Terbinafine\\Terbinafine Hydrochloride', 'Product_Terbutaline', 'Product_Terbutaline Sulfate', 'Product_Teriflunomide', 'Product_Teriparatide', 'Product_Terodiline Hydrochloride', 'Product_Testosterone', 'Product_Testosterone Cypionate', 'Product_Testosterone Enanthate', 'Product_Tetracycline', 'Product_Tetracycline Hydrochloride', 'Product_Tetrahydrozoline Hydrochloride', 'Product_Thalidomide', 'Product_Theophylline', 'Product_Theophylline Anhydrous', 'Product_Thiabendazole', 'Product_Thiamine', 'Product_Thiamine Hydrochloride', 'Product_Thioctic Acid', 'Product_Thiopental Sodium', 'Product_Thioridazine', 'Product_Thymopentin', 'Product_Tianeptine', 'Product_Tiapride', 'Product_Tiapride Hydrochloride', 'Product_Ticagrelor', 'Product_Ticlopidine Hydrochloride', 'Product_Tigecycline', 'Product_Tilidine', 'Product_Timolol Maleate', 'Product_Timolol Maleate\\Travoprost', 'Product_Timolol\\Timolol Maleate', 'Product_Tinzaparin', 'Product_Tinzaparin Sodium', 'Product_Tiotropium', 'Product_Tiotropium Bromide', 'Product_Tiotropium Bromide Monohydrate', 'Product_Tipiracil Hydrochloride\\Trifluridine', 'Product_Tiquizium Bromide', 'Product_Tirzepatide', 'Product_Titanium', 'Product_Titanium Dioxide', 'Product_Tobramycin', 'Product_Tobramycin\\Tobramycin Sulfate', 'Product_Tocilizumab', 'Product_Tocopherol', 'Product_Tofacitinib Citrate', 'Product_Tofogliflozin', 'Product_Tolrestat', 'Product_Tolterodine', 'Product_Tolterodine Tartrate', 'Product_Tolvaptan', 'Product_Topiramate', 'Product_Topotecan Hydrochloride', 'Product_Toremifene', 'Product_Torsemide', 'Product_Tozinameran', 'Product_Tramadol', 'Product_Tramadol Hydrochloride', 'Product_Trametinib', 'Product_Trametinib Dimethyl Sulfoxide', 'Product_Trandolapril', 'Product_Trandolapril Hydrochloride', 'Product_Trandolapril\\Verapamil', 'Product_Trandolapril\\Verapamil Hydrochloride', 'Product_Tranexamic Acid', 'Product_Trastuzumab', 'Product_Travoprost', 'Product_Trazodone Hydrochloride', 'Product_Trelagliptin Succinate', 'Product_Tremelimumab', 'Product_Treprostinil', 'Product_Triamcinolone', 'Product_Triamcinolone Acetonide', 'Product_Triamcinolone Acetonide 21-Acetate', 'Product_Triazolam', 'Product_Trichlormethiazide', 'Product_Tricholine Citrate', 'Product_Triclosan', 'Product_Trihexyphenidyl Hydrochloride', 'Product_Trimebutine', 'Product_Trimebutine Maleate', 'Product_Trimeprazine', 'Product_Trimetazidine', 'Product_Trimetazidine Dihydrochloride', 'Product_Trimethoprim', 'Product_Triptorelin', 'Product_Triptorelin Acetate', 'Product_Triptorelin Pamoate', 'Product_Trisodium Citrate Dihydrate', 'Product_Troglitazone', 'Product_Tropicamide', 'Product_Trospium', 'Product_Trospium Chloride', 'Product_Tuberculin Purified Protein Derivative', 'Product_Umeclidinium', 'Product_Umeclidinium Bromide', 'Product_Umeclidinium Bromide\\Vilanterol Trifenatate', 'Product_Unspecified Ingredient', 'Product_Upadacitinib', 'Product_Urapidil', 'Product_Ursocholic Acid', 'Product_Ursodiol', 'Product_Ursolic Acid', 'Product_Ustekinumab', 'Product_Valacyclovir Hydrochloride', 'Product_Valdecoxib', 'Product_Valganciclovir', 'Product_Valganciclovir Hydrochloride', 'Product_Valproate Sodium', 'Product_Valproic Acid', 'Product_Valsartan', 'Product_Vancomycin', 'Product_Vancomycin Hydrochloride', 'Product_Vandetanib', 'Product_Varenicline Tartrate', 'Product_Vasopressin', 'Product_Vecuronium Bromide', 'Product_Venetoclax', 'Product_Venlafaxine Hydrochloride', 'Product_Verapamil Hydrochloride', 'Product_Vilanterol', 'Product_Vilanterol Trifenatate', 'Product_Vildagliptin', 'Product_Viloxazine Hydrochloride', 'Product_Vinblastine Sulfate', 'Product_Vincristine', 'Product_Vincristine Sulfate', 'Product_Virginiamycin', 'Product_Vitamin A Palmitate', 'Product_Vitamin B', 'Product_Vitamin D Nos', 'Product_Vitamins', 'Product_Voglibose', 'Product_Voriconazole', 'Product_Vortioxetine Hydrobromide', 'Product_Warfarin', 'Product_Warfarin Potassium', 'Product_Warfarin Sodium', 'Product_Water', 'Product_Xanthinol Niacinate', 'Product_Xantofyl Palmitate', 'Product_Xipamide', 'Product_Xylitol', 'Product_Xylometazoline', 'Product_Xylometazoline Hydrochloride', 'Product_Yeast', 'Product_Zeaxanthin', 'Product_Zinc', 'Product_Zinc Sulfate', 'Product_Ziprasidone', 'Product_Zirconium Silicate', 'Product_Zoledronic Acid', 'Product_Zolpidem Tartrate', 'Product_Zopiclone', 'Product_Zuclopenthixol']
print(len(second_model_features))
second_model_predictions = ['Reaction_Abdominal Discomfort', 'Reaction_Abdominal Distension', 'Reaction_Abdominal Pain', 'Reaction_Abdominal Pain Upper', 'Reaction_Abnormal Behaviour', 'Reaction_Accident', 'Reaction_Accidental Exposure To Product', 'Reaction_Accidental Overdose', 'Reaction_Accidental Underdose', 'Reaction_Acute Kidney Injury', 'Reaction_Acute Myocardial Infarction', 'Reaction_Adverse Drug Reaction', 'Reaction_Adverse Event', 'Reaction_Agitation', 'Reaction_Alanine Aminotransferase Increased', 'Reaction_Alopecia', 'Reaction_Altered State Of Consciousness', 'Reaction_Amnesia', 'Reaction_Anaemia', 'Reaction_Anger', 'Reaction_Angina Pectoris', 'Reaction_Angioedema', 'Reaction_Angiopathy', 'Reaction_Ankle Fracture', 'Reaction_Anti-Insulin Antibody Positive', 'Reaction_Anxiety', 'Reaction_Aphasia', 'Reaction_Appendicitis', 'Reaction_Appendicolith', 'Reaction_Arrhythmia', 'Reaction_Arterial Occlusive Disease', 'Reaction_Arteriosclerosis', 'Reaction_Arteriosclerosis Coronary Artery', 'Reaction_Arthralgia', 'Reaction_Arthritis', 'Reaction_Arthropathy', 'Reaction_Ascites', 'Reaction_Aspartate Aminotransferase Increased', 'Reaction_Asthenia', 'Reaction_Asthma', 'Reaction_Atrial Fibrillation', 'Reaction_Back Disorder', 'Reaction_Back Pain', 'Reaction_Bacterial Infection', 'Reaction_Balance Disorder', 'Reaction_Bedridden', 'Reaction_Bladder Cancer', 'Reaction_Bladder Disorder', 'Reaction_Blindness', 'Reaction_Blindness Unilateral', 'Reaction_Blister', 'Reaction_Blood Cholesterol Increased', 'Reaction_Blood Creatinine Increased', 'Reaction_Blood Glucose Abnormal', 'Reaction_Blood Glucose Decreased', 'Reaction_Blood Glucose Fluctuation', 'Reaction_Blood Glucose Increased', 'Reaction_Blood Phosphorus Increased', 'Reaction_Blood Potassium Decreased', 'Reaction_Blood Potassium Increased', 'Reaction_Blood Pressure Abnormal', 'Reaction_Blood Pressure Decreased', 'Reaction_Blood Pressure Increased', 'Reaction_Blood Sodium Decreased', 'Reaction_Blood Triglycerides Increased', 'Reaction_Blood Urea Increased', 'Reaction_Blood Uric Acid Increased', 'Reaction_Body Height Decreased', 'Reaction_Body Temperature Increased', 'Reaction_Bradycardia', 'Reaction_Brain Injury', 'Reaction_Breast Cancer', 'Reaction_Bronchitis', 'Reaction_Burning Sensation', 'Reaction_C-Reactive Protein Increased', 'Reaction_Cardiac Arrest', 'Reaction_Cardiac Disorder', 'Reaction_Cardiac Failure', 'Reaction_Cardiac Failure Congestive', 'Reaction_Cardiac Operation', 'Reaction_Cardiac Pacemaker Insertion', 'Reaction_Cardio-Respiratory Arrest', 'Reaction_Cardiogenic Shock', 'Reaction_Cardiovascular Disorder', 'Reaction_Carotid Artery Occlusion', 'Reaction_Carpal Tunnel Syndrome', 'Reaction_Cataract', 'Reaction_Cataract Operation', 'Reaction_Cellulitis', 'Reaction_Cerebral Haemorrhage', 'Reaction_Cerebral Infarction', 'Reaction_Cerebrovascular Accident', 'Reaction_Chest Discomfort', 'Reaction_Chest Pain', 'Reaction_Chills', 'Reaction_Cholelithiasis', 'Reaction_Cholestasis', 'Reaction_Chronic Kidney Disease', 'Reaction_Chronic Obstructive Pulmonary Disease', 'Reaction_Circulatory Collapse', 'Reaction_Circumstance Or Information Capable Of Leading To Medication Error', 'Reaction_Cognitive Disorder', 'Reaction_Cold Sweat', 'Reaction_Colon Cancer', 'Reaction_Coma', 'Reaction_Completed Suicide', 'Reaction_Complication Associated With Device', 'Reaction_Condition Aggravated', 'Reaction_Confusional State', 'Reaction_Conjunctivitis', 'Reaction_Constipation', 'Reaction_Contraindicated Product Administered', 'Reaction_Contusion', 'Reaction_Coronary Arterial Stent Insertion', 'Reaction_Coronary Artery Bypass', 'Reaction_Coronary Artery Disease', 'Reaction_Coronary Artery Occlusion', 'Reaction_Cough', 'Reaction_Covid-19', 'Reaction_Covid-19 Pneumonia', 'Reaction_Crying', 'Reaction_Cystitis', 'Reaction_Deafness', 'Reaction_Deafness Unilateral', 'Reaction_Death', 'Reaction_Decreased Appetite', 'Reaction_Decubitus Ulcer', 'Reaction_Deep Vein Thrombosis', 'Reaction_Dehydration', 'Reaction_Delirium', 'Reaction_Dementia', "Reaction_Dementia Alzheimer's Type", 'Reaction_Depressed Level Of Consciousness', 'Reaction_Depressed Mood', 'Reaction_Depression', 'Reaction_Dermatitis', 'Reaction_Device Breakage', 'Reaction_Device Defective', 'Reaction_Device Delivery System Issue', 'Reaction_Device Failure', 'Reaction_Device Issue', 'Reaction_Device Leakage', 'Reaction_Device Malfunction', 'Reaction_Device Mechanical Issue', 'Reaction_Device Operational Issue', 'Reaction_Device Use Issue', 'Reaction_Diabetes Mellitus', 'Reaction_Diabetes Mellitus Inadequate Control', 'Reaction_Diabetic Coma', 'Reaction_Diabetic Complication', 'Reaction_Diabetic Foot', 'Reaction_Diabetic Ketoacidosis', 'Reaction_Diabetic Metabolic Decompensation', 'Reaction_Diabetic Nephropathy', 'Reaction_Diabetic Neuropathy', 'Reaction_Diabetic Retinopathy', 'Reaction_Dialysis', 'Reaction_Diarrhoea', 'Reaction_Diplopia', 'Reaction_Disability', 'Reaction_Discomfort', 'Reaction_Disorientation', 'Reaction_Disturbance In Attention', 'Reaction_Diverticulitis', 'Reaction_Dizziness', 'Reaction_Drug Dose Omission By Device', 'Reaction_Drug Hypersensitivity', 'Reaction_Drug Ineffective', 'Reaction_Drug Interaction', 'Reaction_Drug Intolerance', 'Reaction_Dry Eye', 'Reaction_Dry Mouth', 'Reaction_Dry Skin', 'Reaction_Dysarthria', 'Reaction_Dysgeusia', 'Reaction_Dyskinesia', 'Reaction_Dyspepsia', 'Reaction_Dysphagia', 'Reaction_Dysphonia', 'Reaction_Dyspnoea', 'Reaction_Dyspnoea Exertional', 'Reaction_Dysstasia', 'Reaction_Dysuria', 'Reaction_Eating Disorder', 'Reaction_Eczema', 'Reaction_Electrolyte Imbalance', 'Reaction_Emphysema', 'Reaction_Encephalopathy', 'Reaction_End Stage Renal Disease', 'Reaction_Eosinophilia', 'Reaction_Eosinophilic Granulomatosis With Polyangiitis', 'Reaction_Epistaxis', 'Reaction_Erythema', 'Reaction_Expired Product Administered', 'Reaction_Exposure Via Skin Contact', 'Reaction_Extra Dose Administered', 'Reaction_Eye Disorder', 'Reaction_Eye Haemorrhage', 'Reaction_Eye Operation', 'Reaction_Eye Pain', 'Reaction_Eye Swelling', 'Reaction_Fall', 'Reaction_Fatigue', 'Reaction_Fear', 'Reaction_Febrile Neutropenia', 'Reaction_Feeding Disorder', 'Reaction_Feeling Abnormal', 'Reaction_Feeling Cold', 'Reaction_Feeling Hot', 'Reaction_Feeling Jittery', 'Reaction_Femur Fracture', 'Reaction_Fibromyalgia', 'Reaction_Flatulence', 'Reaction_Fluid Retention', 'Reaction_Flushing', 'Reaction_Foot Fracture', 'Reaction_Forced Expiratory Volume Decreased', 'Reaction_Fracture', 'Reaction_Frustration Tolerance Decreased', 'Reaction_Full Blood Count Abnormal', 'Reaction_Fungal Infection', 'Reaction_Gait Disturbance', 'Reaction_Gait Inability', 'Reaction_Gallbladder Disorder', 'Reaction_Gangrene', 'Reaction_Gastrointestinal Disorder', 'Reaction_Gastrointestinal Haemorrhage', 'Reaction_Gastrooesophageal Reflux Disease', 'Reaction_General Physical Health Deterioration', 'Reaction_Glaucoma', 'Reaction_Glycosylated Haemoglobin Abnormal', 'Reaction_Glycosylated Haemoglobin Decreased', 'Reaction_Glycosylated Haemoglobin Increased', 'Reaction_Gout', 'Reaction_Haematochezia', 'Reaction_Haematoma', 'Reaction_Haemoglobin Decreased', 'Reaction_Haemoptysis', 'Reaction_Haemorrhage', 'Reaction_Haemorrhagic Stroke', 'Reaction_Hallucination', 'Reaction_Head Discomfort', 'Reaction_Head Injury', 'Reaction_Headache', 'Reaction_Heart Rate Decreased', 'Reaction_Heart Rate Increased', 'Reaction_Heart Rate Irregular', 'Reaction_Hemiparesis', 'Reaction_Hemiplegia', 'Reaction_Hepatic Cancer', 'Reaction_Hepatic Cirrhosis', 'Reaction_Hepatic Failure', 'Reaction_Hepatic Function Abnormal', 'Reaction_Hepatic Steatosis', 'Reaction_Hernia', 'Reaction_Herpes Zoster', 'Reaction_Hip Arthroplasty', 'Reaction_Hip Fracture', 'Reaction_Hospitalisation', 'Reaction_Hot Flush', 'Reaction_Hunger', 'Reaction_Hyperglycaemia', 'Reaction_Hyperhidrosis', 'Reaction_Hyperkalaemia', 'Reaction_Hyperlipidaemia', 'Reaction_Hyperphosphataemia', 'Reaction_Hypersensitivity', 'Reaction_Hypersomnia', 'Reaction_Hypertension', 'Reaction_Hypoacusis', 'Reaction_Hypoaesthesia', 'Reaction_Hypoglycaemia', 'Reaction_Hypoglycaemia Unawareness', 'Reaction_Hypoglycaemic Coma', 'Reaction_Hypoglycaemic Encephalopathy', 'Reaction_Hypoglycaemic Seizure', 'Reaction_Hypoglycaemic Unconsciousness', 'Reaction_Hypokalaemia', 'Reaction_Hypokinesia', 'Reaction_Hyponatraemia', 'Reaction_Hypophagia', 'Reaction_Hypophosphataemia', 'Reaction_Hypotension', 'Reaction_Hypothermia', 'Reaction_Hypothyroidism', 'Reaction_Hypoxia', 'Reaction_Ill-Defined Disorder', 'Reaction_Illness', 'Reaction_Impaired Healing', 'Reaction_Inappropriate Schedule Of Product Administration', 'Reaction_Incorrect Dose Administered', 'Reaction_Incorrect Dose Administered By Device', 'Reaction_Incorrect Product Administration Duration', 'Reaction_Incorrect Route Of Product Administration', 'Reaction_Increased Appetite', 'Reaction_Infection', 'Reaction_Inflammation', 'Reaction_Influenza', 'Reaction_Influenza Like Illness', 'Reaction_Injection Site Bruising', 'Reaction_Injection Site Discharge', 'Reaction_Injection Site Discolouration', 'Reaction_Injection Site Discomfort', 'Reaction_Injection Site Erythema', 'Reaction_Injection Site Haematoma', 'Reaction_Injection Site Haemorrhage', 'Reaction_Injection Site Hypersensitivity', 'Reaction_Injection Site Induration', 'Reaction_Injection Site Injury', 'Reaction_Injection Site Irritation', 'Reaction_Injection Site Mass', 'Reaction_Injection Site Nodule', 'Reaction_Injection Site Pain', 'Reaction_Injection Site Pruritus', 'Reaction_Injection Site Rash', 'Reaction_Injection Site Reaction', 'Reaction_Injection Site Swelling', 'Reaction_Injection Site Urticaria', 'Reaction_Injury', 'Reaction_Injury Associated With Device', 'Reaction_Insomnia', 'Reaction_Insulin Resistance', 'Reaction_Intentional Dose Omission', 'Reaction_Intentional Overdose', 'Reaction_Intentional Product Misuse', 'Reaction_Intentional Product Use Issue', 'Reaction_Intercepted Product Dispensing Error', 'Reaction_Interstitial Lung Disease', 'Reaction_Intervertebral Disc Protrusion', 'Reaction_Iron Deficiency', 'Reaction_Irritability', 'Reaction_Ischaemic Stroke', 'Reaction_Jaundice', 'Reaction_Joint Injury', 'Reaction_Joint Swelling', 'Reaction_Ketoacidosis', 'Reaction_Kidney Infection', 'Reaction_Knee Arthroplasty', 'Reaction_Knee Operation', 'Reaction_Lacrimation Increased', 'Reaction_Lactic Acidosis', 'Reaction_Leg Amputation', 'Reaction_Lethargy', 'Reaction_Ligament Sprain', 'Reaction_Limb Discomfort', 'Reaction_Limb Injury', 'Reaction_Lip Swelling', 'Reaction_Liquid Product Physical Issue', 'Reaction_Liver Disorder', 'Reaction_Localised Infection', 'Reaction_Loss Of Consciousness', 'Reaction_Loss Of Personal Independence In Daily Activities', 'Reaction_Lower Limb Fracture', 'Reaction_Lower Respiratory Tract Infection', 'Reaction_Lung Disorder', 'Reaction_Lung Neoplasm Malignant', 'Reaction_Lymphoma', 'Reaction_Macular Degeneration', 'Reaction_Macular Oedema', 'Reaction_Malaise', 'Reaction_Medication Error', 'Reaction_Memory Impairment', 'Reaction_Mental Disorder', 'Reaction_Mental Impairment', 'Reaction_Mental Status Changes', 'Reaction_Metabolic Acidosis', 'Reaction_Migraine', 'Reaction_Mobility Decreased', 'Reaction_Movement Disorder', 'Reaction_Multiple Organ Dysfunction Syndrome', 'Reaction_Multiple Use Of Single-Use Product', 'Reaction_Muscle Spasms', 'Reaction_Muscular Weakness', 'Reaction_Musculoskeletal Discomfort', 'Reaction_Musculoskeletal Disorder', 'Reaction_Musculoskeletal Pain', 'Reaction_Musculoskeletal Stiffness', 'Reaction_Myalgia', 'Reaction_Myasthenia Gravis', 'Reaction_Myocardial Infarction', 'Reaction_Myocardial Ischaemia', 'Reaction_Nasal Congestion', 'Reaction_Nasopharyngitis', 'Reaction_Nausea', 'Reaction_Neck Pain', 'Reaction_Needle Issue', 'Reaction_Neoplasm Malignant', 'Reaction_Nephrolithiasis', 'Reaction_Nephropathy', 'Reaction_Nerve Compression', 'Reaction_Nerve Injury', 'Reaction_Nervous System Disorder', 'Reaction_Nervousness', 'Reaction_Neuralgia', 'Reaction_Neuritis', 'Reaction_Neurological Symptom', 'Reaction_Neuropathy Peripheral', 'Reaction_Night Sweats', 'Reaction_No Adverse Event', 'Reaction_Nodule', 'Reaction_Normochromic Normocytic Anaemia', 'Reaction_Obstructive Airways Disorder', 'Reaction_Ocular Hyperaemia', 'Reaction_Oedema', 'Reaction_Oedema Peripheral', 'Reaction_Off Label Use', 'Reaction_Oropharyngeal Pain', 'Reaction_Orthostatic Hypotension', 'Reaction_Osteoarthritis', 'Reaction_Osteomyelitis', 'Reaction_Osteoporosis', 'Reaction_Overdose', 'Reaction_Oxygen Saturation Decreased', 'Reaction_Pain', 'Reaction_Pain In Extremity', 'Reaction_Pallor', 'Reaction_Palpitations', 'Reaction_Pancreatic Carcinoma', 'Reaction_Pancreatic Disorder', 'Reaction_Pancreatitis', 'Reaction_Pancreatitis Acute', 'Reaction_Paraesthesia', 'Reaction_Paralysis', "Reaction_Parkinson's Disease", 'Reaction_Pelvic Fracture', 'Reaction_Pemphigoid', 'Reaction_Peripheral Swelling', 'Reaction_Platelet Count Decreased', 'Reaction_Pleural Effusion', 'Reaction_Pneumonia', 'Reaction_Pneumonia Aspiration', 'Reaction_Pollakiuria', 'Reaction_Poor Peripheral Circulation', 'Reaction_Poor Quality Product Administered', 'Reaction_Post Procedural Complication', 'Reaction_Presyncope', 'Reaction_Product Administration Error', 'Reaction_Product Complaint', 'Reaction_Product Dispensing Error', 'Reaction_Product Dispensing Issue', 'Reaction_Product Dose Omission Issue', 'Reaction_Product Monitoring Error', 'Reaction_Product Odour Abnormal', 'Reaction_Product Prescribing Error', 'Reaction_Product Prescribing Issue', 'Reaction_Product Quality Issue', 'Reaction_Product Storage Error', 'Reaction_Product Use Complaint', 'Reaction_Product Use In Unapproved Indication', 'Reaction_Product Use Issue', 'Reaction_Productive Cough', 'Reaction_Prostate Cancer', 'Reaction_Pruritus', 'Reaction_Psoriasis', 'Reaction_Pulmonary Alveolar Haemorrhage', 'Reaction_Pulmonary Embolism', 'Reaction_Pulmonary Fibrosis', 'Reaction_Pulmonary Oedema', 'Reaction_Pulmonary Vasculitis', 'Reaction_Pyrexia', 'Reaction_Rash', 'Reaction_Rash Erythematous', 'Reaction_Rash Maculo-Papular', 'Reaction_Rash Pruritic', 'Reaction_Renal Cancer', 'Reaction_Renal Disorder', 'Reaction_Renal Failure', 'Reaction_Renal Impairment', 'Reaction_Renal Transplant', 'Reaction_Respiratory Disorder', 'Reaction_Respiratory Failure', 'Reaction_Respiratory Symptom', 'Reaction_Restlessness', 'Reaction_Retinal Detachment', 'Reaction_Retinal Disorder', 'Reaction_Retinal Haemorrhage', 'Reaction_Retinopathy', 'Reaction_Rhabdomyolysis', 'Reaction_Rheumatoid Arthritis', 'Reaction_Rhinorrhoea', 'Reaction_Rib Fracture', 'Reaction_Road Traffic Accident', 'Reaction_Rotator Cuff Syndrome', 'Reaction_Scar', 'Reaction_Sciatica', 'Reaction_Sedation', 'Reaction_Seizure', 'Reaction_Sepsis', 'Reaction_Septic Shock', 'Reaction_Shock', 'Reaction_Shock Hypoglycaemic', 'Reaction_Shoulder Fracture', 'Reaction_Sinusitis', 'Reaction_Skin Cancer', 'Reaction_Skin Discolouration', 'Reaction_Skin Disorder', 'Reaction_Skin Exfoliation', 'Reaction_Skin Laceration', 'Reaction_Skin Lesion', 'Reaction_Skin Ulcer', 'Reaction_Sleep Apnoea Syndrome', 'Reaction_Sleep Disorder', 'Reaction_Sleep Disorder Due To A General Medical Condition', 'Reaction_Sluggishness', 'Reaction_Sneezing', 'Reaction_Somnolence', 'Reaction_Sopor', 'Reaction_Speech Disorder', 'Reaction_Spinal Disorder', 'Reaction_Spinal Fracture', 'Reaction_Spinal Operation', 'Reaction_Spinal Stenosis', 'Reaction_Staphylococcal Infection', 'Reaction_Stent Placement', 'Reaction_Stevens-Johnson Syndrome', 'Reaction_Stress', 'Reaction_Subdural Haematoma', 'Reaction_Suicide Attempt', 'Reaction_Surgery', 'Reaction_Swelling', 'Reaction_Swelling Face', 'Reaction_Swollen Tongue', 'Reaction_Syncope', 'Reaction_Tachycardia', 'Reaction_Taste Disorder', 'Reaction_Therapeutic Product Effect Decreased', 'Reaction_Therapeutic Product Effect Incomplete', 'Reaction_Therapeutic Response Decreased', 'Reaction_Thinking Abnormal', 'Reaction_Thirst', 'Reaction_Thrombocytopenia', 'Reaction_Thrombosis', 'Reaction_Thyroid Disorder', 'Reaction_Tinnitus', 'Reaction_Toe Amputation', 'Reaction_Total Lung Capacity Abnormal', 'Reaction_Toxic Epidermal Necrolysis', 'Reaction_Toxicity To Various Agents', 'Reaction_Transient Ischaemic Attack', 'Reaction_Treatment Noncompliance', 'Reaction_Tremor', 'Reaction_Ulcer', 'Reaction_Underdose', 'Reaction_Unevaluable Event', 'Reaction_Unresponsive To Stimuli', 'Reaction_Upper Limb Fracture', 'Reaction_Urinary Incontinence', 'Reaction_Urinary Retention', 'Reaction_Urinary Tract Infection', 'Reaction_Urticaria', 'Reaction_Vascular Graft', 'Reaction_Vasculitis', 'Reaction_Ventricular Fibrillation', 'Reaction_Vertigo', 'Reaction_Viral Infection', 'Reaction_Vision Blurred', 'Reaction_Visual Acuity Reduced', 'Reaction_Visual Impairment', 'Reaction_Vomiting', 'Reaction_Walking Aid User', 'Reaction_Weight Decreased', 'Reaction_Weight Increased', 'Reaction_Wheelchair User', 'Reaction_Wheezing', 'Reaction_Wound', 'Reaction_Wrist Fracture', 'Reaction_Wrong Patient Received Product', 'Reaction_Wrong Product Administered', 'Reaction_Wrong Technique In Device Usage Process', 'Reaction_Wrong Technique In Product Usage Process']
print(len(second_model_predictions))

2050
2051
570
